In [25]:
from IPython.display import HTML, display
display(HTML("<style>.container { width:95% !important; }</style>"))

# Initial study into the code, what can we do

___

In [19]:
# Importing all packages needed for the code blocks below.
import math
from math import gamma
import numpy as np
from numpy import cross
from numpy.core.umath import cos, sin, sqrt
from scipy.integrate import solve_ivp, DenseOutput, OdeSolver
from scipy.integrate._ivp.common import (
    select_initial_step,
    validate_max_step,
    validate_tol,
    warn_extraneous,
)


from astropy.time import Time
from astropy.constants import Constant, G
from astropy.units import Quantity
from astropy import time, units as u
from astropy import _erfa as erfa
from astropy.constants.iau2015 import M_earth, M_jup as M_jupiter, M_sun
from astropy.coordinates.builtin_frames.utils import DEFAULT_OBSTIME, get_jd12
from astropy.coordinates import (
    Angle,
    AffineTransform,
    BaseEclipticFrame,
    BaseRADecFrame,
    CartesianDifferential,
    CartesianRepresentation,
    CartesianRepresentation,
    DynamicMatrixTransform,
    FunctionTransform,
    FunctionTransformWithFiniteDifference,
    frame_transform_graph,
    GCRS,
    get_body,
    get_body_barycentric,
    get_body_barycentric_posvel,
    GeocentricMeanEcliptic,
    HeliocentricEclipticIAU76 as HeliocentricEclipticJ2000,
    HCRS,
    ICRS,
    ITRS,
    TimeAttribute,
    UnitSphericalRepresentation,
)
from astropy.coordinates.baseframe import FrameMeta
from astropy.coordinates.matrix_utilities import (
    matrix_product,
    matrix_transpose,
    rotation_matrix,
)

from astroquery.jplhorizons import Horizons
from astroquery.jplsbdb import SBDB

from plotly.graph_objects import Figure, Layout, Scatter, Scatter3d, Surface
import plotly.colors

from copy import copy

import functools

from collections import namedtuple

from itertools import cycle

from typing import List, Union, Dict

from collections import namedtuple

from enum import Enum

from warnings import warn
import inspect
import warnings

## Below is a rabbit trail, beginning out pulling out the 2D plotting features and trailing backwards untill I recieved all necessary packages

___

### I will comment between each block to "clear" up what each are doing

Except for this initial block below. I am unsure what this is doing currently. Numba is a 'high-performance' python compiler so converting what is on the system to use this maybe?

In [20]:
"""Just-in-time compiler.
Wraps numba if it is available as a module, uses an identity
decorator instead.
"""


def ijit(first=None, *args, **kwargs):
    """Identity JIT, returns unchanged function.
    """

    def _jit(f):
        return f

    if inspect.isfunction(first):
        return first
    else:
        return _jit


try:
    import numba

    jit = numba.njit
except ImportError:
    warnings.warn(
        "Could not import numba package. All poliastro "
        "functions will work properly but the CPU intensive "
        "algorithms will be slow. Consider installing numba to "
        "boost performance."
    )
    jit = ijit

___

### Below, I needed the norm definition in order to move on. But there are a few additional objects that can be of use. Note that before each call, we use @jit which is changing the compiler? By importing this, I'm removing the import as below where "fast" is part of the call. So I've changed ths definition of the function names to include this *fast*. These functions allow for the definition of movement and transformations.

___

In [21]:
#from poliastro.core.util import (
#    circular_velocity as circular_velocity_fast,
#    norm as norm_fast,
#    rotate as rotate_fast,
#)

@jit
def circular_velocity_fast(k, a):
    r"""Compute circular velocity for a given body given thegravitational parameter and the semimajor axis.
    .. math::
       v = \sqrt{\frac{\mu}{a}}
    Parameters
    ----------
    k : float
        Gravitational Parameter
    a : float
        Semimajor Axis
    """
    return np.sqrt(k / a)


@jit
def rotate_fast(vec, angle, axis):
    r"""Rotates a vector around axis x, y or z a Counter ClockWise angle.
    .. math::
        R(\theta) = \begin{bmatrix}
            \cos(\theta) & -\sin(\theta) \\
            \sin(\theta) & \cos(\theta)
            \end{bmatrix}
    Parameters
    ----------
    vec : ndarray
        Dimension 3 vector.
    angle : float
        Angle of rotation (rad).
    axis : int
        Axis to be rotated (X-axis: 0, Y-axis: 1, Z-axis: 2)
    Note
    ----
    This performs a so-called active or alibi transformation: rotates the
    vector while the coordinate system remains unchanged. To do the opposite
    operation (passive or alias transformation) call the function as
    `rotate(vec, ax, -angle)` or use the convenience function `transform`.
    References
    ----------
    http://en.wikipedia.org/wiki/Rotation_matrix#Ambiguities
    """

    assert vec.shape == (3,)

    rot = np.eye(3)
    if axis == 0:
        sl = slice(1, 3, 1)
    elif axis == 1:
        sl = slice(0, 3, 2)
        angle = -angle
    elif axis == 2:
        sl = slice(0, 2, 1)
    else:
        raise ValueError("Invalid axis: must be one of 'x', 'y' or 'z'")
    rot[sl, sl] = np.array(((cos(angle), -sin(angle)), (sin(angle), cos(angle))))

    # np.dot() arguments must all have the same dtype
    return np.dot(rot, vec.astype(rot.dtype))


@jit
def transform(vec, angle, axis):
    """Rotates a coordinate system around axis a positive right-handed angle.
    Note
    -----
    This is a convenience function, equivalent to `rotate(vec, ax, -angle)`.
    Refer to the documentation of that function for further information.
    """
    return rotate(vec, -angle, axis)


@jit
def norm_fast(vec):
    r"""Returns the norm of a 3 dimension vector.
    .. math::
        \left \| \vec{v} \right \| = \sqrt{\sum_{i=1}^{n}v_{i}^2}
    Parameters
    ----------
    vec: ndarray
        Dimension 3 vector.
    Examples
    --------
    >>> vec = np.array([1, 1, 1])
    >>> norm(vec)
    1.7320508075688772
    """
    vec = 1.0 * vec  # Cast to float
    return np.sqrt(vec.dot(vec))


@jit
def rotation_matrix(angle, axis):
    c = cos(angle)
    s = sin(angle)
    if axis == 0:
        return np.array([[1.0, 0.0, 0.0], [0.0, c, -s], [0.0, s, c]])
    elif axis == 1:
        return np.array([[c, 0.0, s], [0.0, 1.0, 0.0], [s, 0.0, c]])
    elif axis == 2:
        return np.array([[c, -s, 0.0], [s, c, 0.0], [0.0, 0.0, 1.0]])
    else:
        raise ValueError("Invalid axis: must be one of 'x', 'y' or 'z'")

___

### This next cell block is calling the movement and transformations defined above (using numba, I'm beginning to feel more confident in saying that). We have the exact same functions, calling the functions above. But in addition this has definitions for calculating ***time range, eccentricity, inclination*** as well as three additional definitions that I am unsure if they will be useful. For sure one is not, it's for limiting the trua anomaly for hyperbolic orbits. We only care about elliptic orbits. The next calculates the eccentricity with regard to critical inclination. I will need to find out what this is. Finally there is a definition for finding the closest value. Without context I am not sure how to interpret this.

___

In [22]:
u.kms = u.km / u.s
u.km3s2 = u.km ** 3 / u.s ** 2


def circular_velocity(k, a):
    """Compute circular velocity for a given body (k) and semimajor axis (a).
    """
    return circular_velocity_fast(k.to(u.km3s2).value, a.to(u.km).value) * u.kms


def rotate(vector, angle, axis="z"):
    """Rotates a vector around axis a right-handed positive angle.
    This is just a convenience function around
    :py:func:`astropy.coordinates.matrix_utilities.rotation_matrix`.
    Parameters
    ----------
    vector : ~astropy.units.Quantity
        Dimension 3 vector.
    angle : ~astropy.units.Quantity
        Angle of rotation.
    axis : str, optional
        Either 'x', 'y' or 'z'.
    Note
    -----
    This performs a so-called *active* or *alibi* transformation: rotates the
    vector while the coordinate system remains unchanged. To do the opposite
    operation (*passive* or *alias* transformation) call the function as
    ``rotate(vec, ax, -angle, unit)`` or use the convenience function
    :py:func:`transform`, see [1]_.
    References
    ----------
    .. [1] http://en.wikipedia.org/wiki/Rotation_matrix#Ambiguities
    """
    return (
        rotate_fast(vector.value, angle.to(u.rad).value, ["x", "y", "z"].index(axis))
        * vector.unit
    )


def transform(vector, angle, axis="z"):
    """Rotates a coordinate system around axis a positive right-handed angle.
    Note
    -----
    This is a convenience function, equivalent to
    ``rotate(vec, -angle, axis, unit)``.
    Refer to the documentation of :py:func:`rotate` for further information.
    """
    return rotate(vector, -angle, axis)


def norm(vec):
    """Norm of a Quantity vector that respects units.
    Parameters
    ----------
    vec : ~astropy.units.Quantity
        Vector with units.
    """
    return norm_fast(vec.value) * vec.unit


def time_range(start, *, periods=50, spacing=None, end=None, format=None, scale=None):
    """Generates range of astronomical times.
    .. versionadded:: 0.8.0
    Parameters
    ----------
    periods : int, optional
        Number of periods, default to 50.
    spacing : Time or Quantity, optional
        Spacing between periods, optional.
    end : Time or equivalent, optional
        End date.
    Returns
    -------
    Time
        Array of time values.
    """
    start = Time(start, format=format, scale=scale)

    if spacing is not None and end is None:
        result = start + spacing * np.arange(0, periods)

    elif end is not None and spacing is None:
        end = Time(end, format=format, scale=scale)
        result = start + (end - start) * np.linspace(0, 1, periods)

    else:
        raise ValueError("Either 'end' or 'spacing' must be specified")

    return result


@u.quantity_input(a=u.m, inc=u.rad)
def get_eccentricity_critical_argp(attractor, a, inc):
    """Calculates the eccentricity for frozen orbits when the argument of perigee is critical
    Parameters
    ----------
    attractor : Body
        Main attractor.
    a : ~astropy.units.Quantity
        Orbit's semimajor axis
    inc : ~astropy.units.Quantity, optional
         Inclination, default to critical value.
    """
    ecc = -attractor.J3 * attractor.R * np.sin(inc) / 2 / attractor.J2 / a
    return ecc


@u.quantity_input(a=u.m, ecc=u.one)
def get_inclination_critical_argp(attractor, a, ecc):
    """Calculates the inclination for frozen orbits when the argument of perigee is critical and the eccentricity is given
    Parameters
    ----------
    attractor : Body
        Main attractor.
    a : ~astropy.units.Quantity
        Orbit's semimajor axis
    ecc : ~astropy.units.Quantity, optional
         Eccentricity
    """
    inc = np.arcsin(-ecc * a * attractor.J2 * 2 / attractor.R / attractor.J3) * u.rad
    return inc


@u.quantity_input(ecc=u.one)
def get_eccentricity_critical_inc(ecc=None):
    """Calculates the eccentricity when a frozen orbit has critical inclination
    If ecc is None we set an arbitrary value which is the Moon ecc because it seems reasonable
    Parameters
    ----------
    ecc: : ~astropy.units.Quantity, optional
        Eccentricity, or None if it was not defined
    """
    moon_ecc = 0.0549 * u.one
    ecc = moon_ecc if ecc is None else ecc
    return ecc


@u.quantity_input(value=u.rad, values=u.rad)
def find_closest_value(value, values):
    """Calculates the closest value in the given values
    Parameters
    ----------
    value : ~astropy.units.Quantity
    values : ~astropy.units.Quantity
    """
    index = np.abs(np.asarray(values) * u.rad - value).argmin()
    return values[index]


@u.quantity_input(ecc=u.one)
def hyp_nu_limit(ecc, r_max_ratio=np.inf):
    r"""Limit true anomaly for hyperbolic orbits.
    Parameters
    ----------
    ecc : ~astropy.units.Quantity
        Eccentricity, should be larger than 1.
    r_max_ratio : float, optional
        Value of :math:`r_{\text{max}} / p` for this angle, default to infinity.
    """
    return np.arccos(-(1 - 1 / r_max_ratio) / ecc)

___

### Below we are actually getting to some of the plotting options. These initial plotting options are not exactly what we're looking for. Generate label might be the most useful for customizing plots at the end. But note the "body colors" defined below. They don't use these here from what I can see, but if we want to vary the color scheme it looks like we can.

___

In [23]:
BODY_COLORS = {"Sun": "#ffcc00", "Earth": "#204a87", "Jupiter": "#ba3821"}


def generate_label(orbit, label):
    orbit.epoch.out_subfmt = "date_hm"
    label_ = f"{orbit.epoch.iso}"
    if label:
        label_ += f" ({label})"

    return label_


def generate_sphere(radius, center, num=20):
    u1 = np.linspace(0, 2 * np.pi, num)
    v1 = u1.copy()
    uu, vv = np.meshgrid(u1, v1)

    x_center, y_center, z_center = center

    xx = x_center + radius * np.cos(uu) * np.sin(vv)
    yy = y_center + radius * np.sin(uu) * np.sin(vv)
    zz = z_center + radius * np.cos(vv)

    return xx, yy, zz


def generate_circle(radius, center, num=500):
    u1 = np.linspace(0, 2 * np.pi, num)
    x_center, y_center, z_center = center

    xx = x_center + radius * np.cos(u1)
    yy = y_center + radius * np.sin(u1)

    return xx, yy

___

### As noted in the initial comments below, this next block is the parent class for the 2D and 3D plotting options.

#### There is a lot going on below that I don't fully understand. It seems to take parameters/objects and creating them as a class? This is an area of coding I'm not familiar with.

___

In [24]:
class BaseOrbitPlotter:
    """
    Parent Class for the 2D and 3D OrbitPlotter Classes based on Plotly.
    """

    def __init__(self, figure=None):
        self._figure = figure or Figure()
        self._layout = None

        self._trajectories = []  # type: List[Trajectory]

        self._attractor = None
        self._attractor_radius = np.inf * u.km

        self._color_cycle = cycle(plotly.colors.DEFAULT_PLOTLY_COLORS)

    @property
    def trajectories(self):
        return self._trajectories

    def _set_attractor(self, attractor):
        if self._attractor is None:
            self._attractor = attractor
        elif attractor is not self._attractor:
            raise NotImplementedError(
                f"Attractor has already been set to {self._attractor.name}."
            )

    def set_attractor(self, attractor):
        """Sets plotting attractor.
        Parameters
        ----------
        attractor : ~poliastro.bodies.Body
            Central body.
        """
        self._set_attractor(attractor)

    def _redraw_attractor(self):
        # Select a sensible value for the radius: realistic for low orbits,
        # visible for high and very high orbits
        min_radius = min(
            [
                trajectory.represent_as(CartesianRepresentation).norm().min() * 0.15
                for trajectory, _, _, _ in self._trajectories
            ]
            or [0 * u.m]
        )
        radius = max(self._attractor.R.to(u.km), min_radius.to(u.km))
        # TODO: Remove previously plotted sphere?
        self._plot_sphere(
            radius,
            BODY_COLORS.get(self._attractor.name, "#999999"),
            self._attractor.name,
        )

        self._attractor_radius = radius

    def _plot_point(self, radius, color, name, center=None):
        raise NotImplementedError

    def _plot_sphere(self, radius, color, name, center=None):
        raise NotImplementedError

    def plot_trajectory(self, trajectory, *, label=None, color=None):
        """Plots a precomputed trajectory.
        An attractor must be set first.
        Parameters
        ----------
        trajectory : ~astropy.coordinates.CartesianRepresentation
            Trajectory to plot.
        label : string, optional
        color : string, optional
        """
        if self._attractor is None:
            raise ValueError(
                "An attractor must be set up first, please use "
                "set_attractor(Major_Body) or plot(orbit)."
            )
        else:
            if color is None:
                color = next(self._color_cycle)

            trace = self._plot_trajectory(trajectory, str(label), color, False)

            self._trajectories.append(
                Trajectory(trajectory, None, label, trace.line.color)
            )

        if not self._figure._in_batch_mode:
            return self.show()

    def _plot_trajectory(self, trajectory, label, color, dashed):
        raise NotImplementedError

    def plot(self, orbit, *, label=None, color=None):
        """Plots state and osculating orbit in their plane.
        Parameters
        ----------
        orbit : ~poliastro.twobody.orbit.Orbit
            Orbit to plot.
        label : string, optional
            Label of the orbit.
        color : string, optional
            Color of the line and the position.
        """
        if color is None:
            color = next(self._color_cycle)

        self._set_attractor(orbit.attractor)

        label = generate_label(orbit, label)
        trajectory = orbit.sample()

        trace = self._plot_trajectory(trajectory, label, color, True)

        self._trajectories.append(
            Trajectory(trajectory, orbit.r, label, trace.line.color)
        )

        # Redraw the attractor now to compute the attractor radius
        self._redraw_attractor()

        # Plot required 2D/3D shape in the position of the body
        radius = min(
            self._attractor_radius * 0.5, (norm(orbit.r) - orbit.attractor.R) * 0.5
        )  # Arbitrary thresholds
        self._plot_point(radius, color, label, center=orbit.r)

        if not self._figure._in_batch_mode:
            return self.show()

    def _prepare_plot(self):
        if self._attractor is not None:
            self._redraw_attractor()

        self._figure.layout.update(self._layout)

    def show(self):
        """Shows the plot in the Notebook.
        Updates the layout and returns the underlying figure.
        """
        self._prepare_plot()
        return self._figure

___

### Finally to what is done to plot in 2D!

#### Self explanatory on the the definitions do. Need to try to plot something in 2D now.

___

In [25]:
class OrbitPlotter2D(BaseOrbitPlotter):
    """OrbitPlotter2D class.
    .. versionadded:: 0.9.0
    """

    def __init__(self, figure=None):
        super().__init__(figure)
        self._layout = Layout(
            autosize=True,
            xaxis=dict(title="x (km)", constrain="domain"),
            yaxis=dict(title="y (km)", scaleanchor="x"),
            shapes=[],
        )

        self._frame = None

    def _project(self, rr):
        rr_proj = rr - rr.dot(self._frame[2])[:, None] * self._frame[2]
        x = rr_proj.dot(self._frame[0])
        y = rr_proj.dot(self._frame[1])
        return x, y

    def _plot_point(self, radius, color, name, center=[0, 0, 0] * u.km):
        x_center, y_center = self._project(
            center[None]
        )  # Indexing trick to add one extra dimension

        trace = Scatter(
            x=x_center.to(u.km).value,
            y=y_center.to(u.km).value,
            mode="markers",
            marker=dict(size=10, color=color),
            name=name,
        )
        self._figure.add_trace(trace)

        return trace

    def _plot_sphere(self, radius, color, name, center=[0, 0, 0] * u.km):
        x_center, y_center = self._project(
            center[None]
        )  # Indexing trick to add one extra dimension

        shape = {
            "type": "circle",
            "xref": "x",
            "yref": "y",
            "x0": (x_center[0] - radius).to(u.km).value,
            "y0": (y_center[0] - radius).to(u.km).value,
            "x1": (x_center[0] + radius).to(u.km).value,
            "y1": (y_center[0] + radius).to(u.km).value,
            "opacity": 1,
            "fillcolor": color,
            "line": {"color": color},
        }

        self._layout.shapes += (shape,)

        return shape

    def _plot_trajectory(self, trajectory, label, color, dashed):
        if self._frame is None:
            raise ValueError(
                "A frame must be set up first, please use "
                "set_frame(*orbit.pqw()) or plot(orbit)."
            )

        rr = trajectory.represent_as(CartesianRepresentation).xyz.transpose()
        x, y = self._project(rr)

        trace = Scatter(
            x=x.to(u.km).value,
            y=y.to(u.km).value,
            name=label,
            line=dict(color=color, width=2, dash="dash" if dashed else "solid"),
            hoverinfo="none",  # TODO: Review
            mode="lines",  # Boilerplate
        )
        self._figure.add_trace(trace)

        return trace

    def set_frame(self, p_vec, q_vec, w_vec):
        """Sets perifocal frame.
        Raises
        ------
        ValueError
            If the vectors are not a set of mutually orthogonal unit vectors.
        """
        if self._frame and self.trajectories:
            raise NotImplementedError(
                "OrbitPlotter2D does not support reprojecting yet"
            )

        if not np.allclose([norm(v) for v in (p_vec, q_vec, w_vec)], 1):
            raise ValueError("Vectors must be unit.")
        elif not np.allclose([p_vec.dot(q_vec), q_vec.dot(w_vec), w_vec.dot(p_vec)], 0):
            raise ValueError("Vectors must be mutually orthogonal.")
        else:
            self._frame = p_vec, q_vec, w_vec

    def plot(self, orbit, *, label=None, color=None):
        """Plots state and osculating orbit in their plane.
        Parameters
        ----------
        orbit : ~poliastro.twobody.orbit.Orbit
            Orbit to plot.
        label : string, optional
            Label of the orbit.
        color : string, optional
            Color of the line and the position.
        """
        if not self._frame:
            self.set_frame(*orbit.pqw())

        return super().plot(orbit, label=label, color=color)

___

#### Beginning the plotting (we will see what else I will need to dig up to plot)

##### Already found I need to define ss_earch and ss_mars to plot the below. Which needs the orbit functions. So I am returning to the rabbit hole.

___

___

##### Below I import the orbit.py script (and the additional files needed for this .. it seems there is a lot). I am adding all the necessary import packages to the top of the document and then working through what this is doing.

___

Initially I am Grabbing the necessary pieces from _states for the orbit.py script. But also needs content from .core.elements which needs content itself...

In [26]:
@jit
def _kepler_equation(E, M, ecc):
    return E - ecc * np.sin(E) - M


@jit
def _kepler_equation_prime(E, M, ecc):
    return 1 - ecc * np.cos(E)


@jit
def _kepler_equation_hyper(F, M, ecc):
    return -F + ecc * np.sinh(F) - M


@jit
def _kepler_equation_prime_hyper(F, M, ecc):
    return ecc * np.cosh(F) - 1


@jit
def _kepler_equation_parabolic(D, M, ecc):
    return M_parabolic(ecc, D) - M


@jit
def _kepler_equation_prime_parabolic(D, M, ecc):
    return M_parabolic_prime(ecc, D)


@jit
def M_parabolic(ecc, D, tolerance=1e-16):
    x = (ecc - 1.0) / (ecc + 1.0) * (D ** 2)
    small_term = False
    S = 0.0
    k = 0
    while not small_term:
        term = (ecc - 1.0 / (2.0 * k + 3.0)) * (x ** k)
        small_term = np.abs(term) < tolerance
        S += term
        k += 1
    return (
        np.sqrt(2.0 / (1.0 + ecc)) * D + np.sqrt(2.0 / (1.0 + ecc) ** 3) * (D ** 3) * S
    )


@jit
def M_parabolic_prime(ecc, D, tolerance=1e-16):
    x = (ecc - 1.0) / (ecc + 1.0) * (D ** 2)
    small_term = False
    S_prime = 0.0
    k = 0
    while not small_term:
        term = (ecc - 1.0 / (2.0 * k + 3.0)) * (2 * k + 3.0) * (x ** k)
        small_term = np.abs(term) < tolerance
        S_prime += term
        k += 1
    return (
        np.sqrt(2.0 / (1.0 + ecc))
        + np.sqrt(2.0 / (1.0 + ecc) ** 3) * (D ** 2) * S_prime
    )


@jit
def newton(regime, x0, args=(), tol=1.48e-08, maxiter=50):
    p0 = 1.0 * x0
    for iter in range(maxiter):
        if regime == "parabolic":
            fval = _kepler_equation_parabolic(p0, *args)
            fder = _kepler_equation_prime_parabolic(p0, *args)
        elif regime == "hyperbolic":
            fval = _kepler_equation_hyper(p0, *args)
            fder = _kepler_equation_prime_hyper(p0, *args)
        else:
            fval = _kepler_equation(p0, *args)
            fder = _kepler_equation_prime(p0, *args)

        newton_step = fval / fder
        p = p0 - newton_step
        if abs(p - p0) < tol:
            return p
        p0 = p
    return 1.0


@jit
def D_to_nu_fast(D):
    r"""True anomaly from parabolic eccentric anomaly.
    .. math::
        \nu = 2 \cdot \arctan{(D)}
    Parameters
    ----------
    D : ~astropy.units.Quantity
        Eccentric anomaly.
    Returns
    -------
    nu : ~astropy.units.Quantity
        True anomaly.
    Note
    ----
    Taken from Farnocchia, Davide, Davide Bracali Cioci, and Andrea Milani.
    "Robust resolution of Kepler’s equation in all eccentricity regimes."
    Celes
    """

    return 2.0 * np.arctan(D)


@jit
def nu_to_D_fast(nu):
    r"""Parabolic eccentric anomaly from true anomaly.
    .. math::
        D = \tan{\frac{\nu}{2}}
    Parameters
    ----------
    nu : ~astropy.units.Quantity
        True anomaly.
    Returns
    -------
    D : ~astropy.units.Quantity
        Hyperbolic eccentric anomaly.
    Note
    ----
    Taken from Farnocchia, Davide, Davide Bracali Cioci, and Andrea Milani.
    "Robust resolution of Kepler’s equation in all eccentricity regimes."
    Celestial Mechanics and Dynamical Astronomy 116, no. 1 (2013): 21-34.
    """
    return np.tan(nu / 2.0)


@jit
def nu_to_E_fast(nu, ecc):
    r"""Eccentric anomaly from true anomaly.
    .. versionadded:: 0.4.0
    .. math::
        E = 2\arctan{\sqrt{\frac{1-e}{1+e}}\tan{\frac{\nu}{2}}}
    Parameters
    ----------
    nu : ~astropy.units.Quantity
        True anomaly.
    ecc : ~astropy.units.Quantity
        Eccentricity.
    Returns
    -------
    E : ~astropy.units.Quantity
        Eccentric anomaly.
    """

    beta = ecc / (1 + np.sqrt(1 - (ecc ** 2)))
    E = nu - 2 * np.arctan(beta * np.sin(nu) / (1 + beta * np.cos(nu)))
    return E


@jit
def nu_to_F_fast(nu, ecc):
    r"""Hyperbolic eccentric anomaly from true anomaly.
    .. math::
        F = ln{\left ( \frac{\sin{(\nu)}\sqrt{e^{2}-1} + \cos{\nu} + e}{1+e\cos{(\nu)}} \right )}
    Parameters
    ----------
    nu : ~astropy.units.Quantity
        True anomaly.
    ecc : ~astropy.units.Quantity
        Eccentricity (>1).
    Returns
    -------
    F : ~astropy.units.Quantity
        Hyperbolic eccentric anomaly.
    Note
    -----
    Taken from Curtis, H. (2013). *Orbital mechanics for engineering students*. 167
    """
    F = np.log(
        (np.sqrt(ecc + 1) + np.sqrt(ecc - 1) * np.tan(nu / 2))
        / (np.sqrt(ecc + 1) - np.sqrt(ecc - 1) * np.tan(nu / 2))
    )
    return F


@jit
def E_to_nu_fast(E, ecc):
    r"""True anomaly from eccentric anomaly.
    .. versionadded:: 0.4.0
    .. math::
        \nu = 2\arctan{\left ( \sqrt{\frac{1+e}{1-e}}\tan{\frac{E}{2}} \right )}
    Parameters
    ----------
    E : ~astropy.units.Quantity
        Eccentric anomaly.
    ecc : ~astropy.units.Quantity
        Eccentricity.
    Returns
    -------
    nu : ~astropy.units.Quantity
        True anomaly.
    """
    beta = ecc / (1 + np.sqrt((1 - ecc) * (1 + ecc)))
    nu = E + 2 * np.arctan(beta * np.sin(E) / (1 - beta * np.cos(E)))
    return nu


@jit
def F_to_nu_fast(F, ecc):
    """True anomaly from hyperbolic eccentric anomaly.
    Parameters
    ----------
    F : ~astropy.units.Quantity
        Hyperbolic eccentric anomaly.
    ecc : ~astropy.units.Quantity
        Eccentricity (>1).
    Returns
    -------
    nu : ~astropy.units.Quantity
        True anomaly.
    """
    nu = 2 * np.arctan(
        (np.exp(F) * np.sqrt(ecc + 1) - np.sqrt(ecc + 1))
        / (np.exp(F) * np.sqrt(ecc - 1) + np.sqrt(ecc - 1))
    )
    return nu


@jit
def M_to_E_fast(M, ecc):
    """Eccentric anomaly from mean anomaly.
    .. versionadded:: 0.4.0
    Parameters
    ----------
    M : ~astropy.units.Quantity
        Mean anomaly.
    ecc : ~astropy.units.Quantity
        Eccentricity.
    Returns
    -------
    E : ~astropy.units.Quantity
        Eccentric anomaly.
    """
    E = newton("elliptic", M, args=(M, ecc))
    return E


@jit
def M_to_F_fast(M, ecc):
    """Hyperbolic eccentric anomaly from mean anomaly.
    Parameters
    ----------
    M : ~astropy.units.Quantity
        Mean anomaly.
    ecc : ~astropy.units.Quantity
        Eccentricity (>1).
    Returns
    -------
    F : ~astropy.units.Quantity
        Hyperbolic eccentric anomaly.
    """
    F = newton("hyperbolic", np.arcsinh(M / ecc), args=(M, ecc), maxiter=100)
    return F


@jit
def M_to_D_fast(M, ecc):
    """Parabolic eccentric anomaly from mean anomaly.
    Parameters
    ----------
    M : ~astropy.units.Quantity
        Mean anomaly.
    ecc : ~astropy.units.Quantity
        Eccentricity (>1).
    Returns
    -------
    D : ~astropy.units.Quantity
        Parabolic eccentric anomaly.
    """
    B = 3.0 * M / 2.0
    A = (B + (1.0 + B ** 2) ** 0.5) ** (2.0 / 3.0)
    guess = 2 * A * B / (1 + A + A ** 2)
    D = newton("parabolic", guess, args=(M, ecc), maxiter=100)
    return D


@jit
def E_to_M_fast(E, ecc):
    """Mean anomaly from eccentric anomaly.
    .. versionadded:: 0.4.0
    Parameters
    ----------
    E : ~astropy.units.Quantity
        Eccentric anomaly.
    ecc : ~astropy.units.Quantity
        Eccentricity.
    Returns
    -------
    M : ~astropy.units.Quantity
        Mean anomaly.
    """
    M = _kepler_equation(E, 0.0, ecc)
    return M


@jit
def F_to_M_fast(F, ecc):
    """Mean anomaly from eccentric anomaly.
    Parameters
    ----------
    F : ~astropy.units.Quantity
        Hyperbolic eccentric anomaly.
    ecc : ~astropy.units.Quantity
        Eccentricity (>1).
    Returns
    -------
    M : ~astropy.units.Quantity
        Mean anomaly.
    """
    M = _kepler_equation_hyper(F, 0.0, ecc)
    return M


@jit
def D_to_M_fast(D, ecc):
    """Mean anomaly from eccentric anomaly.
    Parameters
    ----------
    D : ~astropy.units.Quantity
        Parabolic eccentric anomaly.
    ecc : ~astropy.units.Quantity
        Eccentricity.
    Returns
    -------
    M : ~astropy.units.Quantity
        Mean anomaly.
    """
    M = _kepler_equation_parabolic(D, 0.0, ecc)
    return M


@jit
def M_to_nu_fast(M, ecc, delta=1e-2):
    """True anomaly from mean anomaly.
    .. versionadded:: 0.4.0
    Parameters
    ----------
    M : ~astropy.units.Quantity
        Mean anomaly.
    ecc : ~astropy.units.Quantity
        Eccentricity.
    delta : float (optional)
        threshold of near-parabolic regime definition (from Davide Farnocchia et al)
    Returns
    -------
    nu : ~astropy.units.Quantity
        True anomaly.
    Examples
    --------
    >>> from numpy import radians, degrees
    >>> degrees(M_to_nu(radians(30.0), 0.06))
    33.67328493021166
    """
    if ecc > 1 + delta:
        F = M_to_F(M, ecc)
        nu = F_to_nu(F, ecc)
    elif ecc < 1 - delta:
        E = M_to_E(M, ecc)
        nu = E_to_nu(E, ecc)
    else:
        D = M_to_D(M, ecc)
        nu = D_to_nu(D)
    return nu


@jit
def nu_to_M_fast(nu, ecc, delta=1e-2):
    """Mean anomaly from true anomaly.
    .. versionadded:: 0.4.0
    Parameters
    ----------
    nu : ~astropy.units.Quantity
        True anomaly.
    ecc : ~astropy.units.Quantity
        Eccentricity.
    delta : float (optional)
        threshold of near-parabolic regime definition (from Davide Farnocchia et al)
    Returns
    -------
    M : ~astropy.units.Quantity
        Mean anomaly.
    """
    if ecc > 1 + delta:
        F = nu_to_F(nu, ecc)
        M = F_to_M(F, ecc)
    elif ecc < 1 - delta:
        E = nu_to_E(nu, ecc)
        M = E_to_M(E, ecc)
    else:
        D = nu_to_D(nu)
        M = D_to_M(D, ecc)
    return M


@jit
def fp_angle_fast(nu, ecc):
    r"""Returns the flight path angle.
    .. math::
        \gamma = \arctan{\frac{e\sin{\theta}}{1 + e\cos{\theta}}}
    Parameters
    ----------
    nu: ~astropy.units.Quantity
        True anomaly
    ecc: ~astropy.units.Quantity
        Eccentricity
    Returns
    fp_angle: ~astropy.units.Quantity
        Flight path angle
    Note
    -----
    Algorithm taken from Vallado 2007, pp. 113.
    """
    return np.arctan2(ecc * np.sin(nu), 1 + ecc * np.cos(nu))

In [27]:
"""Angles and anomalies.
"""
#from poliastro.core.angles import (
#    D_to_M as D_to_M_fast,
#    D_to_nu as D_to_nu_fast,
#    E_to_M as E_to_M_fast,
#    E_to_nu as E_to_nu_fast,
#    F_to_M as F_to_M_fast,
#    F_to_nu as F_to_nu_fast,
#    M_to_D as M_to_D_fast,
#    M_to_E as M_to_E_fast,
#    M_to_F as M_to_F_fast,
#    M_to_nu as M_to_nu_fast,
#    fp_angle as fp_angle_fast,
#    nu_to_D as nu_to_D_fast,
#    nu_to_E as nu_to_E_fast,
#    nu_to_F as nu_to_F_fast,
#    nu_to_M as nu_to_M_fast,
#)


@u.quantity_input(D=u.rad)
def D_to_nu(D):
    """True anomaly from parabolic eccentric anomaly.
    Parameters
    ----------
    D : ~astropy.units.Quantity
        Eccentric anomaly.
    Returns
    -------
    nu : ~astropy.units.Quantity
        True anomaly.
    Notes
    -----
    Taken from Farnocchia, Davide, Davide Bracali Cioci, and Andrea Milani.
    "Robust resolution of Kepler’s equation in all eccentricity regimes."
    Celestial Mechanics and Dynamical Astronomy 116, no. 1 (2013): 21-34.
    """
    return (D_to_nu_fast(D.to(u.rad).value) * u.rad).to(D.unit)


@u.quantity_input(nu=u.rad)
def nu_to_D(nu):
    """Parabolic eccentric anomaly from true anomaly.
    Parameters
    ----------
    nu : ~astropy.units.Quantity
        True anomaly.
    Returns
    -------
    D : ~astropy.units.Quantity
        Hyperbolic eccentric anomaly.
    Notes
    -----
    Taken from Farnocchia, Davide, Davide Bracali Cioci, and Andrea Milani.
    "Robust resolution of Kepler’s equation in all eccentricity regimes."
    Celestial Mechanics and Dynamical Astronomy 116, no. 1 (2013): 21-34.
    """
    return (nu_to_D_fast(nu.to(u.rad).value) * u.rad).to(nu.unit)


@u.quantity_input(nu=u.rad, ecc=u.one)
def nu_to_E(nu, ecc):
    """Eccentric anomaly from true anomaly.
    .. versionadded:: 0.4.0
    Parameters
    ----------
    nu : ~astropy.units.Quantity
        True anomaly.
    ecc : ~astropy.units.Quantity
        Eccentricity.
    Returns
    -------
    E : ~astropy.units.Quantity
        Eccentric anomaly.
    """
    return (nu_to_E_fast(nu.to(u.rad).value, ecc.value) * u.rad).to(nu.unit)


@u.quantity_input(nu=u.rad, ecc=u.one)
def nu_to_F(nu, ecc):
    """Hyperbolic eccentric anomaly from true anomaly.
    Parameters
    ----------
    nu : ~astropy.units.Quantity
        True anomaly.
    ecc : ~astropy.units.Quantity
        Eccentricity (>1).
    Returns
    -------
    F : ~astropy.units.Quantity
        Hyperbolic eccentric anomaly.
    Note
    -----
    Taken from Curtis, H. (2013). *Orbital mechanics for engineering students*. 167
    """
    return (nu_to_F_fast(nu.to(u.rad).value, ecc.value) * u.rad).to(nu.unit)


@u.quantity_input(E=u.rad, ecc=u.one)
def E_to_nu(E, ecc):
    """True anomaly from eccentric anomaly.
    .. versionadded:: 0.4.0
    Parameters
    ----------
    E : ~astropy.units.Quantity
        Eccentric anomaly.
    ecc : ~astropy.units.Quantity
        Eccentricity.
    Returns
    -------
    nu : ~astropy.units.Quantity
        True anomaly.
    """
    return (E_to_nu_fast(E.to(u.rad).value, ecc.value) * u.rad).to(E.unit)


@u.quantity_input(F=u.rad, ecc=u.one)
def F_to_nu(F, ecc):
    """True anomaly from hyperbolic eccentric anomaly.
    Parameters
    ----------
    F : ~astropy.units.Quantity
        Hyperbolic eccentric anomaly.
    ecc : ~astropy.units.Quantity
        Eccentricity (>1).
    Returns
    -------
    nu : ~astropy.units.Quantity
        True anomaly.
    """
    return (F_to_nu_fast(F.to(u.rad).value, ecc.value) * u.rad).to(F.unit)


@u.quantity_input(M=u.rad, ecc=u.one)
def M_to_E(M, ecc):
    """Eccentric anomaly from mean anomaly.
    .. versionadded:: 0.4.0
    Parameters
    ----------
    M : ~astropy.units.Quantity
        Mean anomaly.
    ecc : ~astropy.units.Quantity
        Eccentricity.
    Returns
    -------
    E : ~astropy.units.Quantity
        Eccentric anomaly.
    """
    return (M_to_E_fast(M.to(u.rad).value, ecc.value) * u.rad).to(M.unit)


@u.quantity_input(M=u.rad, ecc=u.one)
def M_to_F(M, ecc):
    """Hyperbolic eccentric anomaly from mean anomaly.
    Parameters
    ----------
    M : ~astropy.units.Quantity
        Mean anomaly.
    ecc : ~astropy.units.Quantity
        Eccentricity (>1).
    Returns
    -------
    F : ~astropy.units.Quantity
        Hyperbolic eccentric anomaly.
    """
    return (M_to_F_fast(M.to(u.rad).value, ecc.value) * u.rad).to(M.unit)


@u.quantity_input(M=u.rad, ecc=u.one)
def M_to_D(M, ecc):
    """Parabolic eccentric anomaly from mean anomaly.
    Parameters
    ----------
    M : ~astropy.units.Quantity
        Mean anomaly.
    ecc : ~astropy.units.Quantity
        Eccentricity (>1).
    Returns
    -------
    D : ~astropy.units.Quantity
        Parabolic eccentric anomaly.
    """
    return (M_to_D_fast(M.to(u.rad).value, ecc.value) * u.rad).to(M.unit)


@u.quantity_input(E=u.rad, ecc=u.one)
def E_to_M(E, ecc):
    """Mean anomaly from eccentric anomaly.
    .. versionadded:: 0.4.0
    Parameters
    ----------
    E : ~astropy.units.Quantity
        Eccentric anomaly.
    ecc : ~astropy.units.Quantity
        Eccentricity.
    Returns
    -------
    M : ~astropy.units.Quantity
        Mean anomaly.
    """
    return (E_to_M_fast(E.to(u.rad).value, ecc.value) * u.rad).to(E.unit)


@u.quantity_input(F=u.rad, ecc=u.one)
def F_to_M(F, ecc):
    """Mean anomaly from eccentric anomaly.
    Parameters
    ----------
    F : ~astropy.units.Quantity
        Hyperbolic eccentric anomaly.
    ecc : ~astropy.units.Quantity
        Eccentricity (>1).
    Returns
    -------
    M : ~astropy.units.Quantity
        Mean anomaly.
    """
    return (F_to_M_fast(F.to(u.rad).value, ecc.value) * u.rad).to(F.unit)


@u.quantity_input(D=u.rad, ecc=u.one)
def D_to_M(D, ecc):
    """Mean anomaly from eccentric anomaly.
    Parameters
    ----------
    D : ~astropy.units.Quantity
        Parabolic eccentric anomaly.
    ecc : ~astropy.units.Quantity
        Eccentricity.
    Returns
    -------
    M : ~astropy.units.Quantity
        Mean anomaly.
    """
    return (D_to_M_fast(D.to(u.rad).value, ecc.value) * u.rad).to(D.unit)


@u.quantity_input(M=u.rad, ecc=u.one)
def M_to_nu(M, ecc, delta=1e-2):
    """True anomaly from mean anomaly.
    .. versionadded:: 0.4.0
    Parameters
    ----------
    M : ~astropy.units.Quantity
        Mean anomaly.
    ecc : ~astropy.units.Quantity
        Eccentricity.
    delta : float (optional)
        threshold of near-parabolic regime definition (from Davide Farnocchia et al)
    Returns
    -------
    nu : ~astropy.units.Quantity
        True anomaly.
    Examples
    --------
    >>> M_to_nu(30.0 * u.deg, 0.06 * u.one)
    <Quantity 33.67328493 deg>
    """
    return (M_to_nu_fast(M.to(u.rad).value, ecc.value, delta) * u.rad).to(M.unit)


@u.quantity_input(nu=u.rad, ecc=u.one)
def nu_to_M(nu, ecc, delta=1e-2):
    """Mean anomaly from true anomaly.
    .. versionadded:: 0.4.0
    Parameters
    ----------
    nu : ~astropy.units.Quantity
        True anomaly.
    ecc : ~astropy.units.Quantity
        Eccentricity.
    delta : float (optional)
        threshold of near-parabolic regime definition (from Davide Farnocchia et al)
    Returns
    -------
    M : ~astropy.units.Quantity
        Mean anomaly.
    """
    return (nu_to_M_fast(nu.to(u.rad).value, ecc.value, delta) * u.rad).to(nu.unit)


@u.quantity_input(nu=u.rad, ecc=u.one)
def fp_angle(nu, ecc):
    """Flight path angle.
    .. versionadded:: 0.4.0
    Parameters
    ----------
    nu : ~astropy.units.Quantity
        True anomaly.
    ecc : ~astropy.units.Quantity
        Eccentricity.
    Note
    -----
    Algorithm taken from Vallado 2007, pp. 113.
    """
    return (fp_angle_fast(nu.to(u.rad).value, ecc.value) * u.rad).to(nu.unit)

In [28]:
from numpy import cross 

"""This module contains a set of functions that can be used
    to convert between different elements that define the orbit
    of a body.
    """

@jit
def rv_pqw(k, p, ecc, nu):
    r"""Returns r and v vectors in perifocal frame.
    .. math::
        \vec{r} = \frac{h^2}{\mu}\frac{1}{1 + e\cos(\theta)}\begin{bmatrix}
        \cos(\theta)\\
        \sin(\theta)\\
        0
        \end{bmatrix} \\\\\\
        \vec{v} = \frac{h^2}{\mu}\begin{bmatrix}
        -\sin(\theta)\\
        e+\cos(\theta)\\
        0
        \end{bmatrix}
    Parameters
    ----------
    k : float
        Standard gravitational parameter (km^3 / s^2).
    p : float
        Semi-latus rectum or parameter (km).
    ecc : float
        Eccentricity.
    nu: float
        True anomaly (rad).
    Returns
    -------
    r: ndarray
        Position. Dimension 3 vector
    v: ndarray
        Velocity. Dimension 3 vector
    Examples
    --------
    >>> from poliastro.constants import GM_earth
    >>> k = GM_earth #Earth gravitational parameter
    >>> ecc = 0.3 #Eccentricity
    >>> h = 60000e6 #Angular momentum of the orbit [m^2]/[s]
    >>> nu = np.deg2rad(120) #True Anomaly [rad]
    >>> p = h**2 / k #Parameter of the orbit
    >>> r, v = rv_pqw(k, p, ecc, nu)
    >>> #Printing the results
    r = [-5312706.25105345  9201877.15251336    0] [m]
    v = [-5753.30180931 -1328.66813933  0] [m]/[s]
    Note
    ----
    These formulas can be checked at Curtis 3rd. Edition, page 110. Also the
    example proposed is 2.11 of Curtis 3rd Edition book.
    """
    r_pqw = (np.array([cos(nu), sin(nu), 0 * nu]) * p / (1 + ecc * cos(nu))).T
    v_pqw = (np.array([-sin(nu), (ecc + cos(nu)), 0]) * sqrt(k / p)).T
    return r_pqw, v_pqw


@jit
def coe_rotation_matrix(inc, raan, argp):
    """Create a rotation matrix for coe transformation
    """
    r = rotation_matrix(raan, 2)
    r = r @ rotation_matrix(inc, 0)
    r = r @ rotation_matrix(argp, 2)
    return r


@jit
def coe2rv(k, p, ecc, inc, raan, argp, nu):
    r"""Converts from classical orbital to state vectors.
    Classical orbital elements are converted into position and velocity
    vectors by `rv_pqw` algorithm. A rotation matrix is applied to position
    and velocity vectors to get them expressed in terms of an IJK basis.
        .. math::
            \begin{align}
                \vec{r}_{IJK} &= [ROT3(-\Omega)][ROT1(-i)][ROT3(-\omega)]\vec{r}_{PQW}
                                   = \left [ \frac{IJK}{PQW} \right ]\vec{r}_{PQW}\\
                \vec{v}_{IJK} &= [ROT3(-\Omega)][ROT1(-i)][ROT3(-\omega)]\vec{v}_{PQW}
                                   = \left [ \frac{IJK}{PQW} \right ]\vec{v}_{PQW}\\
            \end{align}
    Previous rotations (3-1-3) can be expressed in terms of a single rotation matrix:
        .. math::
            \left [ \frac{IJK}{PQW} \right ]
        .. math::
            \begin{bmatrix}
            \cos(\Omega)\cos(\omega) - \sin(\Omega)\sin(\omega)\cos(i) & -\cos(\Omega)\sin(\omega) - \sin(\Omega)\cos(\omega)\cos(i) & \sin(\Omega)\sin(i)\\
            \sin(\Omega)\cos(\omega) + \cos(\Omega)\sin(\omega)\cos(i) & -\sin(\Omega)\sin(\omega) + \cos(\Omega)\cos(\omega)\cos(i) & -\cos(\Omega)\sin(i)\\
            \sin(\omega)\sin(i) & \cos(\omega)\sin(i) & \cos(i)
            \end{bmatrix}
    Parameters
    ----------
    k : float
        Standard gravitational parameter (km^3 / s^2).
    p : float
        Semi-latus rectum or parameter (km).
    ecc : float
        Eccentricity.
    inc : float
        Inclination (rad).
    omega : float
        Longitude of ascending node (rad).
    argp : float
        Argument of perigee (rad).
    nu : float
        True anomaly (rad).
    Returns
    -------
    r_ijk: np.array
        Position vector in basis ijk
    v_ijk: np.array
        Velocity vector in basis ijk
    """
    r_pqw, v_pqw = rv_pqw(k, p, ecc, nu)
    rm = coe_rotation_matrix(inc, raan, argp)

    r_ijk = rm @ r_pqw
    v_ijk = rm @ v_pqw

    return r_ijk, v_ijk


@jit
def coe2mee(p, ecc, inc, raan, argp, nu):
    r"""Converts from classical orbital elements to modified equinoctial
    orbital elements.
    The definition of the modified equinoctial orbital elements is taken from
    [Walker, 1985].
    The modified equinoctial orbital elements are a set of orbital elements that are useful for
    trajectory analysis and optimization. They are valid for circular, elliptic, and hyperbolic
    orbits. These direct modified equinoctial equations exhibit no singularity for zero
    eccentricity and orbital inclinations equal to 0 and 90 degrees. However, two of the
    components are singular for an orbital inclination of 180 degrees.
    .. math::
        \begin{align}
        p &= a(1-e^2) \\
        f &= e\cos(\omega + \Omega) \\
        g &= e\sin(\omega + \Omega) \\
        h &= \tan(\frac{i}{2})\cos(\Omega) \\
        k &= \tan(\frac{i}{2})\sin(\Omega) \\
        L &= \Omega + \omega + \theta \\
        \end{align}
    Parameters
    ----------
    k : float
        Standard gravitational parameter (km^3 / s^2).
    p : float
        Semi-latus rectum or parameter (km).
    ecc : float
        Eccentricity.
    inc : float
        Inclination (rad).
    omega : float
        Longitude of ascending node (rad).
    argp : float
        Argument of perigee (rad).
    nu : float
       True anomaly (rad).
    Returns
    -------
    p: float
        Semi-latus rectum or parameter
    f: float
        Equinoctial parameter f
    g: float
        Equinoctial parameter g
    h: float
        Equinoctial parameter h
    k: float
        Equinoctial parameter k
    L: float
        Longitude
    Note
    -----
    The conversion equations are taken directly from the original paper.
    """
    lonper = raan + argp
    f = ecc * np.cos(lonper)
    g = ecc * np.sin(lonper)
    # TODO: Check polar case (see [Walker, 1985])
    h = np.tan(inc / 2) * np.cos(raan)
    k = np.tan(inc / 2) * np.sin(raan)
    L = lonper + nu
    return p, f, g, h, k, L


@jit
def rv2coe(k, r, v, tol=1e-8):
    r"""Converts from vectors to classical orbital elements.
    1. First the angular momentum is computed:
        .. math::
            \vec{h} = \vec{r} \times \vec{v}
    2. With it the eccentricity can be solved:
        .. math::
            \begin{align}
            \vec{e} &= \frac{1}{\mu}\left [ \left ( v^{2} - \frac{\mu}{r}\right ) \vec{r}  - (\vec{r} \cdot \vec{v})\vec{v} \right ] \\
            e &= \sqrt{\vec{e}\cdot\vec{e}} \\
            \end{align}
    3. The node vector line is solved:
        .. math::
            \begin{align}
            \vec{N} &= \vec{k} \times \vec{h} \\
            N &= \sqrt{\vec{N}\cdot\vec{N}}
            \end{align}
    4. The rigth ascension node is computed:
        .. math::
            \Omega = \left\{ \begin{array}{lcc}
             cos^{-1}{\left ( \frac{N_{x}}{N} \right )} &   if  & N_{y} \geq  0 \\
             \\ 360^{o} -cos^{-1}{\left ( \frac{N_{x}}{N} \right )} &  if & N_{y} < 0 \\
             \end{array}
            \right.
    5. The argument of perigee:
        .. math::
            \omega  = \left\{ \begin{array}{lcc}
             cos^{-1}{\left ( \frac{\vec{N}\vec{e}}{Ne} \right )} &   if  & e_{z} \geq  0 \\
             \\ 360^{o} -cos^{-1}{\left ( \frac{\vec{N}\vec{e}}{Ne} \right )} &  if & e_{z} < 0 \\
             \end{array}
            \right.
    6. And finally the true anomaly:
        .. math::
            \nu  = \left\{ \begin{array}{lcc}
             cos^{-1}{\left ( \frac{\vec{e}\vec{r}}{er} \right )} &   if  & v_{r} \geq  0 \\
             \\ 360^{o} -cos^{-1}{\left ( \frac{\vec{e}\vec{r}}{er} \right )} &  if & v_{r} < 0 \\
             \end{array}
            \right.
    Parameters
    ----------
    k : float
        Standard gravitational parameter (km^3 / s^2)
    r : array
        Position vector (km)
    v : array
        Velocity vector (km / s)
    tol : float, optional
        Tolerance for eccentricity and inclination checks, default to 1e-8
    Returns
    -------
    p : float
        Semi-latus rectum of parameter (km)
    ecc: float
        Eccentricity
    inc: float
        Inclination (rad)
    raan: float
        Right ascension of the ascending nod (rad)
    argp: float
        Argument of Perigee (rad)
    nu: float
        True Anomaly (rad)
    Examples
       --------
        >>> from poliastro.constants import GM_earth
        >>> from astropy import units as u
        >>> k = GM_earth.to(u.km ** 3 / u.s ** 2).value  # Earth gravitational parameter
        >>> r = np.array([-6045., -3490., 2500.])
        >>> v = np.array([-3.457, 6.618, 2.533])
        >>> p, ecc, inc, raan, argp, nu = rv2coe(k, r, v)
         >>> print("p:", p, "[km]")
        p: 8530.47436396927 [km]
        >>> print("ecc:", ecc)
        ecc: 0.17121118195416898
        >>> print("inc:", np.rad2deg(inc), "[deg]")
        inc: 153.2492285182475 [deg]
        >>> print("raan:", np.rad2deg(raan), "[deg]")
        raan: 255.27928533439618 [deg]
        >>> print("argp:", np.rad2deg(argp), "[deg]")
        argp: 20.068139973005366 [deg]
        >>> print("nu:", np.rad2deg(nu), "[deg]")
        nu: 28.445804984192122 [deg]
        Note
        ----
        This example is a real exercise from Orbital Mechanics for Engineering
        students by Howard D.Curtis. This exercise is 4.3 of 3rd. Edition, page 200.
        """

    h = cross(r, v)
    n = cross([0, 0, 1], h)
    e = ( ( v.dot(v) - k / ( norm(r) ) ) * r - r.dot(v) * v ) / k
    ecc = norm(e)
    p = h.dot(h) / k
    inc = np.arccos(h[2] / norm(h))
         
    circular = ecc < tol
    equatorial = abs(inc) < tol

    if equatorial and not circular:
        raan = 0
        argp = np.arctan2(e[1], e[0]) % (2 * np.pi)  # Longitude of periapsis
        nu = np.arctan2(h.dot(cross(e, r)) / norm(h), r.dot(e))
    elif not equatorial and circular:
        raan = np.arctan2(n[1], n[0]) % (2 * np.pi)
        argp = 0
        # Argument of latitude
        nu = np.arctan2(r.dot(cross(h, n)) / norm(h), r.dot(n))
    elif equatorial and circular:
        raan = 0
        argp = 0
        nu = np.arctan2(r[1], r[0]) % (2 * np.pi)  # True longitude
    else:
        a = p / (1 - (ecc ** 2))
        ka = k * a
        if a > 0:
            e_se = r.dot(v) / sqrt(ka)
            e_ce = norm(r) * (norm(v) ** 2) / k - 1
            nu = E_to_nu(np.arctan2(e_se, e_ce), ecc)
        else:
            e_sh = r.dot(v) / sqrt(-ka)
            e_ch = norm(r) * (norm(v) ** 2) / k - 1
            nu = F_to_nu(np.log((e_ch + e_sh) / (e_ch - e_sh)) / 2, ecc)

        raan = np.arctan2(n[1], n[0]) % (2 * np.pi)
        px = r.dot(n)
        py = r.dot(cross(h, n)) / norm(h)
        argp = (np.arctan2(py, px) - nu) % (2 * np.pi)

    nu = (nu + np.pi) % (2 * np.pi) - np.pi

    return p, ecc, inc, raan, argp, nu


@jit
def mee2coe(p, f, g, h, k, L):
    r"""Converts from modified equinoctial orbital elements to classical
    orbital elements.
    The definition of the modified equinoctial orbital elements is taken from
    [Walker, 1985].
    .. math::
        \begin{align}
            p &= a(1 - e^{2})\\
            e &= \sqrt{f^{2} + g^{2}}\\
            i &= 2\arctan{(\sqrt{h^{2} + k^{2}})}\\
            raan &= atan2(k, h) \pmod{2\pi}\\
            argp &= (atan2(g, f) - raan) \pmod{2\pi}\\
            nu &= (L - atan2(g, f)) \pmod{2\pi}\\
        \end{align}
    Parameters
    ----------
    p: float
        Semi-latus rectum
    f: float
        Equinoctial parameter p
    g: float
        Equinoctial parameter g
    h: float
        Equinoctial parameter h
    k: float
        Equinoctial parameter k
    L: float
        Longitude
    Returns
    -------
    p: float
        Semi-latus rectum
    ecc: float
        Eccentricity of the orbit
    inc: float
        Inclination of the orbit
    raan: float
        RAAN of orbit
    argp: float
        Argument of the periapsis
    nu: float
        True anomaly
    Note
    -----
    The conversion is always safe because arctan2 works also for 0, 0
    arguments.
    """
    ecc = np.sqrt(f ** 2 + g ** 2)
    inc = 2 * np.arctan(np.sqrt(h ** 2 + k ** 2))
    lonper = np.arctan2(g, f)
    raan = np.arctan2(k, h) % (2 * np.pi)
    argp = (lonper - raan) % (2 * np.pi)
    nu = (L - lonper) % (2 * np.pi)
    return p, ecc, inc, raan, argp, nu

In [29]:
#from poliastro.core.elements import coe2mee, coe2rv, mee2coe, rv2coe


class BaseState(object):
    """Base State class, meant to be subclassed.
    """

    def __init__(self, attractor):
        """Constructor.
        Parameters
        ----------
        attractor : Body
            Main attractor.
        """
        self._attractor = attractor

    @property
    def attractor(self):
        """Main attractor. """
        return self._attractor

    def to_vectors(self):
        """Converts to position and velocity vector representation.
        Returns
        -------
        RVState
        """
        raise NotImplementedError

    def to_classical(self):
        """Converts to classical orbital elements representation.
        Returns
        -------
        ClassicalState
        """
        raise NotImplementedError

    def to_equinoctial(self):
        """Converts to modified equinoctial elements representation.
        Returns
        -------
        ModifiedEquinoctialState
        """
        raise NotImplementedError


class ClassicalState(BaseState):
    """State defined by its classical orbital elements.
    """

    def __init__(self, attractor, p, ecc, inc, raan, argp, nu):
        super().__init__(attractor)
        self._p = p
        self._ecc = ecc
        self._inc = inc
        self._raan = raan
        self._argp = argp
        self._nu = nu

    @property
    def p(self):
        """Semilatus rectum. """
        return self._p

    @property
    def ecc(self):
        """Eccentricity. """
        return self._ecc

    @property
    def inc(self):
        """Inclination. """
        return self._inc

    @property
    def raan(self):
        """Right ascension of the ascending node. """
        return self._raan

    @property
    def argp(self):
        """Argument of the perigee. """
        return self._argp

    @property
    def nu(self):
        """True anomaly. """
        return self._nu

    def to_vectors(self):
        """Converts to position and velocity vector representation.
        """
        r, v = coe2rv(
            self.attractor.k.to(u.km ** 3 / u.s ** 2).value,
            self.p.to(u.km).value,
            self.ecc.value,
            self.inc.to(u.rad).value,
            self.raan.to(u.rad).value,
            self.argp.to(u.rad).value,
            self.nu.to(u.rad).value,
        )

        return RVState(self.attractor, r * u.km, v * u.km / u.s)

    def to_classical(self):
        """Converts to classical orbital elements representation.
        """
        return self

    def to_equinoctial(self):
        """Converts to modified equinoctial elements representation.
        """
        p, f, g, h, k, L = coe2mee(
            self.p.to(u.km).value,
            self.ecc.value,
            self.inc.to(u.rad).value,
            self.raan.to(u.rad).value,
            self.argp.to(u.rad).value,
            self.nu.to(u.rad).value,
        )

        return ModifiedEquinoctialState(
            self.attractor,
            p * u.km,
            f * u.rad,
            g * u.rad,
            h * u.rad,
            k * u.rad,
            L * u.rad,
        )


class RVState(BaseState):
    """State defined by its position and velocity vectors.
    """

    def __init__(self, attractor, r, v):
        super().__init__(attractor)
        self._r = r
        self._v = v

    @property
    def r(self):
        """Position vector. """
        return self._r

    @property
    def v(self):
        """Velocity vector. """
        return self._v

    def to_vectors(self):
        """Converts to position and velocity vector representation.
        """
        return self

    def to_classical(self):
        """Converts to classical orbital elements representation.
        """
        (p, ecc, inc, raan, argp, nu) = rv2coe(
            self.attractor.k.to(u.km ** 3 / u.s ** 2).value,
            self.r.to(u.km).value,
            self.v.to(u.km / u.s).value,
        )

        return ClassicalState(
            self.attractor,
            p * u.km,
            ecc * u.one,
            inc * u.rad,
            raan * u.rad,
            argp * u.rad,
            nu * u.rad,
        )


class ModifiedEquinoctialState(BaseState):
    def __init__(self, attractor, p, f, g, h, k, L):
        super().__init__(attractor)
        self._p = p
        self._f = f
        self._g = g
        self._h = h
        self._k = k
        self._L = L

    @property
    def p(self):
        return self._p

    @property
    def f(self):
        return self._f

    @property
    def g(self):
        return self._g

    @property
    def h(self):
        return self._h

    @property
    def k(self):
        return self._k

    @property
    def L(self):
        return self._L

    def to_classical(self):
        p, ecc, inc, raan, argp, nu = mee2coe(
            self.p.to(u.km).value,
            self.f.to(u.rad).value,
            self.g.to(u.rad).value,
            self.h.to(u.rad).value,
            self.k.to(u.rad).value,
            self.L.to(u.rad).value,
        )

        return ClassicalState(
            self.attractor,
            p * u.km,
            ecc * u.one,
            inc * u.rad,
            raan * u.rad,
            argp * u.rad,
            nu * u.rad,
        )

___

#### This cell below is poliastro - bodies.py    -    starting with the necessary routines

___

In [30]:
#from poliastro import constants
# HACK: Constants cannot be pickled
# (see https://github.com/astropy/astropy/issues/9139)
# so we will convert them all to normal Quantities

'''   _init_.py
from .general import (
    J2000,
    J2000_TDB,
    J2000_TT,
    GM_earth,
    GM_jupiter,
    GM_mars,
    GM_mercury,
    GM_moon,
    GM_neptune,
    GM_pluto,
    GM_saturn,
    GM_sun,
    GM_uranus,
    GM_venus,
    H0_earth,
    J2_earth,
    J2_mars,
    J2_sun,
    J2_venus,
    J3_earth,
    J3_mars,
    J3_venus,
    M_earth,
    M_jupiter,
    M_sun,
    R_earth,
    R_jupiter,
    R_mars,
    R_mean_earth,
    R_mean_jupiter,
    R_mean_mars,
    R_mean_mercury,
    R_mean_moon,
    R_mean_neptune,
    R_mean_pluto,
    R_mean_saturn,
    R_mean_uranus,
    R_mean_venus,
    R_mercury,
    R_moon,
    R_neptune,
    R_pluto,
    R_polar_earth,
    R_polar_jupiter,
    R_polar_mars,
    R_polar_mercury,
    R_polar_moon,
    R_polar_neptune,
    R_polar_pluto,
    R_polar_saturn,
    R_polar_uranus,
    R_polar_venus,
    R_saturn,
    R_sun,
    R_uranus,
    R_venus,
    Wdivc_sun,
    rho0_earth,
)
from .rotational_elements import (
    rotational_period_earth,
    rotational_period_jupiter,
    rotational_period_mars,
    rotational_period_mercury,
    rotational_period_moon,
    rotational_period_neptune,
    rotational_period_pluto,
    rotational_period_saturn,
    rotational_period_sun,
    rotational_period_uranus,
    rotational_period_venus,
)

__all__ = [
    "J2000",
    "J2000_TDB",
    "J2000_TT",
    "GM_sun",
    "GM_earth",
    "GM_mercury",
    "GM_venus",
    "GM_mars",
    "GM_jupiter",
    "GM_saturn",
    "GM_uranus",
    "GM_neptune",
    "GM_pluto",
    "GM_moon",
    "M_earth",
    "M_jupiter",
    "M_sun",
    "R_mean_earth",
    "R_mean_mercury",
    "R_mean_venus",
    "R_mean_mars",
    "R_mean_jupiter",
    "R_mean_saturn",
    "R_mean_uranus",
    "R_mean_neptune",
    "R_mean_pluto",
    "R_mean_moon",
    "R_earth",
    "R_mercury",
    "R_venus",
    "R_mars",
    "R_jupiter",
    "R_saturn",
    "R_sun",
    "R_uranus",
    "R_neptune",
    "R_pluto",
    "R_moon",
    "R_polar_earth",
    "R_polar_mercury",
    "R_polar_venus",
    "R_polar_mars",
    "R_polar_jupiter",
    "R_polar_saturn",
    "R_polar_uranus",
    "R_polar_neptune",
    "R_polar_pluto",
    "R_polar_moon",
    "J2_sun",
    "J2_earth",
    "J3_earth",
    "J2_mars",
    "J3_mars",
    "J2_venus",
    "J3_venus",
    "H0_earth",
    "rho0_earth",
    "Wdivc_sun",
    "rotational_period_earth",
    "rotational_period_sun",
    "rotational_period_mercury",
    "rotational_period_venus",
    "rotational_period_moon",
    "rotational_period_mars",
    "rotational_period_jupiter",
    "rotational_period_saturn",
    "rotational_period_uranus",
    "rotational_period_neptune",
    "rotational_period_pluto",
]
'''








"""Astronomical and physics constants.
This module complements constants defined in `astropy.constants`,
with gravitational paremeters and radii.
Note that `GM_jupiter` and `GM_neptune` are both referred to the whole planetary system gravitational parameter.
Unless otherwise specified, gravitational and mass parameters were obtained from:
* Luzum, Brian et al. “The IAU 2009 System of Astronomical Constants: The Report of the IAU Working Group on Numerical
  Standards for Fundamental Astronomy.” Celestial Mechanics and Dynamical Astronomy 110.4 (2011): 293–304.
  Crossref. Web. `DOI: 10.1007/s10569-011-9352-4`_
radii were obtained from:
* Archinal, B. A. et al. “Report of the IAU Working Group on Cartographic Coordinates and Rotational Elements: 2009.”
  Celestial Mechanics and Dynamical Astronomy 109.2 (2010): 101–135. Crossref. Web. `DOI: 10.1007/s10569-010-9320-4`_
.. _`DOI: 10.1007/s10569-011-9352-4`: http://dx.doi.org/10.1007/s10569-011-9352-4
.. _`DOI: 10.1007/s10569-010-9320-4`: http://dx.doi.org/10.1007/s10569-010-9320-4
J2 for the Sun was obtained from:
* https://hal.archives-ouvertes.fr/hal-00433235/document (New values of gravitational moments J2 and J4 deduced
  from helioseismology, Redouane Mecheri et al)
"""

__all__ = [
    "J2000",
    "J2000_TDB",
    "J2000_TT",
    "GM_sun",
    "GM_earth",
    "GM_mercury",
    "GM_venus",
    "GM_mars",
    "GM_jupiter",
    "GM_saturn",
    "GM_uranus",
    "GM_neptune",
    "GM_pluto",
    "GM_moon",
    "M_earth",
    "M_jupiter",
    "M_sun",
    "R_mean_earth",
    "R_mean_mercury",
    "R_mean_venus",
    "R_mean_mars",
    "R_mean_jupiter",
    "R_mean_saturn",
    "R_mean_uranus",
    "R_mean_neptune",
    "R_mean_pluto",
    "R_mean_moon",
    "R_earth",
    "R_mercury",
    "R_venus",
    "R_mars",
    "R_jupiter",
    "R_saturn",
    "R_sun",
    "R_uranus",
    "R_neptune",
    "R_pluto",
    "R_moon",
    "R_polar_earth",
    "R_polar_mercury",
    "R_polar_venus",
    "R_polar_mars",
    "R_polar_jupiter",
    "R_polar_saturn",
    "R_polar_uranus",
    "R_polar_neptune",
    "R_polar_pluto",
    "R_polar_moon",
    "J2_sun",
    "J2_earth",
    "J3_earth",
    "J2_mars",
    "J3_mars",
    "J2_venus",
    "J3_venus",
    "H0_earth",
    "rho0_earth",
    "Wdivc_sun",
]

# See for example USNO Circular 179
J2000_TT = time.Time("J2000", scale="tt")
J2000_TDB = time.Time("J2000", scale="tdb")
J2000 = J2000_TT

GM_sun = Constant(
    "GM_sun",
    "Heliocentric gravitational constant",
    1.32712442099e20,
    "m3 / (s2)",
    0.0000000001e20,
    "IAU 2009 system of astronomical constants",
    system="si",
)

GM_earth = Constant(
    "GM_earth",
    "Geocentric gravitational constant",
    3.986004418e14,
    "m3 / (s2)",
    0.000000008e14,
    "IAU 2009 system of astronomical constants",
    system="si",
)

# Anderson, John D. et al. “The Mass, Gravity Field, and Ephemeris of Mercury.” Icarus 71.3 (1987): 337–349.
# Crossref. Web. DOI: 10.1016/0019-1035(87)90033-9
GM_mercury = Constant(
    "GM_mercury",
    "Mercury gravitational constant",
    2.203209e13,
    "m3 / (s2)",
    0.91,
    "IAU 2009 system of astronomical constants",
    system="si",
)

# Konopliv, A.S., W.B. Banerdt, and W.L. Sjogren. “Venus Gravity: 180th Degree and Order Model.”
# Icarus 139.1 (1999): 3–18. Crossref. Web. DOI: 10.1006/icar.1999.6086
GM_venus = Constant(
    "GM_venus",
    "Venus gravitational constant",
    3.24858592e14,
    "m3 / (s2)",
    0.006,
    "IAU 2009 system of astronomical constants",
    system="si",
)

# Konopliv, Alex S. et al. “A Global Solution for the Mars Static and Seasonal Gravity, Mars Orientation, Phobos and
# Deimos Masses, and Mars Ephemeris.” Icarus 182.1 (2006): 23–50.
# Crossref. Web. DOI: 10.1016/j.icarus.2005.12.025
GM_mars = Constant(
    "GM_mars",
    "Mars gravitational constant",
    4.282837440e13,
    "m3 / (s2)",
    0.00028,
    "IAU 2009 system of astronomical constants",
    system="si",
)

# Jacobson, R. A. et al. “A comprehensive orbit reconstruction for the galileo prime mission in the JS200 system.”
# The Journal of the Astronautical Sciences 48.4 (2000): 495–516.
# Crossref. Web.
GM_jupiter = Constant(
    "GM_jupiter",
    "Jovian system gravitational constant",
    1.2671276253e17,
    "m3 / (s2)",
    2.00,
    "IAU 2009 system of astronomical constants",
    system="si",
)

# Jacobson, R. A. et al. “The Gravity Field of the Saturnian System from Satellite Observations and Spacecraft
# Tracking Data.” The Astronomical Journal 132.6 (2006): 2520–2526.
# Crossref. Web. DOI: 10.1086/508812
GM_saturn = Constant(
    "GM_saturn",
    "Saturn gravitational constant",
    3.79312077e16,
    "m3 / (s2)",
    1.1,
    "IAU 2009 system of astronomical constants",
    system="si",
)

# Jacobson, R. A. et al. “The Masses of Uranus and Its Major Satellites from Voyager Tracking Data and Earth-Based
# Uranian Satellite Data.” The Astronomical Journal 103 (1992): 2068.
# Crossref. Web. DOI: 10.1086/116211
GM_uranus = Constant(
    "GM_uranus",
    "Uranus gravitational constant",
    5.7939393e15,
    "m3 / (s2)",
    13.0,
    "IAU 2009 system of astronomical constants",
    system="si",
)

# Jacobson, R. A. “THE ORBITS OF THE NEPTUNIAN SATELLITES AND THE ORIENTATION OF THE POLE OF NEPTUNE.”
# The Astronomical Journal 137.5 (2009): 4322–4329. Crossref. Web. DOI:
# 10.1088/0004-6256/137/5/4322
GM_neptune = Constant(
    "GM_neptune",
    "Neptunian system gravitational constant",
    6.836527100580397e15,
    "m3 / (s2)",
    10.0,
    "IAU 2009 system of astronomical constants",
    system="si",
)

# Tholen, David J. et al. “MASSES OF NIX AND HYDRA.” The Astronomical Journal 135.3 (2008): 777–784. Crossref. Web.
# DOI: 10.1088/0004-6256/135/3/777
GM_pluto = Constant(
    "GM_pluto",
    "Pluto gravitational constant",
    8.703e11,
    "m3 / (s2)",
    3.7,
    "IAU 2009 system of astronomical constants",
    system="si",
)

# Lemoine, Frank G. et al. “High-Degree Gravity Models from GRAIL Primary Mission Data.”
# Journal of Geophysical Research: Planets 118.8 (2013): 1676–1698.
# Crossref. Web. DOI: 10.1002/jgre.20118
GM_moon = Constant(
    "GM_moon",
    "Moon gravitational constant",
    4.90279981e12,
    "m3 / (s2)",
    0.00000774,
    "Journal of Geophysical Research: Planets 118.8 (2013)",
    system="si",
)

# Archinal, B. A., Acton, C. H., A’Hearn, M. F., Conrad, A., Consolmagno,
# G. J., Duxbury, T., … Williams, I. P. (2018). Report of the IAU Working
# Group on Cartographic Coordinates and Rotational Elements: 2015. Celestial
# Mechanics and Dynamical Astronomy, 130(3). doi:10.1007/s10569-017-9805-5

R_mean_earth = Constant(
    "R_mean_earth",
    "Earth mean radius",
    6.3710084e6,
    "m",
    0.1,
    "IAU Working Group on Cartographic Coordinates and Rotational Elements: 2015",
    system="si",
)

R_mean_mercury = Constant(
    "R_mean_mercury",
    "Mercury mean radius",
    2.4394e6,
    "m",
    100,
    "IAU Working Group on Cartographic Coordinates and Rotational Elements: 2015",
    system="si",
)

R_mean_venus = Constant(
    "R_mean_venus",
    "Venus mean radius",
    6.0518e6,
    "m",
    1000,
    "IAU Working Group on Cartographic Coordinates and Rotational Elements: 2015",
    system="si",
)

R_mean_mars = Constant(
    "R_mean_mars",
    "Mars mean radius",
    3.38950e6,
    "m",
    2000,
    "IAU Working Group on Cartographic Coordinates and Rotational Elements: 2015",
    system="si",
)

R_mean_jupiter = Constant(
    "R_mean_jupiter",
    "Jupiter mean radius",
    6.9911e7,
    "m",
    6000,
    "IAU Working Group on Cartographic Coordinates and Rotational Elements: 2009",
    system="si",
)

R_mean_saturn = Constant(
    "R_mean_saturn",
    "Saturn mean radius",
    5.8232e7,
    "m",
    6000,
    "IAU Working Group on Cartographic Coordinates and Rotational Elements: 2015",
    system="si",
)

R_mean_uranus = Constant(
    "R_mean_uranus",
    "Uranus mean radius",
    2.5362e7,
    "m",
    7000,
    "IAU Working Group on Cartographic Coordinates and Rotational Elements: 2015",
    system="si",
)

R_mean_neptune = Constant(
    "R_mean_neptune",
    "Neptune mean radius",
    2.4622e7,
    "m",
    19000,
    "IAU Working Group on Cartographic Coordinates and Rotational Elements: 2015",
    system="si",
)

R_mean_pluto = Constant(
    "R_mean_pluto",
    "Pluto mean radius",
    1.188e6,
    "m",
    1600,
    "IAU Working Group on Cartographic Coordinates and Rotational Elements: 2015",
    system="si",
)

R_mean_moon = Constant(
    "R_mean_moon",
    "Moon mean radius",
    1.7374e6,
    "m",
    0,
    "IAU Working Group on Cartographic Coordinates and Rotational Elements: 2015",
    system="si",
)

R_sun = Constant(
    "R_sun",
    "Sun equatorial radius",
    6.95700e8,
    "m",
    0,
    "IAU Working Group on Cartographic Coordinates and Rotational Elements: 2015",
    system="si",
)

R_earth = Constant(
    "R_earth",
    "Earth equatorial radius",
    6.3781366e6,
    "m",
    0.1,
    "IAU Working Group on Cartographic Coordinates and Rotational Elements: 2015",
    system="si",
)


R_mercury = Constant(
    "R_mercury",
    "Mercury equatorial radius",
    2.44053e6,
    "m",
    40,
    "IAU Working Group on Cartographic Coordinates and Rotational Elements: 2015",
    system="si",
)

R_venus = Constant(
    "R_venus",
    "Venus equatorial radius",
    6.0518e6,
    "m",
    1000,
    "IAU Working Group on Cartographic Coordinates and Rotational Elements: 2015",
    system="si",
)

R_mars = Constant(
    "R_mars",
    "Mars equatorial radius",
    3.39619e6,
    "m",
    100,
    "IAU Working Group on Cartographic Coordinates and Rotational Elements: 2015",
    system="si",
)

R_jupiter = Constant(
    "R_jupiter",
    "Jupiter equatorial radius",
    7.1492e7,
    "m",
    4000,
    "IAU Working Group on Cartographic Coordinates and Rotational Elements: 2009",
    system="si",
)

R_saturn = Constant(
    "R_saturn",
    "Saturn equatorial radius",
    6.0268e7,
    "m",
    4000,
    "IAU Working Group on Cartographic Coordinates and Rotational Elements: 2015",
    system="si",
)

R_uranus = Constant(
    "R_uranus",
    "Uranus equatorial radius",
    2.5559e7,
    "m",
    4000,
    "IAU Working Group on Cartographic Coordinates and Rotational Elements: 2015",
    system="si",
)

R_neptune = Constant(
    "R_neptune",
    "Neptune equatorial radius",
    2.4764e7,
    "m",
    15000,
    "IAU Working Group on Cartographic Coordinates and Rotational Elements: 2015",
    system="si",
)

R_pluto = Constant(
    "R_pluto",
    "Pluto equatorial radius",
    1.1883e6,
    "m",
    1600,
    "IAU Working Group on Cartographic Coordinates and Rotational Elements: 2015",
    system="si",
)

R_moon = Constant(
    "R_moon",
    "Moon equatorial radius",
    1.7374e6,
    "m",
    0,
    "IAU Working Group on Cartographic Coordinates and Rotational Elements: 2015",
    system="si",
)

R_polar_earth = Constant(
    "R_polar_earth",
    "Earth polar radius",
    6.3567519e6,
    "m",
    0.1,
    "IAU Working Group on Cartographic Coordinates and Rotational Elements: 2015",
    system="si",
)

R_polar_mercury = Constant(
    "R_polar_mercury",
    "Mercury polar radius",
    2.43826e6,
    "m",
    40,
    "IAU Working Group on Cartographic Coordinates and Rotational Elements: 2015",
    system="si",
)

R_polar_venus = Constant(
    "R_polar_venus",
    "Venus polar radius",
    6.0518e6,
    "m",
    1000,
    "IAU Working Group on Cartographic Coordinates and Rotational Elements: 2015",
    system="si",
)

R_polar_mars = Constant(
    "R_polar_mars",
    "Mars polar radius",
    3.376220e6,
    "m",
    100,
    "IAU Working Group on Cartographic Coordinates and Rotational Elements: 2015",
    system="si",
)

R_polar_jupiter = Constant(
    "R_polar_jupiter",
    "Jupiter polar radius",
    6.6854,
    "m",
    10000,
    "IAU Working Group on Cartographic Coordinates and Rotational Elements: 2009",
    system="si",
)

R_polar_saturn = Constant(
    "R_polar_saturn",
    "Saturn polar radius",
    5.4364e7,
    "m",
    10000,
    "IAU Working Group on Cartographic Coordinates and Rotational Elements: 2015",
    system="si",
)

R_polar_uranus = Constant(
    "R_polar_uranus",
    "Uranus polar radius",
    2.4973e7,
    "m",
    20000,
    "IAU Working Group on Cartographic Coordinates and Rotational Elements: 2015",
    system="si",
)

R_polar_neptune = Constant(
    "R_polar_neptune",
    "Neptune polar radius",
    2.4341e7,
    "m",
    30000,
    "IAU Working Group on Cartographic Coordinates and Rotational Elements: 2015",
    system="si",
)

R_polar_pluto = Constant(
    "R_polar_pluto",
    "Pluto polar radius",
    1.1883e6,
    "m",
    1600,
    "IAU Working Group on Cartographic Coordinates and Rotational Elements: 2015",
    system="si",
)

R_polar_moon = Constant(
    "R_polar_moon",
    "Moon polar radius",
    1.7374e6,
    "m",
    0,
    "IAU Working Group on Cartographic Coordinates and Rotational Elements: 2015",
    system="si",
)


J2_sun = Constant(
    "J2_sun",
    "Sun J2 oblateness coefficient",
    2.20e-7,
    "",
    0.01e-7,
    "HAL archives",
    system="si",
)

J2_earth = Constant(
    "J2_earth",
    "Earth J2 oblateness coefficient",
    0.00108263,
    "",
    1,
    "HAL archives",
    system="si",
)

J3_earth = Constant(
    "J3_earth",
    "Earth J3 asymmetry between the northern and southern hemispheres",
    -2.5326613168e-6,
    "",
    1,
    "HAL archives",
    system="si",
)

J2_mars = Constant(
    "J2_mars",
    "Mars J2 oblateness coefficient",
    0.0019555,
    "",
    1,
    "HAL archives",
    system="si",
)

J3_mars = Constant(
    "J3_mars",
    "Mars J3 asymmetry between the northern and southern hemispheres",
    3.1450e-5,
    "",
    1,
    "HAL archives",
    system="si",
)

J2_venus = Constant(
    "J2_venus",
    "Venus J2 oblateness coefficient",
    4.4044e-6,
    "",
    1,
    "HAL archives",
    system="si",
)

J3_venus = Constant(
    "J3_venus",
    "Venus J3 asymmetry between the northern and southern hemispheres",
    -2.1082e-6,
    "",
    1,
    "HAL archives",
    system="si",
)

H0_earth = Constant(
    "H0_earth",
    "Earth H0 atmospheric scale height",
    8_500,
    "m",
    1,
    "de Pater and Lissauer 2010",
    system="si",
)

rho0_earth = Constant(
    "rho0_earth",
    "Earth rho0 atmospheric density prefactor",
    1.3,
    "kg / (m3)",
    1,
    "de Pater and Lissauer 2010",
    system="si",
)

Wdivc_sun = Constant(
    "Wdivc_sun",
    "total radiation power of Sun divided by the speed of light",
    1.0203759306204136e14,
    "kg km / (s2)",
    1,
    "Howard Curtis",
    system="si",
)


rotational_period_earth = Constant(
    "rotational_period_earth",
    "Earth rotational period",
    0.9972698,
    "d",
    0,
    "Vallado, D. and McClain, W. Fundamentals of astrodynamics and applications.",
    system="si",
)

rotational_period_sun = Constant(
    "rotational_period_sun",
    "Sun rotational period",
    25.38,
    "d",
    0,
    "Vallado, D. and McClain, W. Fundamentals of astrodynamics and applications.",
    system="si",
)

rotational_period_mercury = Constant(
    "rotational_period_mercury",
    "Mercury rotational period",
    58.6462,
    "d",
    0,
    "Vallado, D. and McClain, W. Fundamentals of astrodynamics and applications.",
    system="si",
)

rotational_period_venus = Constant(
    "rotational_period_venus",
    "Venus rotational period",
    -243.01,
    "d",
    0,
    "",
    system="si",
)

rotational_period_moon = Constant(
    "rotational_period_moon",
    "Moon rotational period",
    27.32166,
    "d",
    0,
    "Vallado, D. and McClain, W. Fundamentals of astrodynamics and applications.",
    system="si",
)

rotational_period_mars = Constant(
    "rotational_period_mars",
    "Mars rotational period",
    1.02595675,
    "d",
    0,
    "Vallado, D. and McClain, W. Fundamentals of astrodynamics and applications.",
    system="si",
)

rotational_period_jupiter = Constant(
    "rotational_period_jupiter",
    "Jupiter rotational period",
    0.41354,
    "d",
    0,
    "Vallado, D. and McClain, W. Fundamentals of astrodynamics and applications.",
    system="si",
)

rotational_period_saturn = Constant(
    "rotational_period_saturn",
    "Saturn rotational period",
    0.4375,
    "d",
    0,
    "Vallado, D. and McClain, W. Fundamentals of astrodynamics and applications.",
    system="si",
)

rotational_period_uranus = Constant(
    "rotational_period_uranus",
    "Uranus rotational period",
    -0.65,
    "d",
    0,
    "Vallado, D. and McClain, W. Fundamentals of astrodynamics and applications.",
    system="si",
)

rotational_period_neptune = Constant(
    "rotational_period_neptune",
    "Neptune rotational period",
    0.768,
    "d",
    0,
    "",
    system="si",
)

rotational_period_pluto = Constant(
    "rotational_period_pluto",
    "Pluto rotational period",
    -6.3867,
    "d",
    0,
    "Vallado, D. and McClain, W. Fundamentals of astrodynamics and applications.",
    system="si",
)

In [31]:
"""Bodies of the Solar System.
Contains some predefined bodies of the Solar System:
* Sun (☉)
* Earth (♁)
* Moon (☾)
* Mercury (☿)
* Venus (♀)
* Mars (♂)
* Jupiter (♃)
* Saturn (♄)
* Uranus (⛢)
* Neptune (♆)
* Pluto (♇)
and a way to define new bodies (:py:class:`~Body` class).
Data references can be found in :py:mod:`~poliastro.constants`
"""
def _q(c):
    return Quantity(c)


class _Body(
    namedtuple(
        "_Body",
        [
            "parent",
            "k",
            "name",
            "symbol",
            "R",
            "R_polar",
            "R_mean",
            "rotational_period",
            "J2",
            "J3",
            "mass",
        ],
    )
):
    @property
    def angular_velocity(self):
        return (2 * math.pi * u.rad) / (self.rotational_period).to(u.s)

    def __str__(self):
        return f"{self.name} ({self.symbol})"

    def __repr__(self):
        return self.__str__()

    @classmethod
    @u.quantity_input(k=u.km ** 3 / u.s ** 2, R=u.km)
    def from_parameters(cls, parent, k, name, symbol, R, **kwargs):
        return cls(parent, k, name, symbol, R, **kwargs)

    @classmethod
    def from_relative(
        cls, reference, parent=None, k=None, name=None, symbol=None, R=None, **kwargs
    ):
        k = k * reference.k
        R = R * reference.R
        return cls(parent, k, name, symbol, R, **kwargs)


# https://stackoverflow.com/a/16721002/554319
class Body(_Body):
    __slots__ = ()

    def __new__(
        cls,
        parent,
        k,
        name,
        symbol=None,
        R=0 * u.km,
        R_polar=0 * u.km,
        R_mean=0 * u.km,
        rotational_period=0.0 * u.day,
        J2=0.0 * u.one,
        J3=0.0 * u.one,
        mass=None,
    ):
        if mass is None:
            mass = k / G

        return super().__new__(
            cls,
            parent,
            _q(k),
            name,
            symbol,
            _q(R),
            _q(R_polar),
            _q(R_mean),
            _q(rotational_period),
            _q(J2),
            _q(J3),
            _q(mass),
        )


class SolarSystemBody(Body):
    pass


Sun = SolarSystemBody(
    parent=None,
    k=GM_sun,
    name="Sun",
    symbol="\u2609",
    R=R_sun,
    rotational_period=rotational_period_sun,
    J2=_q(J2_sun),
    mass=_q(M_sun),
)


Mercury = SolarSystemBody(
    parent=Sun,
    k=GM_mercury,
    name="Mercury",
    symbol="\u263F",
    R=R_mercury,
    R_mean=R_mean_mercury,
    R_polar=R_polar_mercury,
    rotational_period=rotational_period_mercury,
)

Venus = SolarSystemBody(
    parent=Sun,
    k=GM_venus,
    name="Venus",
    symbol="\u2640",
    R=R_venus,
    R_mean=R_mean_venus,
    R_polar=R_polar_venus,
    rotational_period=rotational_period_venus,
    J2=_q(J2_venus),
    J3=_q(J3_venus),
)
Earth = SolarSystemBody(
    parent=Sun,
    k=GM_earth,
    name="Earth",
    symbol="\u2641",
    R=R_earth,
    R_mean=R_mean_earth,
    R_polar=R_polar_earth,
    rotational_period=rotational_period_earth,
    mass=_q(M_earth),
    J2=_q(J2_earth),
    J3=_q(J3_earth),
)
Mars = SolarSystemBody(
    parent=Sun,
    k=GM_mars,
    name="Mars",
    symbol="\u2642",
    R=R_mars,
    R_mean=R_mean_mars,
    R_polar=R_polar_mars,
    rotational_period=rotational_period_mars,
    J2=_q(J2_mars),
    J3=_q(J3_mars),
)
Jupiter = SolarSystemBody(
    parent=Sun,
    k=GM_jupiter,
    name="Jupiter",
    symbol="\u2643",
    R=R_jupiter,
    R_mean=R_mean_jupiter,
    R_polar=R_polar_jupiter,
    rotational_period=rotational_period_jupiter,
    mass=_q(M_jupiter),
)
Saturn = SolarSystemBody(
    parent=Sun,
    k=GM_saturn,
    name="Saturn",
    symbol="\u2644",
    R=R_saturn,
    R_mean=R_mean_saturn,
    R_polar=R_polar_saturn,
    rotational_period=rotational_period_saturn,
)
Uranus = SolarSystemBody(
    parent=Sun,
    k=GM_uranus,
    name="Uranus",
    symbol="\u26E2",
    R=R_uranus,
    R_mean=R_mean_uranus,
    R_polar=R_polar_uranus,
    rotational_period=rotational_period_uranus,
)
Neptune = SolarSystemBody(
    parent=Sun,
    k=GM_neptune,
    name="Neptune",
    symbol="\u2646",
    R=R_neptune,
    R_mean=R_mean_neptune,
    R_polar=R_polar_neptune,
    rotational_period=rotational_period_neptune,
)

Pluto = SolarSystemBody(
    parent=Sun,
    k=GM_pluto,
    name="Pluto",
    symbol="\u2647",
    R=R_pluto,
    R_mean=R_mean_pluto,
    R_polar=R_polar_pluto,
    rotational_period=rotational_period_pluto,
)


Moon = SolarSystemBody(
    parent=Earth,
    k=GM_moon,
    name="Moon",
    symbol="\u263E",
    R=R_moon,
    R_mean=R_mean_moon,
    R_polar=R_polar_moon,
    rotational_period=rotational_period_moon,
)

___

#### The next cell block is going to import the frames dependancies

___

In [32]:
# init
#from .ecliptic import GeocentricSolarEcliptic, HeliocentricEclipticJ2000
#from .enums import Planes
#from .equatorial import (
#    GCRS,
#    HCRS,
#    ICRS,
#    JupiterICRS,
#    MarsICRS,
#    MercuryICRS,
#    NeptuneICRS,
#    PlutoICRS,
#    SaturnICRS,
#    UranusICRS,
#    VenusICRS,
#)
#from .util import get_frame

#__all__ = [
#    "Planes",
#    "get_frame",
#    "ICRS",
#    "HCRS",
#    "MercuryICRS",
#    "VenusICRS",
#    "GCRS",
#    "MarsICRS",
#    "JupiterICRS",
#    "SaturnICRS",
#    "UranusICRS",
#    "NeptuneICRS",
#    "PlutoICRS",
#    "HeliocentricEclipticJ2000",
#    "GeocentricSolarEcliptic",
#]


# equitorial.py

__all__ = [
    "ICRS",
    "HCRS",
    "MercuryICRS",
    "VenusICRS",
    "GCRS",
    "MarsICRS",
    "JupiterICRS",
    "SaturnICRS",
    "UranusICRS",
    "NeptuneICRS",
    "PlutoICRS",
]


class _PlanetaryICRS(BaseRADecFrame):
    obstime = TimeAttribute(default=DEFAULT_OBSTIME)

    def __new__(cls, *args, **kwargs):
        frame_transform_graph.transform(AffineTransform, cls, ICRS)(cls.to_icrs)
        frame_transform_graph.transform(AffineTransform, ICRS, cls)(cls.from_icrs)
        frame_transform_graph.transform(
            FunctionTransformWithFiniteDifference, cls, cls
        )(cls.self_transform)

        return super().__new__(cls)

    @staticmethod
    def to_icrs(planet_coo, _):
        # this is just an origin translation so without a distance it cannot go ahead
        if isinstance(planet_coo.data, UnitSphericalRepresentation):
            raise u.UnitsError(_NEED_ORIGIN_HINT.format(planet_coo.__class__.__name__))

        if planet_coo.data.differentials:
            bary_sun_pos, bary_sun_vel = get_body_barycentric_posvel(
                planet_coo.body.name, planet_coo.obstime
            )
            bary_sun_pos = bary_sun_pos.with_differentials(
                bary_sun_vel.represent_as(CartesianDifferential)
            )

        else:
            bary_sun_pos = get_body_barycentric(
                planet_coo.body.name, planet_coo.obstime
            )
            bary_sun_vel = None

        return None, bary_sun_pos

    @staticmethod
    def from_icrs(icrs_coo, planet_frame):
        # this is just an origin translation so without a distance it cannot go ahead
        if isinstance(icrs_coo.data, UnitSphericalRepresentation):
            raise u.UnitsError(_NEED_ORIGIN_HINT.format(icrs_coo.__class__.__name__))

        if icrs_coo.data.differentials:
            bary_sun_pos, bary_sun_vel = get_body_barycentric_posvel(
                planet_frame.body.name, planet_frame.obstime
            )
            # Beware! Negation operation is not supported for differentials
            bary_sun_pos = (-bary_sun_pos).with_differentials(
                -bary_sun_vel.represent_as(CartesianDifferential)
            )

        else:
            bary_sun_pos = -get_body_barycentric(
                planet_frame.body.name, planet_frame.obstime
            )
            bary_sun_vel = None

        return None, bary_sun_pos

    @staticmethod
    def self_transform(from_coo, to_frame):
        if np.all(from_coo.obstime == to_frame.obstime):
            return to_frame.realize_frame(from_coo.data)
        else:
            # like CIRS, we do this self-transform via ICRS
            return from_coo.transform_to(ICRS).transform_to(to_frame)


class MercuryICRS(_PlanetaryICRS):
    body = Mercury


class VenusICRS(_PlanetaryICRS):
    body = Venus


class MarsICRS(_PlanetaryICRS):
    body = Mars


class JupiterICRS(_PlanetaryICRS):
    body = Jupiter


class SaturnICRS(_PlanetaryICRS):
    body = Saturn


class UranusICRS(_PlanetaryICRS):
    body = Uranus


class NeptuneICRS(_PlanetaryICRS):
    body = Neptune


class PlutoICRS(_PlanetaryICRS):
    body = Pluto


class MoonICRS(_PlanetaryICRS):
    body = Moon


_NEED_ORIGIN_HINT = (
    "The input {0} coordinates do not have length units. This "
    "probably means you created coordinates with lat/lon but "
    "no distance.  PlanetaryICRS<->ICRS transforms cannot "
    "function in this case because there is an origin shift."
)


# Eecliptic.py


__all__ = [
    "GeocentricSolarEcliptic",
    "GeocentricMeanEcliptic",
    "HeliocentricEclipticJ2000",
]


class GeocentricSolarEcliptic(BaseEclipticFrame):
    """
    This system has its X axis towards the Sun and its Z axis perpendicular to
    the plane of the Earth's orbit around the Sun (positive North). This system
    is fixed with respect to the Earth-Sun line. It is convenient for specifying
    magnetospheric boundaries. It has also been widely adopted as the system for
    representing vector quantities in space physics databases.
    """

    obstime = TimeAttribute(default=DEFAULT_OBSTIME)


@frame_transform_graph.transform(DynamicMatrixTransform, GCRS, GeocentricSolarEcliptic)
def gcrs_to_geosolarecliptic(gcrs_coo, to_frame):

    if not to_frame.obstime.isscalar:
        raise ValueError(
            "To perform this transformation the obstime Attribute must be a scalar."
        )

    _earth_orbit_perpen_point_gcrs = UnitSphericalRepresentation(
        lon=0 * u.deg, lat=(90 * u.deg - _obliquity_rotation_value(to_frame.obstime))
    )

    _earth_detilt_matrix = _make_rotation_matrix_from_reprs(
        _earth_orbit_perpen_point_gcrs, CartesianRepresentation(0, 0, 1)
    )

    sun_pos_gcrs = get_body("sun", to_frame.obstime).cartesian
    earth_pos_gcrs = get_body("earth", to_frame.obstime).cartesian
    sun_earth = sun_pos_gcrs - earth_pos_gcrs

    sun_earth_detilt = sun_earth.transform(_earth_detilt_matrix)

    # Earth-Sun Line in Geocentric Solar Ecliptic Frame
    x_axis = CartesianRepresentation(1, 0, 0)

    rot_matrix = _make_rotation_matrix_from_reprs(sun_earth_detilt, x_axis)

    return matrix_product(rot_matrix, _earth_detilt_matrix)


@frame_transform_graph.transform(DynamicMatrixTransform, GeocentricSolarEcliptic, GCRS)
def geosolarecliptic_to_gcrs(from_coo, gcrs_frame):
    return matrix_transpose(gcrs_to_geosolarecliptic(gcrs_frame, from_coo))


def _obliquity_rotation_value(equinox):
    """
    Function to calculate obliquity of the earth.
    This uses obl06 of erfa.
    """
    jd1, jd2 = get_jd12(equinox, "tt")
    obl = erfa.obl06(jd1, jd2) * u.radian
    return obl.to(u.deg)


def _make_rotation_matrix_from_reprs(start_representation, end_representation):
    """
    Return the matrix for the direct rotation from one representation to a second representation.
    The representations need not be normalized first.
    """
    A = start_representation.to_cartesian()
    B = end_representation.to_cartesian()
    rotation_axis = A.cross(B)
    rotation_angle = -np.arccos(
        A.dot(B) / (A.norm() * B.norm())
    )  # negation is required

    # This line works around some input/output quirks of Astropy's rotation_matrix()
    matrix = np.array(rotation_matrix(rotation_angle, rotation_axis.xyz.value.tolist()))
    return matrix





In [33]:
"""Coordinate frames definitions.
"""

class Planes(Enum):
    EARTH_EQUATOR = "Earth mean Equator and Equinox of epoch (J2000.0)"
    EARTH_ECLIPTIC = "Earth mean Ecliptic and Equinox of epoch (J2000.0)"
    BODY_FIXED = "Rotating body mean Equator and node of date"

In [34]:
# fixes.py

__all__ = [
    "SunFixed",
    "MercuryFixed",
    "VenusFixed",
    "ITRS",
    "MarsFixed",
    "JupiterFixed",
    "SaturnFixed",
    "UranusFixed",
    "NeptuneFixed",
    "PlutoFixed",
]


class _PlanetaryFixed(BaseRADecFrame):
    obstime = TimeAttribute(default=DEFAULT_OBSTIME)

    def __new__(cls, *args, **kwargs):
        frame_transform_graph.transform(FunctionTransform, cls, cls.equatorial)(
            cls.to_equatorial
        )
        frame_transform_graph.transform(FunctionTransform, cls.equatorial, cls)(
            cls.from_equatorial
        )

        return super().__new__(cls)

    @staticmethod
    def to_equatorial(fixed_coo, equatorial_frame):
        assert fixed_coo.body == equatorial_frame.body

        r = fixed_coo.data.xyz
        v = fixed_coo.data.differentials["s"].d_xyz

        ra, dec, W = fixed_coo.rot_elements_at_epoch(fixed_coo.obstime)

        r = transform_vector(r, -W, "z")
        v = transform_vector(v, -W, "z")

        r_trans1 = transform_vector(r, -(90 * u.deg - dec), "x")
        r_trans2 = transform_vector(r_trans1, -(90 * u.deg + ra), "z")

        v_trans1 = transform_vector(v, -(90 * u.deg - dec), "x")
        v_trans2 = transform_vector(v_trans1, -(90 * u.deg + ra), "z")

        icrs_frame_pos_coord, icrs_frame_vel_coord = get_body_barycentric_posvel(
            fixed_coo.body.name, time=fixed_coo.obstime
        )

        r_f = icrs_frame_pos_coord.xyz + r_trans2
        v_f = icrs_frame_vel_coord.xyz + v_trans2

        data = CartesianRepresentation(r_f, differentials=CartesianDifferential(v_f))
        return equatorial_frame.realize_frame(data)

    @staticmethod
    def from_equatorial(equatorial_coo, fixed_frame):
        assert equatorial_coo.body == fixed_frame.body

        r = equatorial_coo.data.xyz
        v = equatorial_coo.data.differentials["s"].d_xyz

        ra, dec, W = fixed_frame.rot_elements_at_epoch(equatorial_coo.obstime)

        icrs_frame_pos_coord, icrs_frame_vel_coord = get_body_barycentric_posvel(
            equatorial_coo.body.name, time=equatorial_coo.obstime
        )

        r_trans1 = r - icrs_frame_pos_coord.xyz
        r_trans2 = transform_vector(r_trans1, (90 * u.deg + ra), "z")
        r_f = transform_vector(r_trans2, (90 * u.deg - dec), "x")

        v_trans1 = v - icrs_frame_vel_coord.xyz
        v_trans2 = transform_vector(v_trans1, (90 * u.deg + ra), "z")
        v_f = transform_vector(v_trans2, (90 * u.deg - dec), "x")

        r_f = transform_vector(r_f, W, "z")
        v_f = transform_vector(v_f, W, "z")

        data = CartesianRepresentation(r_f, differentials=CartesianDifferential(v_f))
        return fixed_frame.realize_frame(data)

    @classmethod
    def rot_elements_at_epoch(cls, epoch):
        """Provides rotational elements at epoch.
        Provides north pole of body and angle to prime meridian.
        Parameters
        ----------
        epoch : ~astropy.time.Time, optional
            Epoch, default to J2000.
        Returns
        -------
        ra, dec, W: tuple (~astropy.units.Quantity)
            Right ascension and declination of north pole, and angle of the prime meridian.
        """
        T = (epoch.tdb - J2000).to(u.day).value / 36525
        d = (epoch.tdb - J2000).to(u.day).value
        return cls._rot_elements_at_epoch(T, d)

    @staticmethod
    def _rot_elements_at_epoch(T, d):
        raise NotImplementedError


class SunFixed(_PlanetaryFixed):
    body = Sun
    equatorial = HCRS

    @staticmethod
    def _rot_elements_at_epoch(T, d):
        ra = 286.13 * u.deg
        dec = 63.87 * u.deg
        W = (84.176 + 14.1844000 * d) * u.deg

        return ra, dec, W


class MercuryFixed(_PlanetaryFixed):
    body = Mercury
    equatorial = MercuryICRS

    @staticmethod
    def _rot_elements_at_epoch(T, d):
        M1 = (174.7910857 + 4.092335 * d) * u.deg
        M2 = (349.5821714 + 8.184670 * d) * u.deg
        M3 = (164.3732571 + 12.277005 * d) * u.deg
        M4 = (339.1643429 + 16.369340 * d) * u.deg
        M5 = (153.9554286 + 20.461675 * d) * u.deg
        ra = (281.0103 - 0.0328 * T) * u.deg
        dec = (61.45 - 0.005 * T) * u.deg
        W = (329.5988 + 6.1385108 * d) * u.deg + (
            0.01067257 * math.sin(M1.to("rad").value)
            - 0.00112309 * math.sin(M2.to("rad").value)
            - 0.00011040 * math.sin(M3.to("rad").value)
            - 0.00002539 * math.sin(M4.to("rad").value)
            - 0.00000571 * math.sin(M5.to("rad").value)
        ) * u.deg

        return ra, dec, W


class VenusFixed(_PlanetaryFixed):
    body = Venus
    equatorial = VenusICRS

    @staticmethod
    def _rot_elements_at_epoch(T, d):
        ra = 272.76 * u.deg
        dec = 67.16 * u.deg
        W = (160.20 - 1.4813688 * d) * u.deg

        return ra, dec, W


class MarsFixed(_PlanetaryFixed):
    body = Mars
    equatorial = MarsICRS

    @staticmethod
    def _rot_elements_at_epoch(T, d):
        M1 = (198.991226 + 19139.4819985 * T) * u.deg
        M2 = (226.292679 + 38280.8511281 * T) * u.deg
        M3 = (249.663391 + 57420.7251593 * T) * u.deg
        M4 = (266.183510 + 76560.6367950 * T) * u.deg
        M5 = (79.398797 + 0.5042615 * T) * u.deg

        ra = (
            317.269202
            - 0.10927547 * T
            + 0.000068 * math.sin(M1.to("rad").value)
            + 0.000238 * math.sin(M2.to("rad").value)
            + 0.000052 * math.sin(M3.to("rad").value)
            + 0.000009 * math.sin(M4.to("rad").value)
            + 0.419057 * math.sin(M5.to("rad").value)
        ) * u.deg

        K1 = (122.433576 + 19139.9407476 * T) * u.deg
        K2 = (43.058401 + 38280.8753272 * T) * u.deg
        K3 = (57.663379 + 57420.7517205 * T) * u.deg
        K4 = (79.476401 + 76560.6495004 * T) * u.deg
        K5 = (166.325722 + 0.5042615 * T) * u.deg

        dec = (
            54.432516
            - 0.05827105 * T
            + 0.000051 * math.cos(K1.to("rad").value)
            + 0.000141 * math.cos(K2.to("rad").value)
            + 0.000031 * math.cos(K3.to("rad").value)
            + 0.000005 * math.cos(K4.to("rad").value)
            + 1.591274 * math.cos(K5.to("rad").value)
        ) * u.deg

        J1 = (129.071773 + 19140.0328244 * T) * u.deg
        J2 = (36.352167 + 38281.0473591 * T) * u.deg
        J3 = (56.668646 + 57420.9295360 * T) * u.deg
        J4 = (67.364003 + 76560.2552215 * T) * u.deg
        J5 = (104.792680 + 95700.4387578 * T) * u.deg
        J6 = (95.391654 + 0.5042615 * T) * u.deg

        W = (
            176.049863
            + 350.891982443297 * d
            + 0.000145 * math.sin(J1.to("rad").value)
            + 0.000157 * math.sin(J2.to("rad").value)
            + 0.000040 * math.sin(J3.to("rad").value)
            + 0.000001 * math.sin(J4.to("rad").value)
            + 0.000001 * math.sin(J5.to("rad").value)
            + 0.584542 * math.sin(J6.to("rad").value)
        ) * u.deg

        return ra, dec, W


class JupiterFixed(_PlanetaryFixed):
    body = Jupiter
    equatorial = JupiterICRS

    @staticmethod
    def _rot_elements_at_epoch(T, d):
        Ja = (99.360714 + 4850.4046 * T) * u.deg
        Jb = (175.895369 + 1191.9605 * T) * u.deg
        Jc = (300.323162 + 262.5475 * T) * u.deg
        Jd = (114.012305 + 6070.2476 * T) * u.deg
        Je = (49.511251 + 64.3000 * T) * u.deg

        ra = (
            268.056595
            - 0.006499 * T
            + 0.000117 * math.sin(Ja.to("rad").value)
            + 0.000938 * math.sin(Jb.to("rad").value)
            + 0.001432 * math.sin(Jc.to("rad").value)
            + 0.000030 * math.sin(Jd.to("rad").value)
            + 0.002150 * math.sin(Je.to("rad").value)
        ) * u.deg
        dec = (
            64.495303
            + 0.002413 * T
            + 0.000050 * math.cos(Ja.to("rad").value)
            + 0.000404 * math.cos(Jb.to("rad").value)
            + 0.000617 * math.cos(Jc.to("rad").value)
            - 0.000013 * math.cos(Jd.to("rad").value)
            + 0.000926 * math.cos(Je.to("rad").value)
        ) * u.deg
        W = (284.95 + 870.536 * d) * u.deg

        return ra, dec, W


class SaturnFixed(_PlanetaryFixed):
    body = Saturn
    equatorial = SaturnICRS

    @staticmethod
    def _rot_elements_at_epoch(T, d):
        ra = (40.589 - 0.036 * T) * u.deg
        dec = (83.537 - 0.004 * T) * u.deg
        W = (38.90 + 810.7939024 * d) * u.deg

        return ra, dec, W


class UranusFixed(_PlanetaryFixed):
    body = Uranus
    equatorial = UranusICRS

    @staticmethod
    def _rot_elements_at_epoch(T, d):
        ra = 257.311 * u.deg
        dec = -15.175 * u.deg
        W = (203.81 - 501.1600928 * d) * u.deg

        return ra, dec, W


class NeptuneFixed(_PlanetaryFixed):
    body = Neptune
    equatorial = NeptuneICRS

    @staticmethod
    def _rot_elements_at_epoch(T, d):
        N = (357.85 + 52.316 * T) * u.deg

        ra = (299.36 + 0.70 * math.sin(N.to("rad").value)) * u.deg
        dec = (43.46 - 0.51 * math.cos(N.to("rad").value)) * u.deg
        W = (249.978 + 541.1397757 * d - 0.48 * math.sin(N.to("rad").value)) * u.deg

        return ra, dec, W


class PlutoFixed(_PlanetaryFixed):
    body = Pluto
    equatorial = PlutoICRS

    @staticmethod
    def _rot_elements_at_epoch(T, d):
        ra = 132.993 * u.deg
        dec = -6.163 * u.deg
        W = (302.695 + 56.3625225 * d) * u.deg

        return ra, dec, W


class MoonFixed(_PlanetaryFixed):
    body = Moon
    equatorial = MoonICRS

    @staticmethod
    def _rot_elements_at_epoch(T, d):
        E1 = (125.045 - 0.0529921 * d) * u.deg
        E2 = (250.089 - 0.1059842 * d) * u.deg
        E3 = (260.008 + 13.0120009 * d) * u.deg
        E4 = (176.625 + 13.3407154 * d) * u.deg
        E5 = (357.529 + 0.9856003 * d) * u.deg
        E6 = (311.589 + 26.4057084 * d) * u.deg
        E7 = (134.963 + 13.0649930 * d) * u.deg
        E8 = (276.617 + 0.3287146 * d) * u.deg
        E9 = (34.226 + 1.7484877 * d) * u.deg
        E10 = (15.134 - 0.1589763 * d) * u.deg
        E11 = (119.743 + 0.0036096 * d) * u.deg
        E12 = (239.961 + 0.1643573 * d) * u.deg
        E13 = (25.053 + 12.9590088 * d) * u.deg

        ra = (
            269.9949
            + 0.0031 * T
            - 3.8787 * math.sin(E1.to("rad").value)
            - 0.1204 * math.sin(E2.to("rad").value)
            + 0.0700 * math.sin(E3.to("rad").value)
            - 0.0172 * math.sin(E4.to("rad").value)
            + 0.0072 * math.sin(E6.to("rad").value)
            - 0.0052 * math.sin(E10.to("rad").value)
            + 0.0043 * math.sin(E13.to("rad").value)
        ) * u.deg
        dec = (
            66.5392
            + 0.0130 * T
            + 1.5419 * math.cos(E1.to("rad").value)
            + 0.0239 * math.cos(E2.to("rad").value)
            - 0.0278 * math.cos(E3.to("rad").value)
            + 0.0068 * math.cos(E4.to("rad").value)
            - 0.0029 * math.cos(E6.to("rad").value)
            + 0.0009 * math.cos(E7.to("rad").value)
            + 0.0008 * math.cos(E10.to("rad").value)
            - 0.0009 * math.cos(E13.to("rad").value)
        ) * u.deg
        W = (
            38.321
            + 13.17635815 * d
            - 1.4e-12 * d ** 2
            + 3.5610 * math.sin(E1.to("rad").value)
            + 0.1208 * math.sin(E2.to("rad").value)
            - 0.0642 * math.sin(E3.to("rad").value)
            + 0.0158 * math.sin(E4.to("rad").value)
            + 0.0252 * math.sin(E5.to("rad").value)
            - 0.0066 * math.sin(E6.to("rad").value)
            - 0.0047 * math.sin(E7.to("rad").value)
            - 0.0046 * math.sin(E8.to("rad").value)
            + 0.0028 * math.sin(E9.to("rad").value)
            + 0.0052 * math.sin(E10.to("rad").value)
            + 0.0040 * math.sin(E11.to("rad").value)
            + 0.0019 * math.sin(E12.to("rad").value)
            - 0.0044 * math.sin(E13.to("rad").value)
        ) * u.deg

        return ra, dec, W

In [35]:
# util.py

_FRAME_MAPPING = {
    Sun: {Planes.EARTH_EQUATOR: HCRS, Planes.EARTH_ECLIPTIC: HeliocentricEclipticJ2000},
    Mercury: {Planes.EARTH_EQUATOR: MercuryICRS, Planes.BODY_FIXED: MercuryFixed},
    Venus: {Planes.EARTH_EQUATOR: VenusICRS, Planes.BODY_FIXED: VenusFixed},
    Earth: {
        Planes.EARTH_EQUATOR: GCRS,
        Planes.EARTH_ECLIPTIC: GeocentricMeanEcliptic,
        Planes.BODY_FIXED: ITRS,
    },
    Mars: {Planes.EARTH_EQUATOR: MarsICRS, Planes.BODY_FIXED: MarsFixed},
    Jupiter: {Planes.EARTH_EQUATOR: JupiterICRS, Planes.BODY_FIXED: JupiterFixed},
    Saturn: {Planes.EARTH_EQUATOR: SaturnICRS, Planes.BODY_FIXED: SaturnFixed},
    Uranus: {Planes.EARTH_EQUATOR: UranusICRS, Planes.BODY_FIXED: UranusFixed},
    Neptune: {Planes.EARTH_EQUATOR: NeptuneICRS, Planes.BODY_FIXED: NeptuneFixed},
    Pluto: {Planes.EARTH_EQUATOR: PlutoICRS, Planes.BODY_FIXED: PlutoFixed},
}  # type: Dict[SolarSystemBody, Dict[Planes, FrameMeta]]


def get_frame(attractor, plane, obstime=J2000):
    """Returns an appropriate reference frame from an attractor and a plane.
    Available planes are Earth equator (parallel to GCRS) and Earth ecliptic.
    The fundamental direction of both is the equinox of epoch (J2000).
    An obstime is needed to properly locate the attractor.
    Parameters
    ----------
    attractor : ~poliastro.bodies.Body
        Body that serves as the center of the frame.
    plane : ~poliastro.frames.Planes
        Fundamental plane of the frame.
    obstime : ~astropy.time.Time
        Time of the frame.
    """
    try:
        frames = _FRAME_MAPPING[attractor]
    except KeyError:
        raise NotImplementedError(
            "Frames for orbits around custom bodies are not yet supported"
        )

    try:
        frame_class = frames[plane]
    except KeyError:
        raise NotImplementedError(
            "A frame with plane {} around body {} is not yet implemented".format(
                plane, attractor
            )
        )

    return frame_class(obstime=obstime)

___

#### Below is the def for the threebody.soi import. We don't want to model a three body system so I am only inputting this definition without the rest of the threebody directory. Hopefully it will be fine.

___

In [36]:
@u.quantity_input(a=u.m)
def laplace_radius(body, a=None):
    """Approximated radius of the Laplace Sphere of Influence (SOI) for a body.
    Parameters
    ----------
    body : `~poliastro.bodies.Body`
           Astronomical body which the SOI's radius is computed for.
    a : float, optional
        Semimajor axis of the body's orbit, default to None (will be computed from ephemerides).
    Returns
    -------
    astropy.units.quantity.Quantity
        Approximated radius of the Sphere of Influence (SOI) [m]
    """
    # Compute semimajor axis at epoch J2000 for the body if it was not
    # introduced by the user
    if a is None:
        # https://github.com/poliastro/poliastro/pull/679#issuecomment-503902597
        from poliastro.twobody.orbit import Orbit

        a = Orbit.from_body_ephem(body, J2000_TDB).a

    r_SOI = a * (body.k / body.parent.k) ** (2 / 5)

    return r_SOI.decompose()

___

#### The next is going to be the propagation directory, beginning with the dependencies

___

In [37]:
@jit
def c2(psi):
    r"""Second Stumpff function.
    For positive arguments:
    .. math::
        c_2(\psi) = \frac{1 - \cos{\sqrt{\psi}}}{\psi}
    """
    eps = 1.0
    if psi > eps:
        res = (1 - np.cos(np.sqrt(psi))) / psi
    elif psi < -eps:
        res = (np.cosh(np.sqrt(-psi)) - 1) / (-psi)
    else:
        res = 1.0 / 2.0
        delta = (-psi) / gamma(2 + 2 + 1)
        k = 1
        while res + delta != res:
            res = res + delta
            k += 1
            delta = (-psi) ** k / gamma(2 * k + 2 + 1)

    return res


@jit
def c3(psi):
    r"""Third Stumpff function.
    For positive arguments:
    .. math::
        c_3(\psi) = \frac{\sqrt{\psi} - \sin{\sqrt{\psi}}}{\sqrt{\psi^3}}
    """
    eps = 1.0
    if psi > eps:
        res = (np.sqrt(psi) - np.sin(np.sqrt(psi))) / (psi * np.sqrt(psi))
    elif psi < -eps:
        res = (np.sinh(np.sqrt(-psi)) - np.sqrt(-psi)) / (-psi * np.sqrt(-psi))
    else:
        res = 1.0 / 6.0
        delta = (-psi) / gamma(2 + 3 + 1)
        k = 1
        while res + delta != res:
            res = res + delta
            k += 1
            delta = (-psi) ** k / gamma(2 * k + 3 + 1)

    return res

In [38]:
def func_twobody(t0, u_, k, ad, ad_kwargs):
    """Differential equation for the initial value two body problem.
    This function follows Cowell's formulation.
    Parameters
    ----------
    t0 : float
        Time.
    u_ : ~numpy.ndarray
        Six component state vector [x, y, z, vx, vy, vz] (km, km/s).
    k : float
        Standard gravitational parameter.
    ad : function(t0, u, k)
        Non Keplerian acceleration (km/s2).
    ad_kwargs : optional
        perturbation parameters passed to ad.
    """
    ax, ay, az = ad(t0, u_, k, **ad_kwargs)

    x, y, z, vx, vy, vz = u_
    r3 = (x ** 2 + y ** 2 + z ** 2) ** 1.5

    du = np.array([vx, vy, vz, -k * x / r3 + ax, -k * y / r3 + ay, -k * z / r3 + az])
    return du


@jit
def mean_motion_fast(k, r0, v0, tof):
    r"""Propagates orbit using mean motion. This algorithm depends on the geometric shape of the
    orbit.
    For the case of the strong elliptic or strong hyperbolic orbits:
    ..  math::
        M = M_{0} + \frac{\mu^{2}}{h^{3}}\left ( 1 -e^{2}\right )^{\frac{3}{2}}t
    .. versionadded:: 0.9.0
    Parameters
    ----------
    k : float
        Standar Gravitational parameter
    r0 : ~astropy.units.Quantity
        Initial position vector wrt attractor center.
    v0 : ~astropy.units.Quantity
        Initial velocity vector.
    tof : float
        Time of flight (s).
    Note
    ----
    This method takes initial :math:`\vec{r}, \vec{v}`, calculates classical orbit parameters,
    increases mean anomaly and performs inverse transformation to get final :math:`\vec{r}, \vec{v}`
    The logic is based on formulae (4), (6) and (7) from http://dx.doi.org/10.1007/s10569-013-9476-9
    """

    # get the initial true anomaly and orbit parameters that are constant over time
    p, ecc, inc, raan, argp, nu0 = rv2coe(k, r0, v0)

    # get the initial mean anomaly
    M0 = nu_to_M(nu0, ecc)
    # strong elliptic or strong hyperbolic orbits
    if np.abs(ecc - 1.0) > 1e-2:
        a = p / (1.0 - ecc ** 2)
        # given the initial mean anomaly, calculate mean anomaly
        # at the end, mean motion (n) equals sqrt(mu / |a^3|)
        M = M0 + tof * np.sqrt(k / np.abs(a ** 3))
        nu = M_to_nu(M, ecc)

    # near-parabolic orbit
    else:
        q = p * np.abs(1.0 - ecc) / np.abs(1.0 - ecc ** 2)
        # mean motion n = sqrt(mu / 2 q^3) for parabolic orbit
        M = M0 + tof * np.sqrt(k / 2.0 / (q ** 3))
        nu = M_to_nu(M, ecc)

    return coe2rv(k, p, ecc, inc, raan, argp, nu)


@jit
def kepler_fast(k, r0, v0, tof, numiter):
    r"""Solves Kepler's Equation by applying a Newton-Raphson method.
    If the position of a body along its orbit wants to be computed
    for an specific time, it can be solved by terms of the Kepler's Equation:
    .. math::
        E = M + e\sin{E}
    In this case, the equation is written in terms of the Universal Anomaly:
    .. math::
        \sqrt{\mu}\Delta t = \frac{r_{o}v_{o}}{\sqrt{\mu}}\chi^{2}C(\alpha \chi^{2}) + (1 - \alpha r_{o})\chi^{3}S(\alpha \chi^{2}) + r_{0}\chi
    This equation is solved for the universal anomaly by applying a Newton-Raphson numerical method.
    Once it is solved, the Lagrange coefficients are returned:
    .. math::
        \begin{align}
            f &= 1 \frac{\chi^{2}}{r_{o}}C(\alpha \chi^{2}) \\
            g &= \Delta t - \frac{1}{\sqrt{\mu}}\chi^{3}S(\alpha \chi^{2}) \\
            \dot{f} &= \frac{\sqrt{\mu}}{rr_{o}}(\alpha \chi^{3}S(\alpha \chi^{2}) - \chi) \\
            \dot{g} &= 1 - \frac{\chi^{2}}{r}C(\alpha \chi^{2}) \\
        \end{align}
    Lagrange coefficients can be related then with the position and velocity vectors:
    .. math::
        \begin{align}
            \vec{r} &= f\vec{r_{o}} + g\vec{v_{o}} \\
            \vec{v} &= \dot{f}\vec{r_{o}} + \dot{g}\vec{v_{o}} \\
        \end{align}
    Parameters
    ----------
    k: float
        Standard gravitational parameter
    r0: ~numpy.array
        Initial position vector
    v0: ~numpy.array
        Initial velocity vector
    numiter: int
        Number of iterations
    Returns
    -------
    f: float
        First Lagrange coefficient
    g: float
        Second Lagrange coefficient
    fdot: float
        Derivative of the first coefficient
    gdot: float
        Derivative of the second coefficient
    Note
    ----
    The theoretical procedure is explained in section 3.7 of Curtis in really
    deep detail. For analytical example, check in the same book for example 3.6.
    """

    # Cache some results
    dot_r0v0 = np.dot(r0, v0)
    norm_r0 = np.dot(r0, r0) ** 0.5
    sqrt_mu = k ** 0.5
    alpha = -np.dot(v0, v0) / k + 2 / norm_r0

    # First guess
    if alpha > 0:
        # Elliptic orbit
        xi_new = sqrt_mu * tof * alpha
    elif alpha < 0:
        # Hyperbolic orbit
        xi_new = (
            np.sign(tof)
            * (-1 / alpha) ** 0.5
            * np.log(
                (-2 * k * alpha * tof)
                / (
                    dot_r0v0
                    + np.sign(tof) * np.sqrt(-k / alpha) * (1 - norm_r0 * alpha)
                )
            )
        )
    else:
        # Parabolic orbit
        # (Conservative initial guess)
        xi_new = sqrt_mu * tof / norm_r0

    # Newton-Raphson iteration on the Kepler equation
    count = 0
    while count < numiter:
        xi = xi_new
        psi = xi * xi * alpha
        c2_psi = c2(psi)
        c3_psi = c3(psi)
        norm_r = (
            xi * xi * c2_psi
            + dot_r0v0 / sqrt_mu * xi * (1 - psi * c3_psi)
            + norm_r0 * (1 - psi * c2_psi)
        )
        xi_new = (
            xi
            + (
                sqrt_mu * tof
                - xi * xi * xi * c3_psi
                - dot_r0v0 / sqrt_mu * xi * xi * c2_psi
                - norm_r0 * xi * (1 - psi * c3_psi)
            )
            / norm_r
        )
        if abs(xi_new - xi) < 1e-7:
            break
        else:
            count += 1
    else:
        raise RuntimeError("Maximum number of iterations reached")

    # Compute Lagrange coefficients
    f = 1 - xi ** 2 / norm_r0 * c2_psi
    g = tof - xi ** 3 / sqrt_mu * c3_psi

    gdot = 1 - xi ** 2 / norm_r * c2_psi
    fdot = sqrt_mu / (norm_r * norm_r0) * xi * (psi * c3_psi - 1)

    return f, g, fdot, gdot


@jit
def mikkola_fast(k, r0, v0, tof, rtol=None):
    """ Raw algorithm for Mikkola's Kepler solver.
    Parameters
    ----------
    k : ~astropy.units.Quantity
        Standard gravitational parameter of the attractor.
    r : ~astropy.units.Quantity
        Position vector.
    v : ~astropy.units.Quantity
        Velocity vector.
    tofs : ~astropy.units.Quantity
        Array of times to propagate.
    rtol: float
        This method does not require of tolerance since it is non iterative.
    Returns
    -------
    rr : ~astropy.units.Quantity
        Propagated position vectors.
    vv : ~astropy.units.Quantity
    Note
    ----
    Original paper: https://doi.org/10.1007/BF01235850
    """

    # Solving for the classical elements
    p, ecc, inc, raan, argp, nu = rv2coe(k, r0, v0)
    M0 = nu_to_M(nu, ecc, delta=0)
    a = p / (1 - ecc ** 2)
    n = np.sqrt(k / np.abs(a) ** 3)
    M = M0 + n * tof

    # Solve for specific geometrical case
    if ecc < 1.0:
        # Equation (9a)
        alpha = (1 - ecc) / (4 * ecc + 1 / 2)
    else:
        alpha = (ecc - 1) / (4 * ecc + 1 / 2)

    beta = M / 2 / (4 * ecc + 1 / 2)

    # Equation (9b)
    if beta >= 0:
        z = (beta + np.sqrt(beta ** 2 + alpha ** 3)) ** (1 / 3)
    else:
        z = (beta - np.sqrt(beta ** 2 + alpha ** 3)) ** (1 / 3)

    s = z - alpha / z

    # Apply initial correction
    if ecc < 1.0:
        ds = -0.078 * s ** 5 / (1 + ecc)
    else:
        ds = 0.071 * s ** 5 / (1 + 0.45 * s ** 2) / (1 + 4 * s ** 2) / ecc

    s += ds

    # Solving for the true anomaly
    if ecc < 1.0:
        E = M + ecc * (3 * s - 4 * s ** 3)
        f = E - ecc * np.sin(E) - M
        f1 = 1.0 - ecc * np.cos(E)
        f2 = ecc * np.sin(E)
        f3 = ecc * np.cos(E)
        f4 = -f2
        f5 = -f3
    else:
        E = 3 * np.log(s + np.sqrt(1 + s ** 2))
        f = -E + ecc * np.sinh(E) - M
        f1 = -1.0 + ecc * np.cosh(E)
        f2 = ecc * np.sinh(E)
        f3 = ecc * np.cosh(E)
        f4 = f2
        f5 = f3

    # Apply Taylor expansion
    u1 = -f / f1
    u2 = -f / (f1 + 0.5 * f2 * u1)
    u3 = -f / (f1 + 0.5 * f2 * u2 + (1.0 / 6.0) * f3 * u2 ** 2)
    u4 = -f / (
        f1 + 0.5 * f2 * u3 + (1.0 / 6.0) * f3 * u3 ** 2 + (1.0 / 24.0) * f4 * (u3 ** 3)
    )
    u5 = -f / (
        f1
        + f2 * u4 / 2
        + f3 * (u4 * u4) / 6.0
        + f4 * (u4 * u4 * u4) / 24.0
        + f5 * (u4 * u4 * u4 * u4) / 120.0
    )

    E += u5

    if ecc < 1.0:
        nu = E_to_nu(E, ecc)
    else:
        if ecc == 1.0:
            # Parabolic
            nu = D_to_nu(E)
        else:
            # Hyperbolic
            nu = F_to_nu(E, ecc)

    return coe2rv(k, p, ecc, inc, raan, argp, nu)


@jit
def markley_fast(k, r0, v0, tof):
    """ Solves the kepler problem by a non iterative method. Relative error is
    around 1e-18, only limited by machine double-precission errors.
    Parameters
    ----------
    k : float
        Standar Gravitational parameter
    r0 : array
        Initial position vector wrt attractor center.
    v0 : array
        Initial velocity vector.
    tof : float
        Time of flight.
    Returns
    -------
    rf: array
        Final position vector
    vf: array
        Final velocity vector
    Note
    ----
    The following algorithm was taken from http://dx.doi.org/10.1007/BF00691917.
    """

    # Solve first for eccentricity and mean anomaly
    p, ecc, inc, raan, argp, nu = rv2coe(k, r0, v0)

    M0 = nu_to_M(nu, ecc, delta=0)
    a = p / (1 - ecc ** 2)
    n = np.sqrt(k / a ** 3)
    M = M0 + n * tof

    # Range between -pi and pi
    M = M % (2 * np.pi)
    if M > np.pi:
        M = -(2 * np.pi - M)

    # Equation (20)
    alpha = (3 * np.pi ** 2 + 1.6 * (np.pi - np.abs(M)) / (1 + ecc)) / (np.pi ** 2 - 6)

    # Equation (5)
    d = 3 * (1 - ecc) + alpha * ecc

    # Equation (9)
    q = 2 * alpha * d * (1 - ecc) - M ** 2

    # Equation (10)
    r = 3 * alpha * d * (d - 1 + ecc) * M + M ** 3

    # Equation (14)
    w = (np.abs(r) + np.sqrt(q ** 3 + r ** 2)) ** (2 / 3)

    # Equation (15)
    E = (2 * r * w / (w ** 2 + w * q + q ** 2) + M) / d

    # Equation (26)
    f0 = _kepler_equation(E, M, ecc)
    f1 = _kepler_equation_prime(E, M, ecc)
    f2 = ecc * np.sin(E)
    f3 = ecc * np.cos(E)
    f4 = -f2

    # Equation (22)
    delta3 = -f0 / (f1 - 0.5 * f0 * f2 / f1)
    delta4 = -f0 / (f1 + 0.5 * delta3 * f2 + 1 / 6 * delta3 ** 2 * f3)
    delta5 = -f0 / (
        f1 + 0.5 * delta4 * f2 + 1 / 6 * delta4 ** 2 * f3 + 1 / 24 * delta4 ** 3 * f4
    )

    E += delta5
    nu = E_to_nu(E, ecc)

    return coe2rv(k, p, ecc, inc, raan, argp, nu)


@jit
def pimienta_fast(k, r0, v0, tof):
    """ Raw algorithm for Adonis' Pimienta and John L. Crassidis 15th order
    polynomial Kepler solver.
    Parameters
    ----------
    k : float
        Standar Gravitational parameter
    r0 : array
        Initial position vector wrt attractor center.
    v0 : array
        Initial velocity vector.
    tof : float
        Time of flight.
    Returns
    -------
    rf: array
        Final position vector
    vf: array
        Final velocity vector
    Note
    ----
    This algorithm was drived from the original paper: http://hdl.handle.net/10477/50522
    """

    # TODO: implement hyperbolic case

    # Solve first for eccentricity and mean anomaly
    p, ecc, inc, raan, argp, nu = rv2coe(k, r0, v0)

    M0 = nu_to_M(nu, ecc, delta=0)
    semi_axis_a = p / (1 - ecc ** 2)
    n = np.sqrt(k / np.abs(semi_axis_a) ** 3)
    M = M0 + n * tof

    # Equation (32a), (32b), (32c) and (32d)
    c3 = 5 / 2 + 560 * ecc
    a = 15 * (1 - ecc) / c3
    b = -M / c3
    y = np.sqrt(b ** 2 / 4 + a ** 3 / 27)

    # Equation (33)
    x_bar = (-b / 2 + y) ** (1 / 3) - (b / 2 + y) ** (1 / 3)

    # Coefficients from equations (34a) and (34b)
    c15 = 3003 / 14336 + 16384 * ecc
    c13 = 3465 / 13312 - 61440 * ecc
    c11 = 945 / 2816 + 92160 * ecc
    c9 = 175 / 384 - 70400 * ecc
    c7 = 75 / 112 + 28800 * ecc
    c5 = 9 / 8 - 6048 * ecc

    # Precompute x_bar powers, equations (35a) to (35d)
    x_bar2 = x_bar ** 2
    x_bar3 = x_bar2 * x_bar
    x_bar4 = x_bar3 * x_bar
    x_bar5 = x_bar4 * x_bar
    x_bar6 = x_bar5 * x_bar
    x_bar7 = x_bar6 * x_bar
    x_bar8 = x_bar7 * x_bar
    x_bar9 = x_bar8 * x_bar
    x_bar10 = x_bar9 * x_bar
    x_bar11 = x_bar10 * x_bar
    x_bar12 = x_bar11 * x_bar
    x_bar13 = x_bar12 * x_bar
    x_bar14 = x_bar13 * x_bar
    x_bar15 = x_bar14 * x_bar

    # Function f and its derivatives are given by all the (36) equation set
    f = (
        c15 * x_bar15
        + c13 * x_bar13
        + c11 * x_bar11
        + c9 * x_bar9
        + c7 * x_bar7
        + c5 * x_bar5
        + c3 * x_bar3
        + 15 * (1 - ecc) * x_bar
        - M
    )
    f1 = (
        15 * c15 * x_bar14
        + 13 * c13 * x_bar12
        + 11 * c11 * x_bar10
        + 9 * c9 * x_bar8
        + 7 * c7 * x_bar6
        + 5 * c5 * x_bar4
        + 3 * c3 * x_bar2
        + 15 * (1 - ecc)
    )
    f2 = (
        210 * c15 * x_bar13
        + 156 * c13 * x_bar11
        + 110 * c11 * x_bar9
        + 72 * c9 * x_bar7
        + 42 * c7 * x_bar5
        + 20 * c5 * x_bar3
        + 6 * c3 * x_bar
    )
    f3 = (
        2730 * c15 * x_bar12
        + 1716 * c13 * x_bar10
        + 990 * c11 * x_bar8
        + 504 * c9 * x_bar6
        + 210 * c7 * x_bar4
        + 60 * c5 * x_bar2
        + 6 * c3
    )
    f4 = (
        32760 * c15 * x_bar11
        + 17160 * c13 * x_bar9
        + 7920 * c11 * x_bar7
        + 3024 * c9 * x_bar5
        + 840 * c7 * x_bar3
        + 120 * c5 * x_bar
    )
    f5 = (
        360360 * c15 * x_bar10
        + 154440 * c13 * x_bar8
        + 55440 * c11 * x_bar6
        + 15120 * c9 * x_bar4
        + 2520 * c7 * x_bar2
        + 120 * c5
    )
    f6 = (
        3603600 * c15 * x_bar9
        + 1235520 * c13 * x_bar7
        + 332640 * c11 * x_bar5
        + 60480 * c9 * x_bar3
        + 5040 * c7 * x_bar
    )
    f7 = (
        32432400 * c15 * x_bar8
        + 8648640 * c13 * x_bar6
        + 1663200 * c11 * x_bar4
        + 181440 * c9 * x_bar2
        + 5040 * c7
    )
    f8 = (
        259459200 * c15 * x_bar7
        + 51891840 * c13 * x_bar5
        + 6652800 * c11 * x_bar3
        + 362880 * c9 * x_bar
    )
    f9 = (
        1.8162144e9 * c15 * x_bar6
        + 259459200 * c13 * x_bar4
        + 19958400 * c11 * x_bar2
        + 362880 * c9
    )
    f10 = (
        1.08972864e10 * c15 * x_bar5
        + 1.0378368e9 * c13 * x_bar3
        + 39916800 * c11 * x_bar
    )
    f11 = 5.4486432e10 * c15 * x_bar4 + 3.1135104e9 * c13 * x_bar2 + 39916800 * c11
    f12 = 2.17945728e11 * c15 * x_bar3 + 6.2270208e9 * c13 * x_bar
    f13 = 6.53837184 * c15 * x_bar2 + 6.2270208e9 * c13
    f14 = 1.307674368e13 * c15 * x_bar
    f15 = 1.307674368e13 * c15

    # Solving g parameters defined by equations (37a), (37b), (37c) and (37d)
    g1 = 1 / 2
    g2 = 1 / 6
    g3 = 1 / 24
    g4 = 1 / 120
    g5 = 1 / 720
    g6 = 1 / 5040
    g7 = 1 / 40320
    g8 = 1 / 362880
    g9 = 1 / 3628800
    g10 = 1 / 39916800
    g11 = 1 / 479001600
    g12 = 1 / 6.2270208e9
    g13 = 1 / 8.71782912e10
    g14 = 1 / 1.307674368e12

    # Solving for the u_{i} and h_{i} variables defined by equation (38)
    u1 = -f / f1

    h2 = f1 + g1 * u1 * f2
    u2 = -f / h2

    h3 = f1 + g1 * u2 * f2 + g2 * u2 ** 2 * f3
    u3 = -f / h3

    h4 = f1 + g1 * u3 * f2 + g2 * u3 ** 2 * f3 + g3 * u3 ** 3 * f4
    u4 = -f / h4

    h5 = f1 + g1 * u4 * f2 + g2 * u4 ** 2 * f3 + g3 * u4 ** 3 * f4 + g4 * u4 ** 4 * f5
    u5 = -f / h5

    h6 = (
        f1
        + g1 * u5 * f2
        + g2 * u5 ** 2 * f3
        + g3 * u5 ** 3 * f4
        + g4 * u5 ** 4 * f5
        + g5 * u5 ** 5 * f6
    )
    u6 = -f / h6

    h7 = (
        f1
        + g1 * u6 * f2
        + g2 * u6 ** 2 * f3
        + g3 * u6 ** 3 * f4
        + g4 * u6 ** 4 * f5
        + g5 * u6 ** 5 * f6
        + g6 * u6 ** 6 * f7
    )
    u7 = -f / h7

    h8 = (
        f1
        + g1 * u7 * f2
        + g2 * u7 ** 2 * f3
        + g3 * u7 ** 3 * f4
        + g4 * u7 ** 4 * f5
        + g5 * u7 ** 5 * f6
        + g6 * u7 ** 6 * f7
        + g7 * u7 ** 7 * f8
    )
    u8 = -f / h8

    h9 = (
        f1
        + g1 * u8 * f2
        + g2 * u8 ** 2 * f3
        + g3 * u8 ** 3 * f4
        + g4 * u8 ** 4 * f5
        + g5 * u8 ** 5 * f6
        + g6 * u8 ** 6 * f7
        + g7 * u8 ** 7 * f8
        + g8 * u8 ** 8 * f9
    )
    u9 = -f / h9

    h10 = (
        f1
        + g1 * u9 * f2
        + g2 * u9 ** 2 * f3
        + g3 * u9 ** 3 * f4
        + g4 * u9 ** 4 * f5
        + g5 * u9 ** 5 * f6
        + g6 * u9 ** 6 * f7
        + g7 * u9 ** 7 * f8
        + g8 * u9 ** 8 * f9
        + g9 * u9 ** 9 * f10
    )
    u10 = -f / h10

    h11 = (
        f1
        + g1 * u10 * f2
        + g2 * u10 ** 2 * f3
        + g3 * u10 ** 3 * f4
        + g4 * u10 ** 4 * f5
        + g5 * u10 ** 5 * f6
        + g6 * u10 ** 6 * f7
        + g7 * u10 ** 7 * f8
        + g8 * u10 ** 8 * f9
        + g9 * u10 ** 9 * f10
        + g10 * u10 ** 10 * f11
    )
    u11 = -f / h11

    h12 = (
        f1
        + g1 * u11 * f2
        + g2 * u11 ** 2 * f3
        + g3 * u11 ** 3 * f4
        + g4 * u11 ** 4 * f5
        + g5 * u11 ** 5 * f6
        + g6 * u11 ** 6 * f7
        + g7 * u11 ** 7 * f8
        + g8 * u11 ** 8 * f9
        + g9 * u11 ** 9 * f10
        + g10 * u11 ** 10 * f11
        + g11 * u11 ** 11 * f12
    )
    u12 = -f / h12

    h13 = (
        f1
        + g1 * u12 * f2
        + g2 * u12 ** 2 * f3
        + g3 * u12 ** 3 * f4
        + g4 * u12 ** 4 * f5
        + g5 * u12 ** 5 * f6
        + g6 * u12 ** 6 * f7
        + g7 * u12 ** 7 * f8
        + g8 * u12 ** 8 * f9
        + g9 * u12 ** 9 * f10
        + g10 * u12 ** 10 * f11
        + g11 * u12 ** 11 * f12
        + g12 * u12 ** 12 * f13
    )
    u13 = -f / h13

    h14 = (
        f1
        + g1 * u13 * f2
        + g2 * u13 ** 2 * f3
        + g3 * u13 ** 3 * f4
        + g4 * u13 ** 4 * f5
        + g5 * u13 ** 5 * f6
        + g6 * u13 ** 6 * f7
        + g7 * u13 ** 7 * f8
        + g8 * u13 ** 8 * f9
        + g9 * u13 ** 9 * f10
        + g10 * u13 ** 10 * f11
        + g11 * u13 ** 11 * f12
        + g12 * u13 ** 12 * f13
        + g13 * u13 ** 13 * f14
    )
    u14 = -f / h14

    h15 = (
        f1
        + g1 * u14 * f2
        + g2 * u14 ** 2 * f3
        + g3 * u14 ** 3 * f4
        + g4 * u14 ** 4 * f5
        + g5 * u14 ** 5 * f6
        + g6 * u14 ** 6 * f7
        + g7 * u14 ** 7 * f8
        + g8 * u14 ** 8 * f9
        + g9 * u14 ** 9 * f10
        + g10 * u14 ** 10 * f11
        + g11 * u14 ** 11 * f12
        + g12 * u14 ** 12 * f13
        + g13 * u14 ** 13 * f14
        + g14 * u14 ** 14 * f15
    )
    u15 = -f / h15

    # Solving for x
    x = x_bar + u15
    w = x - 0.01171875 * x ** 17 / (1 + ecc)

    # Solving for the true anomaly from eccentricity anomaly
    E = M + ecc * (
        -16384 * w ** 15
        + 61440 * w ** 13
        - 92160 * w ** 11
        + 70400 * w ** 9
        - 28800 * w ** 7
        + 6048 * w ** 5
        - 560 * w ** 3
        + 15 * w
    )

    nu = E_to_nu(E, ecc)

    return coe2rv(k, p, ecc, inc, raan, argp, nu)


@jit
def gooding_fast(k, r0, v0, tof, numiter=150, rtol=1e-8):
    """ Solves the Elliptic Kepler Equation with a cubic convergence and
    accuracy better than 10e-12 rad is normally achieved. It is not valid for
    eccentricities equal or higher than 1.0.
    Parameters
    ----------
    k : float
        Standard gravitational parameter of the attractor.
    r : 1x3 vector
        Position vector.
    v : 1x3 vector
        Velocity vector.
    tof : float
        Time of flight.
    rtol: float
        Relative error for accuracy of the method.
    Returns
    -------
    rr : 1x3 vector
        Propagated position vectors.
     vv : 1x3 vector
    Note
    ----
    Original paper for the algorithm: https://doi.org/10.1007/BF01238923
    """

    # Solve first for eccentricity and mean anomaly
    p, ecc, inc, raan, argp, nu = rv2coe(k, r0, v0)

    # TODO: parabolic and hyperbolic not implemented cases
    if ecc >= 1.0:
        raise NotImplementedError(
            "Parabolic/Hyperbolic cases still not implemented in gooding."
        )

    M0 = nu_to_M(nu, ecc, delta=0)
    semi_axis_a = p / (1 - ecc ** 2)
    n = np.sqrt(k / np.abs(semi_axis_a) ** 3)
    M = M0 + n * tof

    # Start the computation
    n = 0
    c = ecc * np.cos(M)
    s = ecc * np.sin(M)
    psi = s / np.sqrt(1 - 2 * c + ecc ** 2)
    f = 1.0
    while f ** 2 >= rtol and n <= numiter:
        xi = np.cos(psi)
        eta = np.sin(psi)
        fd = (1 - c * xi) + s * eta
        fdd = c * eta + s * xi
        f = psi - fdd
        psi = psi - f * fd / (fd ** 2 - 0.5 * f * fdd)
        n += 1

    E = M + psi
    nu = E_to_nu(E, ecc)

    return coe2rv(k, p, ecc, inc, raan, argp, nu)


@jit
def danby_fast(k, r0, v0, tof, numiter=20, rtol=1e-8):
    """ Kepler solver for both elliptic and parabolic orbits based on Danby's
    algorithm.
    Parameters
    ----------
    k : float
        Standard gravitational parameter of the attractor.
    r : 1x3 vector
        Position vector.
    v : 1x3 vector
        Velocity vector.
    tof : float
        Time of flight.
    rtol: float
        Relative error for accuracy of the method.
    Returns
    -------
    rr : 1x3 vector
        Propagated position vectors.
    vv : 1x3 vector
    Note
    ----
    This algorithm was developed by Danby in his paper *The solution of Kepler
    Equation* with DOI: https://doi.org/10.1007/BF01686811
    """

    # Solve first for eccentricity and mean anomaly
    p, ecc, inc, raan, argp, nu = rv2coe(k, r0, v0)
    M0 = nu_to_M(nu, ecc, delta=0)
    semi_axis_a = p / (1 - ecc ** 2)
    n = np.sqrt(k / np.abs(semi_axis_a) ** 3)
    M = M0 + n * tof

    # Range mean anomaly
    xma = M - 2 * np.pi * np.floor(M / 2 / np.pi)

    if ecc == 0:
        # Solving for circular orbit
        nu = xma
        return coe2rv(k, p, ecc, inc, raan, argp, nu)

    elif ecc < 1.0:
        # For elliptical orbit
        E = xma + 0.85 * np.sign(np.sin(xma)) * ecc

    else:
        # For parabolic and hyperbolic
        E = np.log(2 * xma / ecc + 1.8)

    # Iterations begin
    n = 0
    while n <= numiter:

        if ecc < 1.0:
            s = ecc * np.sin(E)
            c = ecc * np.cos(E)
            f = E - s - xma
            fp = 1 - c
            fpp = s
            fppp = c
        else:
            s = ecc * np.sinh(E)
            c = ecc * np.cosh(E)
            f = s - E - xma
            fp = c - 1
            fpp = s
            fppp = c

        if np.abs(f) <= rtol:

            if ecc < 1.0:
                sta = np.sqrt(1 - ecc ** 2) * np.sin(E)
                cta = np.cos(E) - ecc
            else:
                sta = np.sqrt(ecc ** 2 - 1) * np.sinh(E)
                cta = ecc - np.cosh(E)

            nu = np.arctan2(sta, cta)
            return coe2rv(k, p, ecc, inc, raan, argp, nu)

        else:
            delta = -f / fp
            delta_star = -f / (fp + 0.5 * delta * fpp)
            deltak = -f / (fp + 0.5 * delta_star * fpp + delta_star ** 2 * fppp / 6)
            E = E + deltak
            n += 1

    raise ValueError("Maximum number of iterations has been reached.")

In [39]:
# poliastro.integrators
A = [
    np.array([]),
    np.array([5.26001519587677318785587544488e-2]),
    np.array([1.97250569845378994544595329183e-2, 5.91751709536136983633785987549e-2]),
    np.array(
        [2.95875854768068491816892993775e-2, 0, 8.87627564304205475450678981324e-2]
    ),
    np.array(
        [
            2.41365134159266685502369798665e-1,
            0,
            -8.84549479328286085344864962717e-1,
            9.24834003261792003115737966543e-1,
        ]
    ),
    np.array(
        [
            3.7037037037037037037037037037e-2,
            0,
            0,
            1.70828608729473871279604482173e-1,
            1.25467687566822425016691814123e-1,
        ]
    ),
    np.array(
        [
            3.7109375e-2,
            0,
            0,
            1.70252211019544039314978060272e-1,
            6.02165389804559606850219397283e-2,
            -1.7578125e-2,
        ]
    ),
    np.array(
        [
            3.70920001185047927108779319836e-2,
            0,
            0,
            1.70383925712239993810214054705e-1,
            1.07262030446373284651809199168e-1,
            -1.53194377486244017527936158236e-2,
            8.27378916381402288758473766002e-3,
        ]
    ),
    np.array(
        [
            6.24110958716075717114429577812e-1,
            0,
            0,
            -3.36089262944694129406857109825e-0,
            -8.68219346841726006818189891453e-1,
            2.75920996994467083049415600797e1,
            2.01540675504778934086186788979e1,
            -4.34898841810699588477366255144e1,
        ]
    ),
    np.array(
        [
            4.77662536438264365890433908527e-1,
            0,
            0,
            -2.48811461997166764192642586468e-0,
            -5.90290826836842996371446475743e-1,
            2.12300514481811942347288949897e1,
            1.52792336328824235832596922938e1,
            -3.32882109689848629194453265587e1,
            -2.03312017085086261358222928593e-2,
        ]
    ),
    np.array(
        [
            -9.3714243008598732571704021658e-1,
            0,
            0,
            5.18637242884406370830023853209e0,
            1.09143734899672957818500254654e0,
            -8.14978701074692612513997267357e0,
            -1.85200656599969598641566180701e1,
            2.27394870993505042818970056734e1,
            2.49360555267965238987089396762e0,
            -3.0467644718982195003823669022e0,
        ]
    ),
    np.array(
        [
            2.27331014751653820792359768449e0,
            0,
            0,
            -1.05344954667372501984066689879e1,
            -2.00087205822486249909675718444e0,
            -1.79589318631187989172765950534e1,
            2.79488845294199600508499808837e1,
            -2.85899827713502369474065508674e0,
            -8.87285693353062954433549289258e0,
            1.23605671757943030647266201528e1,
            6.43392746015763530355970484046e-1,
        ]
    ),
    np.array([]),
    np.array(
        [
            5.61675022830479523392909219681e-2,
            0,
            0,
            0,
            0,
            0,
            2.53500210216624811088794765333e-1,
            -2.46239037470802489917441475441e-1,
            -1.24191423263816360469010140626e-1,
            1.5329179827876569731206322685e-1,
            8.20105229563468988491666602057e-3,
            7.56789766054569976138603589584e-3,
            -8.298e-3,
        ]
    ),
    np.array(
        [
            3.18346481635021405060768473261e-2,
            0,
            0,
            0,
            0,
            2.83009096723667755288322961402e-2,
            5.35419883074385676223797384372e-2,
            -5.49237485713909884646569340306e-2,
            0,
            0,
            -1.08347328697249322858509316994e-4,
            3.82571090835658412954920192323e-4,
            -3.40465008687404560802977114492e-4,
            1.41312443674632500278074618366e-1,
        ]
    ),
    np.array(
        [
            -4.28896301583791923408573538692e-1,
            0,
            0,
            0,
            0,
            -4.69762141536116384314449447206e0,
            7.68342119606259904184240953878e0,
            4.06898981839711007970213554331e0,
            3.56727187455281109270669543021e-1,
            0,
            0,
            0,
            -1.39902416515901462129418009734e-3,
            2.9475147891527723389556272149e0,
            -9.15095847217987001081870187138e0,
        ]
    ),
]

C = np.array(
    [
        0.0,
        0.526001519587677318785587544488e-01,
        0.789002279381515978178381316732e-01,
        0.118350341907227396726757197510e00,
        0.281649658092772603273242802490e00,
        0.333333333333333333333333333333e00,
        0.25e00,
        0.307692307692307692307692307692e00,
        0.651282051282051282051282051282e00,
        0.6e00,
        0.857142857142857142857142857142e00,
        1.0,
        1.0,
        0.1e00,
        0.2e00,
        0.777777777777777777777777777778e00,
    ]
)

B = np.array(
    [
        5.42937341165687622380535766363e-2,
        0,
        0,
        0,
        0,
        4.45031289275240888144113950566e0,
        1.89151789931450038304281599044e0,
        -5.8012039600105847814672114227e0,
        3.1116436695781989440891606237e-1,
        -1.52160949662516078556178806805e-1,
        2.01365400804030348374776537501e-1,
        4.47106157277725905176885569043e-2,
    ]
)


BHH = np.array(
    [
        0.244094488188976377952755905512e00,
        0.733846688281611857341361741547e00,
        0.220588235294117647058823529412e-01,
    ]
)

E = np.array(
    [
        0.1312004499419488073250102996e-01,
        0,
        0,
        0,
        0,
        -0.1225156446376204440720569753e01,
        -0.4957589496572501915214079952e00,
        0.1664377182454986536961530415e01,
        -0.3503288487499736816886487290e00,
        0.3341791187130174790297318841e00,
        0.8192320648511571246570742613e-01,
        -0.2235530786388629525884427845e-01,
    ]
)

D = [
    np.array([]),
    np.array([]),
    np.array([]),
    np.array([]),
    np.array(
        [
            -0.84289382761090128651353491142e01,
            0,
            0,
            0,
            0,
            0.56671495351937776962531783590e00,
            -0.30689499459498916912797304727e01,
            0.23846676565120698287728149680e01,
            0.21170345824450282767155149946e01,
            -0.87139158377797299206789907490e00,
            0.22404374302607882758541771650e01,
            0.63157877876946881815570249290e00,
            -0.88990336451333310820698117400e-01,
            0.18148505520854727256656404962e02,
            -0.91946323924783554000451984436e01,
            -0.44360363875948939664310572000e01,
        ]
    ),
    np.array(
        [
            0.10427508642579134603413151009e02,
            0,
            0,
            0,
            0,
            0.24228349177525818288430175319e03,
            0.16520045171727028198505394887e03,
            -0.37454675472269020279518312152e03,
            -0.22113666853125306036270938578e02,
            0.77334326684722638389603898808e01,
            -0.30674084731089398182061213626e02,
            -0.93321305264302278729567221706e01,
            0.15697238121770843886131091075e02,
            -0.31139403219565177677282850411e02,
            -0.93529243588444783865713862664e01,
            0.35816841486394083752465898540e02,
        ]
    ),
    np.array(
        [
            0.19985053242002433820987653617e02,
            0,
            0,
            0,
            0,
            -0.38703730874935176555105901742e03,
            -0.18917813819516756882830838328e03,
            0.52780815920542364900561016686e03,
            -0.11573902539959630126141871134e02,
            0.68812326946963000169666922661e01,
            -0.10006050966910838403183860980e01,
            0.77771377980534432092869265740e00,
            -0.27782057523535084065932004339e01,
            -0.60196695231264120758267380846e02,
            0.84320405506677161018159903784e02,
            0.11992291136182789328035130030e02,
        ]
    ),
    np.array(
        [
            -0.25693933462703749003312586129e02,
            0,
            0,
            0,
            0,
            -0.15418974869023643374053993627e03,
            -0.23152937917604549567536039109e03,
            0.35763911791061412378285349910e03,
            0.93405324183624310003907691704e02,
            -0.37458323136451633156875139351e02,
            0.10409964950896230045147246184e03,
            0.29840293426660503123344363579e02,
            -0.435334565900111437544321750583e02,
            0.96324553959188282948394950600e02,
            -0.39177261675615439165231486172e02,
            -0.14972683625798562581422125276e03,
        ]
    ),
]


def validate_max_nsteps(max_nsteps):
    if max_nsteps <= 0:
        raise ValueError("`max_nsteps` must be positive.")
    return max_nsteps


def validate_safety_factor(safety_factor):
    if safety_factor >= 1.0 or safety_factor <= 1e-4:
        raise ValueError("`safety_factor` must lie within 1e-4 and 1.0.")
    return safety_factor


def validate_beta_stabilizer(beta_stabilizer):
    if beta_stabilizer < 0 or beta_stabilizer > 0.2:
        raise ValueError("`beta_stabilizer` must lie within 0 and 0.2.")
    return beta_stabilizer


class DOP835(OdeSolver):
    A = A
    C = C
    B = B
    E = E
    BHH = BHH
    D = D

    t: float
    y: np.array

    def __init__(
        self,
        fun,
        t0,
        y0,
        t_bound,
        max_step=np.inf,
        rtol=1e-7,
        atol=1e-12,
        safety_factor=0.9,
        min_step_change=0.333,
        max_step_change=6.0,
        beta_stabilizer=0.00,
        max_nsteps=100000,
        vectorized=False,
        **extraneous
    ):
        warn_extraneous(extraneous)
        super().__init__(fun, t0, y0, t_bound, vectorized, support_complex=True)
        self.y_old = None
        self.max_step = validate_max_step(max_step)
        self.beta_stabilizer = validate_beta_stabilizer(beta_stabilizer)
        self.max_nsteps = validate_max_nsteps(max_nsteps)
        self.safety_factor = validate_safety_factor(safety_factor)
        self.rtol, self.atol = validate_tol(rtol, atol, self.n)
        self.min_step_change = min_step_change
        self.max_step_change = max_step_change
        self.order = 8

        self.f = self.fun(self.t, self.y)
        self.h_abs = select_initial_step(
            self.fun,
            self.t,
            self.y,
            self.f,
            self.direction,
            self.order,
            self.rtol,
            self.atol,
        )
        self.nfev += 2

        self.n_steps = 0
        self.n_accepted = 0
        self.n_rejected = 0
        self.factor_old = 1e-4  # Lund-stabilization factor
        self.K = np.zeros((16, self.n))
        self.interpolation = np.zeros((8, self.n))

    def _step_impl(self):
        t = self.t
        y = self.y
        f = self.f
        K = self.K

        rtol = self.rtol
        atol = self.atol

        min_step = 10 * np.abs(np.nextafter(t, self.direction * np.inf) - t)

        while True:
            if self.h_abs < min_step:
                return False, self.TOO_SMALL_STEP

            h = self.h_abs * self.direction
            t_new = t + h

            if self.direction * (t_new - self.t_bound) > 0:
                t_new = self.t_bound

            h = t_new - t
            h_abs = np.abs(h)

            K[0] = f
            for s in range(1, 12):
                a, c = self.A[s], self.C[s]
                dy = np.dot(K[:s].T, a) * h
                K[s] = self.fun(t + c * h, y + dy)
            self.nfev += 11

            f_B = np.dot(K[:12].T, self.B)
            y_final = y + h * f_B

            scale = atol + np.maximum(np.abs(y), np.abs(y_final)) * rtol
            err_BHH = (
                f_B - self.BHH[0] * K[0] - self.BHH[1] * K[8] - self.BHH[2] * K[11]
            )
            err_BHH = np.sum((err_BHH / scale) ** 2)

            err_E = np.dot(K[:12].T, self.E)
            err_E = np.sum((err_E / scale) ** 2)

            denominator = err_E + 1e-2 * err_BHH
            err_E = h_abs * err_E / np.sqrt(self.n * denominator)

            err_exp = err_E ** (0.125 - self.beta_stabilizer * 0.2)
            dh_factor = err_exp / (self.factor_old ** self.beta_stabilizer)
            dh_factor = np.max(
                [
                    1.0 / self.max_step_change,
                    np.min(
                        [1.0 / self.min_step_change, dh_factor / self.safety_factor]
                    ),
                ]
            )
            h_new_abs = h_abs / dh_factor

            if err_E < 1.0:
                self.factor_old = np.max([err_E, 1e-4])
                self.n_accepted += 1
                K[12] = self.fun(t + h, y_final)

                for s in range(13, 16):
                    a, c = self.A[s], self.C[s]
                    dy = np.dot(K[:s].T, a) * h
                    K[s] = self.fun(t + c * h, y + dy)
                self.nfev += 4

                # prepare the dense output
                self.interpolation[0] = y
                self.interpolation[1] = y_final - y
                self.interpolation[2] = h * K[0] - self.interpolation[1]
                self.interpolation[3] = (
                    self.interpolation[1] - h * K[12] - self.interpolation[2]
                )
                for n in range(4, 8):
                    self.interpolation[n] = h * np.dot(K[:16].T, self.D[n])

                self.y_old = y
                self.t = t_new
                self.y = y_final
                self.f = K[12]
                self.h_abs = h_new_abs

                return True, None
            else:
                self.n_rejected += 1
                self.h_abs /= np.min(
                    [1.0 / self.min_step_change, err_exp / self.safety_factor]
                )

    def _dense_output_impl(self):
        return DOP835DenseOutput(self.t_old, self.t, self.y_old, self.interpolation)


def get_coeffs(s):
    coeffs = np.zeros((8))
    s_back = 1.0 - s
    coeffs[0] = 1.0
    for i in range(7):
        if i % 2 == 0:
            coeffs[i + 1] = coeffs[i] * s
        else:
            coeffs[i + 1] = coeffs[i] * s_back
    return np.array(coeffs)


class DOP835DenseOutput(DenseOutput):
    def __init__(self, t_old, t_new, y_old, interpolation):
        super().__init__(t_old, t_new)
        self.h = t_new - t_old
        self.interpolation = copy(interpolation)
        self.y_old = y_old

    def _call_impl(self, t_eval):
        s = (t_eval - self.t_old) / self.h
        coeffs = get_coeffs(s)
        return np.dot(self.interpolation.T, coeffs)

In [40]:
"""The following script holds the different high level functions for the
different propagators available at poliastro:
+-------------+------------+-----------------+-----------------+
|  Propagator | Elliptical |    Parabolic    |    Hyperbolic   |
+-------------+------------+-----------------+-----------------+
| mean_motion |      ✓     |        ✓        |        ✓        |
+-------------+------------+-----------------+-----------------+
|    kepler   |      ✓     |        ✓        |        ✓        |
+-------------+------------+-----------------+-----------------+
|   mikkola   |      ✓     | NOT IMPLEMENTED |        ✓        |
+-------------+------------+-----------------+-----------------+
|   markley   |      ✓     |        x        |        x        |
+-------------+------------+-----------------+-----------------+
|   pimienta  |      ✓     |        ✓        | NOT IMPLEMENTED |
+-------------+------------+-----------------+-----------------+
|   gooding   |      ✓     | NOT IMPLEMENTED | NOT IMPLEMENTED |
+-------------+------------+-----------------+-----------------+
|    danby    |      ✓     |        x        |        ✓        |
+-------------+------------+-----------------+-----------------+
|    cowell   |      ✓     |        ✓        |        ✓        |
+-------------+------------+-----------------+-----------------+
"""

from poliastro.integrators import DOP835


def cowell(k, r, v, tofs, rtol=1e-11, *, ad=None, **ad_kwargs):
    """Propagates orbit using Cowell's formulation.
    Parameters
    ----------
    k : ~astropy.units.Quantity
        Standard gravitational parameter of the attractor.
    r : ~astropy.units.Quantity
        Position vector.
    v : ~astropy.units.Quantity
        Velocity vector.
    tofs : ~astropy.units.Quantity
        Array of times to propagate.
    rtol : float, optional
        Maximum relative error permitted, default to 1e-10.
    ad : function(t0, u, k), optional
         Non Keplerian acceleration (km/s2), default to None.
    Returns
    -------
    rr : ~astropy.units.Quantity
        Propagated position vectors.
    vv : ~astropy.units.Quantity
        Propagated velocity vectors.
    Raises
    ------
    RuntimeError
        If the algorithm didn't converge.
    Note
    -----
    This method uses a Dormand & Prince method of order 8(5,3) available
    in the :py:class:`poliastro.integrators` module. If multiple tofs
    are provided, the method propagates to the maximum value and
    calculates the other values via dense output
    """
    k = k.to(u.km ** 3 / u.s ** 2).value
    x, y, z = r.to(u.km).value
    vx, vy, vz = v.to(u.km / u.s).value
    tofs = tofs.to(u.s).value

    u0 = np.array([x, y, z, vx, vy, vz])

    # Set the non Keplerian acceleration
    if ad is None:

        def ad(t0, u_, k_):
            return 0, 0, 0

    f_with_ad = functools.partial(func_twobody, k=k, ad=ad, ad_kwargs=ad_kwargs)

    result = solve_ivp(
        f_with_ad,
        (0, max(tofs)),
        u0,
        rtol=rtol,
        atol=1e-12,
        method=DOP835,
        dense_output=True,
    )
    if not result.success:
        raise RuntimeError("Integration failed")

    rrs = []
    vvs = []
    for i in range(len(tofs)):
        y = result.sol(tofs[i])
        rrs.append(y[:3])
        vvs.append(y[3:])

    return rrs * u.km, vvs * u.km / u.s


def mean_motion(k, r, v, tofs, **kwargs):
    """Propagates orbit using Cowell's formulation.
    Parameters
    ----------
    k : ~astropy.units.Quantity
        Standard gravitational parameter of the attractor.
    r : ~astropy.units.Quantity
        Position vector.
    v : ~astropy.units.Quantity
        Velocity vector.
    tofs : ~astropy.units.Quantity
        Array of times to propagate.
    Returns
    -------
    rr : ~astropy.units.Quantity
        Propagated position vectors.
    vv : ~astropy.units.Quantity
        Propagated velocity vectors.
    """
    k = k.to(u.km ** 3 / u.s ** 2).value
    r0 = r.to(u.km).value
    v0 = v.to(u.km / u.s).value
    tofs = tofs.to(u.s).value

    results = [mean_motion_fast(k, r0, v0, tof) for tof in tofs]
    # TODO: Rewrite to avoid iterating twice
    return (
        [result[0] for result in results] * u.km,
        [result[1] for result in results] * u.km / u.s,
    )


def kepler(k, r, v, tofs, numiter=350, **kwargs):
    """Propagates Keplerian orbit.
    Parameters
    ----------
    k : ~astropy.units.Quantity
        Standard gravitational parameter of the attractor.
    r : ~astropy.units.Quantity
        Position vector.
    v : ~astropy.units.Quantity
        Velocity vector.
    tofs : ~astropy.units.Quantity
        Array of times to propagate.
    numiter : int, optional
        Maximum number of iterations, default to 35.
    Returns
    -------
    rr : ~astropy.units.Quantity
        Propagated position vectors.
    vv : ~astropy.units.Quantity
        Propagated velocity vectors.
    Raises
    ------
    RuntimeError
        If the algorithm didn't converge.
    Note
    -----
    This algorithm is based on Vallado implementation, and does basic Newton
    iteration on the Kepler equation written using universal variables. Battin
    claims his algorithm uses the same amount of memory but is between 40 %
    and 85 % faster.
    """
    k = k.to(u.km ** 3 / u.s ** 2).value
    r0 = r.to(u.km).value
    v0 = v.to(u.km / u.s).value
    tofs = tofs.to(u.s).value

    results = [_kepler(k, r0, v0, tof, numiter=numiter) for tof in tofs]
    # TODO: Rewrite to avoid iterating twice
    return (
        [result[0] for result in results] * u.km,
        [result[1] for result in results] * u.km / u.s,
    )


def _kepler(k, r0, v0, tof, *, numiter):
    # Compute Lagrange coefficients
    f, g, fdot, gdot = kepler_fast(k, r0, v0, tof, numiter)

    assert np.abs(f * gdot - fdot * g - 1) < 1e-5  # Fixed tolerance

    # Return position and velocity vectors
    r = f * r0 + g * v0
    v = fdot * r0 + gdot * v0

    return r, v


def mikkola(k, r, v, tofs, rtol=None):
    """ Solves Kepler Equation by a cubic approximation. This method is valid
    no mater the orbit's nature.
    Parameters
    ----------
    k : ~astropy.units.Quantity
        Standard gravitational parameter of the attractor.
    r : ~astropy.units.Quantity
        Position vector.
    v : ~astropy.units.Quantity
        Velocity vector.
    tofs : ~astropy.units.Quantity
        Array of times to propagate.
    rtol: float
        This method does not require of tolerance since it is non iterative.
    Returns
    -------
    rr : ~astropy.units.Quantity
        Propagated position vectors.
    vv : ~astropy.units.Quantity
    Note
    ----
    This method was derived by Seppo Mikola in his paper *A Cubic Approximation
    For Kepler's Equation* with DOI: https://doi.org/10.1007/BF01235850
    """

    k = k.to(u.m ** 3 / u.s ** 2).value
    r0 = r.to(u.m).value
    v0 = v.to(u.m / u.s).value
    tofs = tofs.to(u.s).value

    results = [mikkola_fast(k, r0, v0, tof) for tof in tofs]
    return (
        [result[0] for result in results] * u.m,
        [result[1] for result in results] * u.m / u.s,
    )


def markley(k, r, v, tofs, rtol=None):
    """ Elliptical Kepler Equation solver based on a fifth-order
    refinement of the solution of a cubic equation.
    Parameters
    ----------
    k : ~astropy.units.Quantity
        Standard gravitational parameter of the attractor.
    r : ~astropy.units.Quantity
        Position vector.
    v : ~astropy.units.Quantity
        Velocity vector.
    tofs : ~astropy.units.Quantity
        Array of times to propagate.
    rtol: float
        This method does not require of tolerance since it is non iterative.
    Returns
    -------
    rr : ~astropy.units.Quantity
        Propagated position vectors.
    vv : ~astropy.units.Quantity
        Propagated velocity vectors.
    Note
    ----
    This method was originally presented by Markley in his paper *Kepler Equation Solver*
    with DOI: https://doi.org/10.1007/BF00691917
    """

    k = k.to(u.m ** 3 / u.s ** 2).value
    r0 = r.to(u.m).value
    v0 = v.to(u.m / u.s).value
    tofs = tofs.to(u.s).value

    results = [markley_fast(k, r0, v0, tof) for tof in tofs]
    return (
        [result[0] for result in results] * u.m,
        [result[1] for result in results] * u.m / u.s,
    )


def pimienta(k, r, v, tofs, rtol=None):
    """ Kepler solver for both elliptic and parabolic orbits based on a 15th
    order polynomial with accuracies around 10e-5 for elliptic case and 10e-13
    in the hyperbolic regime.
    Parameters
    ----------
    k : ~astropy.units.Quantity
        Standard gravitational parameter of the attractor.
    r : ~astropy.units.Quantity
        Position vector.
    v : ~astropy.units.Quantity
        Velocity vector.
    tofs : ~astropy.units.Quantity
        Array of times to propagate.
    rtol: float
        This method does not require of tolerance since it is non iterative.
    Returns
    -------
    rr : ~astropy.units.Quantity
        Propagated position vectors.
    vv : ~astropy.units.Quantity
        Propagated velocity vectors.
    Note
    ----
    This algorithm was developed by Pimienta-Peñalver and John L. Crassidis in
    their paper *Accurate Kepler Equation solver without trascendental function
    evaluations*. Original paper is on Buffalo's UBIR repository: http://hdl.handle.net/10477/50522
    """

    k = k.to(u.m ** 3 / u.s ** 2).value
    r0 = r.to(u.m).value
    v0 = v.to(u.m / u.s).value
    tofs = tofs.to(u.s).value

    results = [pimienta_fast(k, r0, v0, tof) for tof in tofs]
    return (
        [result[0] for result in results] * u.m,
        [result[1] for result in results] * u.m / u.s,
    )


def gooding(k, r, v, tofs, numiter=150, rtol=1e-8):
    """ Solves the Elliptic Kepler Equation with a cubic convergence and
    accuracy better than 10e-12 rad is normally achieved. It is not valid for
    eccentricities equal or greater than 1.0.
    Parameters
    ----------
    k : ~astropy.units.Quantity
        Standard gravitational parameter of the attractor.
    r : ~astropy.units.Quantity
        Position vector.
    v : ~astropy.units.Quantity
        Velocity vector.
    tofs : ~astropy.units.Quantity
        Array of times to propagate.
    rtol: float
        This method does not require of tolerance since it is non iterative.
    Returns
    -------
    rr : ~astropy.units.Quantity
        Propagated position vectors.
    vv : ~astropy.units.Quantity
    Note
    ----
    This method was developed by Gooding and Odell in their paper *The
    hyperbolic Kepler equation (and the elliptic equation revisited)* with
    DOI: https://doi.org/10.1007/BF01235540
    """

    k = k.to(u.m ** 3 / u.s ** 2).value
    r0 = r.to(u.m).value
    v0 = v.to(u.m / u.s).value
    tofs = tofs.to(u.s).value

    results = [gooding_fast(k, r0, v0, tof, numiter=numiter, rtol=rtol) for tof in tofs]
    return (
        [result[0] for result in results] * u.m,
        [result[1] for result in results] * u.m / u.s,
    )


def danby(k, r, v, tofs, rtol=1e-8):
    """ Kepler solver for both elliptic and parabolic orbits based on Danby's
    algorithm.
    Parameters
    ----------
    k : ~astropy.units.Quantity
        Standard gravitational parameter of the attractor.
    r : ~astropy.units.Quantity
        Position vector.
    v : ~astropy.units.Quantity
        Velocity vector.
    tofs : ~astropy.units.Quantity
        Array of times to propagate.
    rtol: float
        Relative error for accuracy of the method.
    Returns
    -------
    rr : ~astropy.units.Quantity
        Propagated position vectors.
    vv : ~astropy.units.Quantity
        Propagated velocity vectors.
    Note
    ----
    This algorithm was developed by Danby in his paper *The solution of Kepler
    Equation* with DOI: https://doi.org/10.1007/BF01686811
    """

    k = k.to(u.m ** 3 / u.s ** 2).value
    r0 = r.to(u.m).value
    v0 = v.to(u.m / u.s).value
    tofs = tofs.to(u.s).value

    results = [danby_fast(k, r0, v0, tof) for tof in tofs]
    return (
        [result[0] for result in results] * u.m,
        [result[1] for result in results] * u.m / u.s,
    )


def propagate(orbit, time_of_flight, *, method=mean_motion, rtol=1e-10, **kwargs):
    """Propagate an orbit some time and return the result.
    Parameters
    ----------
    orbit : ~poliastro.twobody.Orbit
        Orbit object to propagate.
    time_of_flight : ~astropy.time.TimeDelta
        Time of propagation.
    method : callable, optional
        Propagation method, default to mean_motion.
    rtol : float, optional
        Relative tolerance, default to 1e-10.
    """

    # Check if propagator fulfills orbit requirements
    if orbit.ecc < 1.0 and method not in ELLIPTIC_PROPAGATORS:
        raise ValueError(
            "Can not use an parabolic/hyperbolic propagator for elliptical orbits."
        )
    elif orbit.ecc == 1.0 and method not in PARABOLIC_PROPAGATORS:
        raise ValueError(
            "Can not use an elliptic/hyperbolic propagator for parabolic orbits."
        )
    elif orbit.ecc > 1.0 and method not in HYPERBOLIC_PROPAGATORS:
        raise ValueError(
            "Can not use an elliptic/parabolic propagator for hyperbolic orbits."
        )
    else:
        pass

    # Use the highest precision we can afford
    # np.atleast_1d does not work directly on TimeDelta objects
    jd1 = np.atleast_1d(time_of_flight.jd1)
    jd2 = np.atleast_1d(time_of_flight.jd2)
    time_of_flight = time.TimeDelta(jd1, jd2, format="jd", scale=time_of_flight.scale)

    rr, vv = method(
        orbit.attractor.k, orbit.r, orbit.v, time_of_flight.to(u.s), rtol=rtol, **kwargs
    )

    # TODO: Turn these into unit tests
    assert rr.ndim == 2
    assert vv.ndim == 2

    cartesian = CartesianRepresentation(
        rr, differentials=CartesianDifferential(vv, xyz_axis=1), xyz_axis=1
    )

    return cartesian


ELLIPTIC_PROPAGATORS = [
    mean_motion,
    kepler,
    mikkola,
    markley,
    pimienta,
    gooding,
    danby,
    cowell,
]
PARABOLIC_PROPAGATORS = [mean_motion, kepler, mikkola, pimienta, gooding, cowell]
HYPERBOLIC_PROPAGATORS = [
    mean_motion,
    kepler,
    mikkola,
    kepler,
    pimienta,
    gooding,
    danby,
    cowell,
]
ALL_PROPAGATORS = list(
    set(ELLIPTIC_PROPAGATORS) & set(PARABOLIC_PROPAGATORS) & set(HYPERBOLIC_PROPAGATORS)
)

# THIS IS THE TWO BODY Orbit.py File

In [41]:
# TwoBody Orbit.py

#from poliastro.bodies import Earth, Moon, Sun
#from poliastro.constants import J2000

#from poliastro.core.angles import nu_to_M as nu_to_M_fast

#from poliastro.frames import Planes, get_frame

#from poliastro.threebody.soi import laplace_radius


# THIS ONE I THINK I ALREADY INPUT - IT RAN
#from poliastro.twobody.angles import E_to_nu, M_to_nu, nu_to_M

#from poliastro.twobody.propagation import mean_motion, propagate


#from poliastro.util import (
#    find_closest_value,
#    get_eccentricity_critical_argp,
#    get_eccentricity_critical_inc,
#    get_inclination_critical_argp,
#    hyp_nu_limit,
#    norm,
#)

#from ._states import BaseState, ClassicalState, ModifiedEquinoctialState, RVState

ORBIT_FORMAT = "{r_p:.0f} x {r_a:.0f} x {inc:.1f} ({frame}) orbit around {body} at epoch {epoch} ({scale})"
# String representation for orbits around bodies without predefined
# reference frame
ORBIT_NO_FRAME_FORMAT = (
    "{r_p:.0f} x {r_a:.0f} x {inc:.1f} orbit around {body} at epoch {epoch} ({scale})"
)


class TimeScaleWarning(UserWarning):
    pass


class OrbitSamplingWarning(UserWarning):
    pass


class PatchedConicsWarning(UserWarning):
    pass


class Orbit(object):
    """Position and velocity of a body with respect to an attractor
    at a given time (epoch).
    Regardless of how the Orbit is created, the implicit
    reference system is an inertial one. For the specific case
    of the Solar System, this can be assumed to be the
    International Celestial Reference System or ICRS.
    """

    def __init__(self, state, epoch, plane):
        """Constructor.
        Parameters
        ----------
        state : BaseState
            Position and velocity or orbital elements.
        epoch : ~astropy.time.Time
            Epoch of the orbit.
        """
        self._state = state  # type: BaseState
        self._epoch = epoch  # type: time.Time
        self._plane = plane
        self._frame = None

    @property
    def attractor(self):
        """Main attractor. """
        return self._state.attractor

    @property
    def epoch(self):
        """Epoch of the orbit. """
        return self._epoch

    @property
    def plane(self):
        """Fundamental plane of the frame. """
        return self._plane

    @property
    def frame(self):
        """Reference frame of the orbit.
        .. versionadded:: 0.11.0
        """
        if self._frame is None:
            self._frame = get_frame(self.attractor, self._plane, self.epoch)

        return self._frame

    @property
    def r(self):
        """Position vector. """
        return self._state.to_vectors().r

    @property
    def v(self):
        """Velocity vector. """
        return self._state.to_vectors().v

    @property
    def a(self):
        """Semimajor axis. """
        return self.p / (1 - self.ecc ** 2)

    @property
    def p(self):
        """Semilatus rectum. """
        return self._state.to_classical().p

    @property
    def r_p(self):
        """Radius of pericenter. """
        return self.a * (1 - self.ecc)

    @property
    def r_a(self):
        """Radius of apocenter. """
        return self.a * (1 + self.ecc)

    @property
    def ecc(self):
        """Eccentricity. """
        return self._state.to_classical().ecc

    @property
    def inc(self):
        """Inclination. """
        return self._state.to_classical().inc

    @property
    def raan(self):
        """Right ascension of the ascending node. """
        return self._state.to_classical().raan

    @property
    def argp(self):
        """Argument of the perigee. """
        return self._state.to_classical().argp

    @property
    def nu(self):
        """True anomaly. """
        return self._state.to_classical().nu

    @property
    def f(self):
        """Second modified equinoctial element. """
        return self._state.to_equinoctial().f

    @property
    def g(self):
        """Third modified equinoctial element. """
        return self._state.to_equinoctial().g

    @property
    def h(self):
        """Fourth modified equinoctial element. """
        return self._state.to_equinoctial().h

    @property
    def k(self):
        """Fifth modified equinoctial element. """
        return self._state.to_equinoctial().k

    @property
    def L(self):
        """True longitude. """
        return self.raan + self.argp + self.nu

    @property
    def period(self):
        """Period of the orbit. """
        return 2 * np.pi * u.rad / self.n

    @property
    def n(self):
        """Mean motion. """
        return (np.sqrt(self.attractor.k / abs(self.a ** 3)) * u.rad).decompose()

    @property
    def energy(self):
        """Specific energy. """
        return self.v.dot(self.v) / 2 - self.attractor.k / np.sqrt(self.r.dot(self.r))

    @property
    def e_vec(self):
        """Eccentricity vector. """
        r, v = self.rv()
        k = self.attractor.k
        e_vec = ((v.dot(v) - k / (norm(r))) * r - r.dot(v) * v) / k
        return e_vec.decompose()

    @property
    def h_vec(self):
        """Specific angular momentum vector. """
        h_vec = np.cross(self.r.to(u.km).value, self.v.to(u.km / u.s)) * u.km ** 2 / u.s
        return h_vec

    @property
    def h_mag(self):
        """Specific angular momentum. """
        h_mag = norm(self.h_vec)
        return h_mag

    @property
    def arglat(self):
        """Argument of latitude. """
        arglat = (self.argp + self.nu) % (360 * u.deg)
        return arglat

    @property
    def M(self):
        """Mean anomaly. """
        return nu_to_M(self.nu, self.ecc)

    @property
    def t_p(self):
        """Elapsed time since latest perifocal passage. """
        t_p = self.period * self.M / (360 * u.deg)
        return t_p

    @classmethod
    @u.quantity_input(r=u.m, v=u.m / u.s)
    def from_vectors(cls, attractor, r, v, epoch=J2000, plane=Planes.EARTH_EQUATOR):
        """Return `Orbit` from position and velocity vectors.
        Parameters
        ----------
        attractor : Body
            Main attractor.
        r : ~astropy.units.Quantity
            Position vector wrt attractor center.
        v : ~astropy.units.Quantity
            Velocity vector.
        epoch : ~astropy.time.Time, optional
            Epoch, default to J2000.
        plane : ~poliastro.frames.Planes
            Fundamental plane of the frame.
        """
        assert np.any(r.value), "Position vector must be non zero"

        if r.ndim > 1 or v.ndim > 1:
            raise ValueError(
                f"Vectors must have dimension 1, got {r.ndim} and {v.ndim}"
            )

        ss = RVState(attractor, r, v)
        return cls(ss, epoch, plane)

    @classmethod
    def from_coords(cls, attractor, coord, plane=Planes.EARTH_EQUATOR):
        """Creates an `Orbit` from an attractor and astropy `SkyCoord`
        or `BaseCoordinateFrame` instance.
        This method accepts position
        and velocity in any reference frame unlike `Orbit.from_vector`
        which can accept inputs in only inertial reference frame centred
        at attractor. Also note that the frame information is lost after
        creation of the orbit and only the inertial reference frame at
        body centre will be used for all purposes.
        Parameters
        ----------
        attractor: Body
            Main attractor
        coord: astropy.coordinates.SkyCoord or BaseCoordinateFrame
            Position and velocity vectors in any reference frame. Note that coord must have
            a representation and its differential with respect to time.
        plane : ~poliastro.frames.Planes, optional
            Final orbit plane, default to Earth Equator.
        """
        if "s" not in coord.cartesian.differentials:
            raise ValueError(
                "Coordinate instance doesn't have a differential with respect to time"
            )
        if coord.size != 1:
            raise ValueError(
                "Coordinate instance must represents exactly 1 position, found: %d"
                % coord.size
            )

        # Reshape coordinate to 0 dimension if it is not already dimensionless.
        coord = coord.reshape(())

        # Get an inertial reference frame parallel to ICRS and centered at
        # attractor
        inertial_frame_at_body_centre = get_frame(attractor, plane, coord.obstime)

        if not coord.is_equivalent_frame(inertial_frame_at_body_centre):
            coord_in_irf = coord.transform_to(inertial_frame_at_body_centre)
        else:
            coord_in_irf = coord

        pos = coord_in_irf.cartesian.xyz
        vel = coord_in_irf.cartesian.differentials["s"].d_xyz

        return cls.from_vectors(attractor, pos, vel, epoch=coord.obstime, plane=plane)

    @classmethod
    @u.quantity_input(a=u.m, ecc=u.one, inc=u.rad, raan=u.rad, argp=u.rad, nu=u.rad)
    def from_classical(
        cls,
        attractor,
        a,
        ecc,
        inc,
        raan,
        argp,
        nu,
        epoch=J2000,
        plane=Planes.EARTH_EQUATOR,
    ):
        """Return `Orbit` from classical orbital elements.
        Parameters
        ----------
        attractor : Body
            Main attractor.
        a : ~astropy.units.Quantity
            Semi-major axis.
        ecc : ~astropy.units.Quantity
            Eccentricity.
        inc : ~astropy.units.Quantity
            Inclination
        raan : ~astropy.units.Quantity
            Right ascension of the ascending node.
        argp : ~astropy.units.Quantity
            Argument of the pericenter.
        nu : ~astropy.units.Quantity
            True anomaly.
        epoch : ~astropy.time.Time, optional
            Epoch, default to J2000.
        plane : ~poliastro.frames.Planes
            Fundamental plane of the frame.
        """
        if ecc == 1.0 * u.one:
            raise ValueError("For parabolic orbits use Orbit.parabolic instead")

        if not 0 * u.deg <= inc <= 180 * u.deg:
            raise ValueError("Inclination must be between 0 and 180 degrees")

        if ecc > 1 and a > 0:
            raise ValueError("Hyperbolic orbits have negative semimajor axis")

        ss = ClassicalState(attractor, a * (1 - ecc ** 2), ecc, inc, raan, argp, nu)
        return cls(ss, epoch, plane)

    @classmethod
    @u.quantity_input(p=u.m, f=u.one, g=u.rad, h=u.rad, k=u.rad, L=u.rad)
    def from_equinoctial(
        cls, attractor, p, f, g, h, k, L, epoch=J2000, plane=Planes.EARTH_EQUATOR
    ):
        """Return `Orbit` from modified equinoctial elements.
        Parameters
        ----------
        attractor : Body
            Main attractor.
        p : ~astropy.units.Quantity
            Semilatus rectum.
        f : ~astropy.units.Quantity
            Second modified equinoctial element.
        g : ~astropy.units.Quantity
            Third modified equinoctial element.
        h : ~astropy.units.Quantity
            Fourth modified equinoctial element.
        k : ~astropy.units.Quantity
            Fifth modified equinoctial element.
        L : ~astropy.units.Quantity
            True longitude.
        epoch : ~astropy.time.Time, optional
            Epoch, default to J2000.
        plane : ~poliastro.frames.Planes
            Fundamental plane of the frame.
        """
        ss = ModifiedEquinoctialState(attractor, p, f, g, h, k, L)
        return cls(ss, epoch, plane)

    @classmethod
    def from_body_ephem(cls, body, epoch=None):
        """Return osculating `Orbit` of a body at a given time.
        """
        # TODO: https://github.com/poliastro/poliastro/issues/445
        if not epoch:
            epoch = time.Time.now().tdb
        elif epoch.scale != "tdb":
            epoch = epoch.tdb
            warn(
                "Input time was converted to scale='tdb' with value "
                f"{epoch.tdb.value}. Use Time(..., scale='tdb') instead.",
                TimeScaleWarning,
            )
        try:
            r, v = get_body_barycentric_posvel(body.name, epoch)
        except KeyError as exc:
            raise RuntimeError(
                """To compute the position and velocity of the Moon and Pluto use the JPL ephemeris:
>>> from astropy.coordinates import solar_system_ephemeris
>>> solar_system_ephemeris.set('jpl')
"""
            ) from exc
        if body == Moon:
            # TODO: The attractor is in fact the Earth-Moon Barycenter
            icrs_cart = r.with_differentials(v.represent_as(CartesianDifferential))
            gcrs_cart = (
                ICRS(icrs_cart)
                .transform_to(GCRS(obstime=epoch))
                .represent_as(CartesianRepresentation)
            )
            ss = cls.from_vectors(
                Earth,
                gcrs_cart.xyz.to(u.km),
                gcrs_cart.differentials["s"].d_xyz.to(u.km / u.day),
                epoch,
            )

        else:
            # TODO: The attractor is not really the Sun, but the Solar System
            # Barycenter
            ss = cls.from_vectors(Sun, r.xyz.to(u.km), v.xyz.to(u.km / u.day), epoch)
            ss._frame = ICRS()  # Hack!

        return ss

    def change_attractor(self, new_attractor, force=False):
        """Changes orbit attractor.
        Only changes from attractor to parent or the other way around are allowed.
        Parameters
        ----------
        new_attractor: poliastro.bodies.Body
            Desired new attractor.
        force: boolean
            If `True`, changes attractor even if physically has no-sense.
        Returns
        -------
        ss: poliastro.twobody.orbit.Orbit
            Orbit with new attractor
        """
        if self.attractor == new_attractor:
            return self
        elif self.attractor == new_attractor.parent:  # "Sun -> Earth"
            r_soi = laplace_radius(new_attractor)
            distance = norm(
                self.r - get_body_barycentric(new_attractor.name, self.epoch).xyz
            )
        elif self.attractor.parent == new_attractor:  # "Earth -> Sun"
            r_soi = laplace_radius(self.attractor)
            distance = norm(self.r)
        else:
            raise ValueError("Cannot change to unrelated attractor")

        if distance > r_soi and not force:
            raise ValueError(
                "Orbit is out of new attractor's SOI. If required, use 'force=True'."
            )
        elif self.ecc < 1.0 and not force:
            raise ValueError("Orbit will never leave the SOI of its current attractor")
        else:
            warn("Leaving the SOI of the current attractor", PatchedConicsWarning)

        new_frame = get_frame(new_attractor, self.plane, obstime=self.epoch)
        coords = self.frame.realize_frame(self.represent_as(CartesianRepresentation))
        ss = Orbit.from_coords(new_attractor, coords.transform_to(new_frame))

        return ss

    @classmethod
    def from_horizons(
        cls, name, epoch=None, plane=Planes.EARTH_EQUATOR, id_type="smallbody"
    ):
        """Return osculating `Orbit` of a body using JPLHorizons module of Astroquery.
        Parameters
        ----------
        name : string
            Name of the body to query for.
        epoch : ~astropy.time.Time, optional
            Epoch, default to None.
        plane : ~poliastro.frames.Planes
            Fundamental plane of the frame.
        id_type : string, optional
            Use "smallbody" for Asteroids and Comets, and "majorbody"
            for Planets and Satellites.
        """
        if not epoch:
            epoch = time.Time.now()
        if plane == Planes.EARTH_EQUATOR:
            refplane = "earth"
        elif plane == Planes.EARTH_ECLIPTIC:
            refplane = "ecliptic"

        obj = Horizons(id=name, epochs=epoch.jd, id_type=id_type).elements(
            refplane=refplane
        )
        a = obj["a"][0] * u.au
        ecc = obj["e"][0] * u.one
        inc = obj["incl"][0] * u.deg
        raan = obj["Omega"][0] * u.deg
        argp = obj["w"][0] * u.deg
        nu = obj["nu"][0] * u.deg
        ss = cls.from_classical(
            Sun, a, ecc, inc, raan, argp, nu, epoch=epoch.tdb, plane=plane
        )
        return ss

    @classmethod
    def from_sbdb(cls, name, **kargs):
        """Return osculating `Orbit` by using `SBDB` from Astroquery.
        Parameters
        ----------
        body_name: string
            Name of the body to make the request.
        Returns
        -------
        ss: poliastro.twobody.orbit.Orbit
            Orbit corresponding to body_name
        Examples
        --------
        >>> from poliastro.twobody.orbit import Orbit
        >>> apophis_orbit = Orbit.from_sbdb('apophis')  # doctest: +REMOTE_DATA
        """

        obj = SBDB.query(name, full_precision=True, **kargs)

        if "count" in obj:
            # no error till now ---> more than one object has been found
            # contains all the name of the objects
            objects_name = obj["list"]["name"]
            objects_name_in_str = (
                ""
            )  # used to store them in string form each in new line
            for i in objects_name:
                objects_name_in_str += i + "\n"

            raise ValueError(
                str(obj["count"]) + " different objects found: \n" + objects_name_in_str
            )

        a = obj["orbit"]["elements"]["a"].to(u.AU) * u.AU
        ecc = float(obj["orbit"]["elements"]["e"]) * u.one
        inc = obj["orbit"]["elements"]["i"].to(u.deg) * u.deg
        raan = obj["orbit"]["elements"]["om"].to(u.deg) * u.deg
        argp = obj["orbit"]["elements"]["w"].to(u.deg) * u.deg

        # Since JPL provides Mean Anomaly (M) we need to make
        # the conversion to the true anomaly (\nu)
        nu = M_to_nu(obj["orbit"]["elements"]["ma"].to(u.deg) * u.deg, ecc)

        epoch = time.Time(obj["orbit"]["epoch"].to(u.d), format="jd")

        ss = cls.from_classical(
            Sun,
            a,
            ecc,
            inc,
            raan,
            argp,
            nu,
            epoch=epoch.tdb,
            plane=Planes.EARTH_ECLIPTIC,
        )

        return ss

    @classmethod
    @u.quantity_input(alt=u.m, inc=u.rad, raan=u.rad, arglat=u.rad)
    def circular(
        cls,
        attractor,
        alt,
        inc=0 * u.deg,
        raan=0 * u.deg,
        arglat=0 * u.deg,
        epoch=J2000,
        plane=Planes.EARTH_EQUATOR,
    ):
        """Return circular `Orbit`.
        Parameters
        ----------
        attractor : Body
            Main attractor.
        alt : ~astropy.units.Quantity
            Altitude over surface.
        inc : ~astropy.units.Quantity, optional
            Inclination, default to 0 deg (equatorial orbit).
        raan : ~astropy.units.Quantity, optional
            Right ascension of the ascending node, default to 0 deg.
        arglat : ~astropy.units.Quantity, optional
            Argument of latitude, default to 0 deg.
        epoch: ~astropy.time.Time, optional
            Epoch, default to J2000.
        plane : ~poliastro.frames.Planes
            Fundamental plane of the frame.
        """
        a = attractor.R + alt
        ecc = 0 * u.one
        argp = 0 * u.deg

        return cls.from_classical(
            attractor, a, ecc, inc, raan, argp, arglat, epoch, plane
        )

    @classmethod
    @u.quantity_input(angular_velocity=u.rad / u.s, period=u.s, hill_radius=u.m)
    def geostationary(
        cls, attractor, angular_velocity=None, period=None, hill_radius=None
    ):
        """Return the geostationary orbit for the given attractor and its rotational speed.
        Parameters
        ----------
        attractor : Body
            Main attractor.
        angular_velocity : ~astropy.units.Quantity
            Rotational angular velocity of the attractor.
        period : ~astropy.units.Quantity
            Attractor's rotational period, ignored if angular_velocity is passed.
        hill_radius : ~astropy.units.Quantity
            Radius of Hill sphere of the attractor (optional). Hill sphere radius(in
            contrast with Laplace's SOI) is used here to validate the stability of the
            geostationary orbit, that is to make sure that the orbital radius required
            for the geostationary orbit is not outside of the gravitational sphere of
            influence of the attractor.
            Hill SOI of parent(if exists) of the attractor is ignored if hill_radius is not provided.
        """

        if angular_velocity is None and period is None:
            raise ValueError(
                "At least one among angular_velocity or period must be passed"
            )

        if angular_velocity is None:
            angular_velocity = 2 * np.pi / period

        # Find out geostationary radius using r = cube_root(GM/(angular
        # velocity)^2)
        with u.set_enabled_equivalencies(u.dimensionless_angles()):
            geo_radius = np.cbrt(attractor.k / np.square(angular_velocity.to(1 / u.s)))

        if hill_radius is not None and geo_radius > hill_radius:
            raise ValueError(
                "Geostationary orbit for the given parameters doesn't exist"
            )

        altitude = geo_radius - attractor.R
        return cls.circular(attractor, altitude)

    @classmethod
    @u.quantity_input(p=u.m, inc=u.rad, raan=u.rad, argp=u.rad, nu=u.rad)
    def parabolic(
        cls, attractor, p, inc, raan, argp, nu, epoch=J2000, plane=Planes.EARTH_EQUATOR
    ):
        """Return parabolic `Orbit`.
        Parameters
        ----------
        attractor : Body
            Main attractor.
        p : ~astropy.units.Quantity
            Semilatus rectum or parameter.
        inc : ~astropy.units.Quantity, optional
            Inclination.
        raan : ~astropy.units.Quantity
            Right ascension of the ascending node.
        argp : ~astropy.units.Quantity
            Argument of the pericenter.
        nu : ~astropy.units.Quantity
            True anomaly.
        epoch: ~astropy.time.Time, optional
            Epoch, default to J2000.
        plane : ~poliastro.frames.Planes
            Fundamental plane of the frame.
        """
        ss = ClassicalState(attractor, p, 1.0 * u.one, inc, raan, argp, nu)
        return cls(ss, epoch, plane)

    @classmethod
    @u.quantity_input(
        alt=u.m, inc=u.rad, argp=u.rad, raan=u.rad, arglat=u.rad, ecc=u.one
    )
    def frozen(
        cls,
        attractor,
        alt,
        inc=None,
        argp=None,
        raan=0 * u.deg,
        arglat=0 * u.deg,
        ecc=None,
        epoch=J2000,
        plane=Planes.EARTH_EQUATOR,
    ):
        r"""Return frozen Orbit. If any of the given arguments results in an impossibility, some values will be overwritten
        To achieve frozen orbit these two equations have to be set to zero.
        .. math::
            \dfrac {d\overline {e}}{dt}=\dfrac {-3\overline {n}J_{3}R^{3}_{E}\sin \left( \overline {i}\right) }{2a^{3}\left( 1-\overline {e}^{2}\right) ^{2}}\left( 1-\dfrac {5}{4}\sin ^{2}\overline {i}\right) \cos \overline {w}
        .. math::
            \dfrac {d\overline {\omega }}{dt}=\dfrac {3\overline {n}J_{2}R^{2}_{E}}{a^{2}\left( 1-\overline {e}^{2}\right) ^{2}}\left( 1-\dfrac {5}{4}\sin ^{2}\overline {i}\right) \left[ 1+\dfrac {J_{3}R_{E}}{2J_{2}\overline {a}\left( 1-\overline {e}^{2}\right) }\left( \dfrac {\sin ^{2}\overline {i}-\overline {e}\cos ^{2}\overline {i}}{\sin \overline {i}}\right) \dfrac {\sin \overline {w}}{\overline {e}}\right]
        The first approach would be to nullify following term to zero:
        .. math::
            ( 1-\dfrac {5}{4}\sin ^{2})
        For which one obtains the so-called critical inclinations: i = 63.4349 or 116.5651 degrees. To escape the inclination requirement, the argument of periapsis can be set to w = 90 or 270 degrees to nullify the second equation. Then, one should nullify the right-hand side of the first equation, which yields an expression that correlates the inclination of the object and the eccentricity of the orbit:
        .. math::
            \overline {e}=-\dfrac {J_{3}R_{E}}{2J_{2}\overline {a}\left( 1-\overline {e}^{2}\right) }\left( \dfrac {\sin ^{2}\overline {i}-\overline {e}\cos ^{2} \overline {i}}{\sin \overline {i}}\right)
        Assuming that e is negligible compared to J2, it can be shown that:
        .. math::
            \overline {e}\approx -\dfrac {J_{3}R_{E}}{2J_{2}\overline {a}}\sin \overline {i}
        The implementation is divided in the following cases:
            1. When the user gives a negative altitude, the method will raise a ValueError
            2. When the attractor has not defined J2 or J3, the method will raise an AttributeError
            3. When the attractor has J2/J3 outside of range 1 to 10 , the method will raise an NotImplementedError. Special case for Venus.See "Extension of the critical inclination" by Xiaodong Liu, Hexi Baoyin, and Xingrui Ma
            4. If argp is not given or the given argp is a critical value:
                * if eccentricity is none and inclination is none, the inclination is set with a critical value and the eccentricity is obtained from the last formula mentioned
                * if only eccentricity is none, we calculate this value with the last formula mentioned
                * if only inclination is none ,we calculate this value with the formula for eccentricity with critical argp.
            5. If inc is not given or the given inc is critical:
                * if the argp and the eccentricity is given we keep these values to create the orbit
                * if the eccentricity is given we keep this value, if not, default to the eccentricity of the Moon's orbit around the Earth
            6. if it's not possible to create an orbit with the the argp and the inclination given, both of them are set to the critical values and the eccentricity is calculate with the last formula
        Parameters
        ----------
        attractor : Body
            Main attractor.
        alt : ~astropy.units.Quantity
            Altitude over surface.
        inc : ~astropy.units.Quantity, optional
            Inclination, default to critical value.
        argp : ~astropy.units.Quantity, optional
            Argument of the pericenter, default to critical value.
        raan : ~astropy.units.Quantity, optional
            Right ascension of the ascending node, default to 0 deg.
        arglat : ~astropy.units.Quantity, optional
            Argument of latitude, default to 0 deg.
        ecc : ~astropy.units.Quantity
            Eccentricity, default to the eccentricity of the Moon's orbit around the Earth
        epoch: ~astropy.time.Time, optional
            Epoch, default to J2000.
        plane : ~poliastro.frames.Planes
            Fundamental plane of the frame.
        """
        if attractor.J2 == 0.0 or attractor.J3 == 0.0:
            raise AttributeError(
                f"Attractor {attractor.name} has not spherical harmonics implemented"
            )

        critical_argps = [np.pi / 2, 3 * np.pi / 2] * u.rad
        critical_inclinations = [63.4349 * np.pi / 180, 116.5651 * np.pi / 180] * u.rad
        try:
            if 1 <= np.abs(attractor.J2 / attractor.J3) <= 10:
                raise NotImplementedError(
                    f"This has not been implemented for {attractor.name}"
                )

            assert alt > 0

            argp = critical_argps[0] if argp is None else argp
            a = attractor.R + alt

            critical_argp = find_closest_value(argp, critical_argps)
            if np.isclose(argp, critical_argp, 1e-8, 1e-5 * u.rad):
                if inc is None and ecc is None:
                    inc = critical_inclinations[0]
                    ecc = get_eccentricity_critical_argp(attractor, a, inc)
                elif ecc is None:
                    ecc = get_eccentricity_critical_argp(attractor, a, inc)
                else:
                    inc = get_inclination_critical_argp(attractor, a, ecc)
                return cls.from_classical(
                    attractor, a, ecc, inc, raan, argp, arglat, epoch, plane
                )

            inc = critical_inclinations[0] if inc is None else inc
            critical_inclination = find_closest_value(inc, critical_inclinations)
            if np.isclose(inc, critical_inclination, 1e-8, 1e-5 * u.rad):
                ecc = get_eccentricity_critical_inc(ecc)
                return cls.from_classical(
                    attractor, a, ecc, inc, raan, argp, arglat, epoch, plane
                )

            argp = critical_argps[0]
            inc = critical_inclinations[0]
            ecc = get_eccentricity_critical_argp(attractor, a, inc)

            return cls.from_classical(
                attractor, a, ecc, inc, raan, argp, arglat, epoch, plane
            )

        except AssertionError as exc:
            raise ValueError(
                f"The semimajor axis may not be smaller that {attractor.name}'s radius"
            ) from exc

    def represent_as(self, representation):
        """Converts the orbit to a specific representation.
        .. versionadded:: 0.11.0
        Parameters
        ----------
        representation : ~astropy.coordinates.BaseRepresentation
            Representation object to use. It must be a class, not an instance.
        Examples
        --------
        >>> from poliastro.examples import iss
        >>> from astropy.coordinates import CartesianRepresentation, SphericalRepresentation
        >>> iss.represent_as(CartesianRepresentation)
        <CartesianRepresentation (x, y, z) in km
            (859.07256, -4137.20368, 5295.56871)
         (has differentials w.r.t.: 's')>
        >>> iss.represent_as(CartesianRepresentation).xyz
        <Quantity [  859.07256, -4137.20368,  5295.56871] km>
        >>> iss.represent_as(CartesianRepresentation).differentials['s']
        <CartesianDifferential (d_x, d_y, d_z) in km / s
            (7.37289205, 2.08223573, 0.43999979)>
        >>> iss.represent_as(CartesianRepresentation).differentials['s'].d_xyz
        <Quantity [7.37289205, 2.08223573, 0.43999979] km / s>
        >>> iss.represent_as(SphericalRepresentation)
        <SphericalRepresentation (lon, lat, distance) in (rad, rad, km)
            (4.91712525, 0.89732339, 6774.76995296)
         (has differentials w.r.t.: 's')>
        """
        # As we do not know the differentials, we first convert to cartesian,
        # then let the frame represent_as do the rest
        # TODO: Perhaps this should be public API as well?
        cartesian = CartesianRepresentation(
            *self.r, differentials=CartesianDifferential(*self.v)
        )
        # See the propagate function for reasoning about the usage of a
        # protected method
        coords = self.frame._replicate(cartesian, representation_type="cartesian")

        return coords.represent_as(representation)

    def to_icrs(self):
        """Creates a new Orbit object with its coordinates transformed to ICRS.
        Notice that, strictly speaking, the center of ICRS is the Solar System Barycenter
        and not the Sun, and therefore these orbits cannot be propagated in the context
        of the two body problem. Therefore, this function exists merely for practical
        purposes.
        .. versionadded:: 0.11.0
        """
        coords = self.frame.realize_frame(self.represent_as(CartesianRepresentation))
        coords.representation_type = CartesianRepresentation

        icrs_cart = coords.transform_to(ICRS).represent_as(CartesianRepresentation)

        # TODO: The attractor is in fact the Solar System Barycenter
        ss = self.from_vectors(
            Sun, r=icrs_cart.xyz, v=icrs_cart.differentials["s"].d_xyz, epoch=self.epoch
        )
        ss._frame = ICRS()  # Hack!
        return ss

    def rv(self):
        """Position and velocity vectors. """
        return self.r, self.v

    def classical(self):
        """Classical orbital elements. """
        return (
            self.a,
            self.ecc,
            self.inc.to(u.deg),
            self.raan.to(u.deg),
            self.argp.to(u.deg),
            self.nu.to(u.deg),
        )

    def pqw(self):
        """Perifocal frame (PQW) vectors. """
        if self.ecc < 1e-8:
            if abs(self.inc.to(u.rad).value) > 1e-8:
                node = np.cross([0, 0, 1], self.h_vec) / norm(self.h_vec)
                p_vec = node / norm(node)  # Circular inclined
            else:
                p_vec = [1, 0, 0] * u.one  # Circular equatorial
        else:
            p_vec = self.e_vec / self.ecc
        w_vec = self.h_vec / norm(self.h_vec)
        q_vec = np.cross(w_vec, p_vec) * u.one
        return p_vec, q_vec, w_vec

    def __str__(self):
        if self.a > 1e7 * u.km:
            unit = u.au
        else:
            unit = u.km

        try:
            return ORBIT_FORMAT.format(
                r_p=self.r_p.to(unit).value,
                r_a=self.r_a.to(unit),
                inc=self.inc.to(u.deg),
                frame=self.frame.__class__.__name__,
                body=self.attractor,
                epoch=self.epoch,
                scale=self.epoch.scale.upper(),
            )
        except NotImplementedError:
            return ORBIT_NO_FRAME_FORMAT.format(
                r_p=self.r_p.to(unit).value,
                r_a=self.r_a.to(unit),
                inc=self.inc.to(u.deg),
                body=self.attractor,
                epoch=self.epoch,
                scale=self.epoch.scale.upper(),
            )

    def __repr__(self):
        return self.__str__()

    def propagate(self, value, method=mean_motion, rtol=1e-10, **kwargs):
        """Propagates an orbit a specified time.
        If value is true anomaly, propagate orbit to this anomaly and return the result.
        Otherwise, if time is provided, propagate this `Orbit` some `time` and return the result.
        Parameters
        ----------
        value : ~astropy.units.Quantity, ~astropy.time.Time, ~astropy.time.TimeDelta
            Scalar time to propagate.
        rtol : float, optional
            Relative tolerance for the propagation algorithm, default to 1e-10.
        method : function, optional
            Method used for propagation
        **kwargs
            parameters used in perturbation models
        Returns
        -------
        Orbit
            New orbit after propagation.
        """
        if isinstance(value, time.Time) and not isinstance(value, time.TimeDelta):
            time_of_flight = value - self.epoch
        else:
            # Works for both Quantity and TimeDelta objects
            time_of_flight = time.TimeDelta(value)

        cartesian = propagate(self, time_of_flight, method=method, rtol=rtol, **kwargs)
        new_epoch = self.epoch + time_of_flight

        # TODO: Unify with sample
        # If the frame supports obstime, set the time values
        try:
            kwargs = {}
            if "obstime" in self.frame.frame_attributes:
                kwargs["obstime"] = new_epoch

            # Use of a protected method instead of frame.realize_frame
            # because the latter does not let the user choose the representation type
            # in one line despite its parameter names, see
            # https://github.com/astropy/astropy/issues/7784
            coords = self.frame._replicate(
                cartesian, representation_type="cartesian", **kwargs
            )

            return self.from_coords(self.attractor, coords, plane=self.plane)

        except NotImplementedError:
            return self.from_vectors(
                self.attractor,
                cartesian[0].xyz,
                cartesian[0].differentials["s"].d_xyz,
                new_epoch,
            )

    @u.quantity_input(value=u.rad)
    def time_to_anomaly(self, value):
        """ Returns time required to be in a specific true anomaly.
        Parameters
        ----------
        value : ~astropy.units.Quantity
        Returns
        -------
        tof: ~astropy.units.Quantity
            Time of flight required.
        """

        # Compute time of flight for correct epoch
        M = nu_to_M(self.nu, self.ecc)
        new_M = nu_to_M(value, self.ecc)
        tof = Angle(new_M - M).wrap_at(360 * u.deg) / self.n

        return tof

    @u.quantity_input(value=u.rad)
    def propagate_to_anomaly(self, value):
        """Propagates an orbit to a specific true anomaly.
        Parameters
        ----------
        value : ~astropy.units.Quantity
        Returns
        -------
        Orbit
            Resulting orbit after propagation.
        """

        # Compute time of flight for correct epoch
        time_of_flight = self.time_to_anomaly(value)

        return self.from_classical(
            self.attractor,
            self.a,
            self.ecc,
            self.inc,
            self.raan,
            self.argp,
            value,
            epoch=self.epoch + time_of_flight,
            plane=self.plane,
        )

    def _sample_closed(self, values, min_anomaly, max_anomaly):
        min_anomaly = (
            min_anomaly.to(u.rad).value
            if min_anomaly is not None
            else self.nu.to(u.rad).value
        )
        max_anomaly = (
            max_anomaly.to(u.rad).value
            if max_anomaly is not None
            else self.nu.to(u.rad).value + 2 * np.pi
        )
        limits = [min_anomaly, max_anomaly] * u.rad

        # First sample eccentric anomaly, then transform into true anomaly
        # to minimize error in the apocenter, see
        # http://www.dtic.mil/dtic/tr/fulltext/u2/a605040.pdf
        # Start from pericenter
        E_values = np.linspace(*limits, values)
        nu_values = E_to_nu(E_values, self.ecc)
        return nu_values

    def _sample_open(self, values, min_anomaly, max_anomaly):
        # Select a sensible limiting value for non-closed orbits
        # This corresponds to max(r = 3p, r = self.r)
        # We have to wrap nu in [-180, 180) to compare it with the output of
        # the arc cosine, which is in the range [0, 180)
        # Start from -nu_limit
        wrapped_nu = Angle(self.nu).wrap_at(180 * u.deg)
        nu_limit = max(hyp_nu_limit(self.ecc, 3.0), abs(wrapped_nu)).to(u.rad).value

        limits = [
            min_anomaly.to(u.rad).value if min_anomaly is not None else -nu_limit,
            max_anomaly.to(u.rad).value if max_anomaly is not None else nu_limit,
        ] * u.rad  # type: u.Quantity

        # Now we check that none of the provided values
        # is outside of the hyperbolic range
        nu_max = hyp_nu_limit(self.ecc) - 1e-3 * u.rad  # Arbitrary delta
        if not Angle(limits).is_within_bounds(-nu_max, nu_max):
            warn("anomaly outside range, clipping", OrbitSamplingWarning)
            limits = limits.clip(-nu_max, nu_max)

        nu_values = np.linspace(*limits, values)
        return nu_values

    def sample(self, values=100, *, min_anomaly=None, max_anomaly=None):
        r"""Samples an orbit to some specified time values.
        .. versionadded:: 0.8.0
        Parameters
        ----------
        values : int
            Number of interval points (default to 100).
        min_anomaly, max_anomaly : ~astropy.units.Quantity, optional
            Anomaly limits to sample the orbit.
            For elliptic orbits the default will be :math:`E \in \left[0, 2 \pi \right]`,
            and for hyperbolic orbits it will be :math:`\nu \in \left[-\nu_c, \nu_c \right]`,
            where :math:`\nu_c` is either the current true anomaly
            or a value that corresponds to :math:`r = 3p`.
        Returns
        -------
        positions: ~astropy.coordinates.BaseCoordinateFrame
            Array of x, y, z positions, with proper times as the frame attributes if supported.
        Notes
        -----
        When specifying a number of points, the initial and final
        position is present twice inside the result (first and
        last row). This is more useful for plotting.
        Examples
        --------
        >>> from astropy import units as u
        >>> from poliastro.examples import iss
        >>> iss.sample()  # doctest: +ELLIPSIS
        <GCRS Coordinate ...>
        >>> iss.sample(10)  # doctest: +ELLIPSIS
        <GCRS Coordinate ...>
        """
        if self.ecc < 1:
            nu_values = self._sample_closed(values, min_anomaly, max_anomaly)
        else:
            nu_values = self._sample_open(values, min_anomaly, max_anomaly)

        time_values = time.TimeDelta(self._generate_time_values(nu_values))
        cartesian = propagate(self, time_values)

        # TODO: Unify with propagate
        # If the frame supports obstime, set the time values
        try:
            kwargs = {}
            if "obstime" in self.frame.frame_attributes:
                kwargs["obstime"] = self.epoch + time_values
            else:
                warn(
                    f"Frame {self.frame.__class__} does not support 'obstime', time values were not returned"
                )

            # Use of a protected method instead of frame.realize_frame
            # because the latter does not let the user choose the representation type
            # in one line despite its parameter names, see
            # https://github.com/astropy/astropy/issues/7784
            coords = self.frame._replicate(
                cartesian, representation_type="cartesian", **kwargs
            )
            return coords

        except NotImplementedError:
            warn(
                f"No frame found for attractor {self.attractor}, returning only cartesian coordinates instead"
            )
            return cartesian

    def _generate_time_values(self, nu_vals):
        # Subtract current anomaly to start from the desired point
        ecc = self.ecc.value
        nu = self.nu.to(u.rad).value

        M_vals = [
            nu_to_M_fast(nu_val, ecc) - nu_to_M_fast(nu, ecc)
            for nu_val in nu_vals.to(u.rad).value
        ] * u.rad

        time_values = (M_vals / self.n).decompose()
        return time_values

    def apply_maneuver(self, maneuver, intermediate=False):
        """Returns resulting `Orbit` after applying maneuver to self.
        Optionally return intermediate states (default to False).
        Parameters
        ----------
        maneuver : Maneuver
            Maneuver to apply.
        intermediate : bool, optional
            Return intermediate states, default to False.
        """
        orbit_new = self  # Initialize
        states = []
        attractor = self.attractor
        for delta_t, delta_v in maneuver:
            if not delta_t == 0 * u.s:
                orbit_new = orbit_new.propagate(delta_t)
            r, v = orbit_new.rv()
            vnew = v + delta_v
            orbit_new = self.from_vectors(attractor, r, vnew, orbit_new.epoch)
            states.append(orbit_new)
        if intermediate:
            res = states  # type: Union[Orbit, List[Orbit]]
        else:
            res = orbit_new
        return res

    def plot(self, label=None, use_3d=False, interactive=False):
        """Plots the orbit as an interactive widget.
        Parameters
        ----------
        label : str, optional
            Label for the orbit, defaults to empty.
        use_3d : bool, optional
            Produce a 3D plot, default to False.
        interactive : bool, optional
            Produce an interactive (rather than static) image of the orbit, default to False.
            This option requires Plotly properly installed and configured for your environment.
        """
        if not interactive and use_3d:
            raise ValueError(
                "The static plotter does not support 3D, use `interactive=True`"
            )
        elif not interactive:
            from poliastro.plotting.static import StaticOrbitPlotter

            return StaticOrbitPlotter().plot(self, label=label)
        elif use_3d:
            from poliastro.plotting.core import OrbitPlotter3D

            return OrbitPlotter3D().plot(self, label=label)
        else:
            from poliastro.plotting.core import OrbitPlotter2D

            return OrbitPlotter2D().plot(self, label=label)

In [42]:
date_launch = time.Time("2011-11-26 15:02", scale="utc")
date_arrival = time.Time("2012-08-06 05:17", scale="utc")

ss_earth = Orbit.from_body_ephem(Earth, date_launch)
ss_mars = Orbit.from_body_ephem(Mars, date_arrival)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:394: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2011-11-26 15:03:06.183. Use Time(..., scale='tdb') instead.

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:394: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2012-08-06 05:18:07.183. Use Time(..., scale='tdb') instead.



In [43]:


plotter = OrbitPlotter2D()
plotter.plot(ss_earth, label="Earth at launch position", color="navy")
plotter.plot(ss_mars, label="Mars at arrival position", color="red")
plotter.plot_trajectory(ss_trans.sample(max_anomaly=180*u.deg).cartesian, color="black", label="Transfer orbit")

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1mUntyped global name 'cross':[0m [1m[1mcannot determine Numba type of <class 'function'>[0m
[1m
File "<ipython-input-28-da5c01c928bc>", line 277:[0m
[1mdef rv2coe(k, r, v, tol=1e-8):
    <source elided>

[1m    h = cross(r, v)
[0m    [1m^[0m[0m
[0m
This is not usually a problem with Numba itself but instead often caused by
the use of unsupported features or an issue in resolving types.

To see Python/NumPy features supported by the latest release of Numba visit:
http://numba.pydata.org/numba-doc/latest/reference/pysupported.html
and
http://numba.pydata.org/numba-doc/latest/reference/numpysupported.html

For more information about typing errors and how to debug them visit:
http://numba.pydata.org/numba-doc/latest/user/troubleshoot.html#my-code-doesn-t-compile

If you think your code should work with Numba, please report the error message
and traceback, along with a minimal reproducer at:
https://gith